<a href="https://colab.research.google.com/github/LastPudding/Text_Mining_LSTM/blob/main/TextMiningCoursework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import spacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Concatenate, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


  # Load spaCy English tokenizer
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [3]:
!ls
#Check GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

'batch_size=8;epoch=14;num_NN=128;num_Dense=64'   dev.txt	  test.sent	  train.sent
 dev.pointer					  relations.txt   test.tup	  train.tup
 dev.sent					  sample_data	  test.txt	  train.txt
 dev.tup					  test.pointer	  train.pointer
Num GPUs Available:  1


# Some util function definition

In [4]:

#Function to extract Annotations of Sentences
def extract_relations(file_path):
  result=[]
  with open(file_path, 'r') as file:
    for each_line in file:
      replaced_string = each_line.replace(' ', '_').replace('/', '_or_').replace('\n', '')
      result.append(replaced_string)
  return result


def extract_triplets(file_path):
  triplets = []
  with open(file_path, 'r') as file:
      for line in file:
          line = line.strip()  # Remove leading/trailing whitespace
          line=line.replace('/', 'or').replace('\n', '')
          if line:  # Check if the line is not empty
              # Split the line into individual triplets using the delimiter "|"
              individual_triplets = line.split('|')
              triplet_list = []
              # Split each individual triplet into entities and relation
              for triplet in individual_triplets:
                  triplet_parts = triplet.strip().split(';')
                  if len(triplet_parts) == 3:
                      subject, related_entity, relation = [part.strip() for part in triplet_parts]
                      triplet_list.append((subject, related_entity, relation))
              triplets.append(triplet_list)
  return triplets

def read_document(file_path):
      with open(file_path, 'r') as file:
          lines = [line.strip() for line in file]
      return lines

  # Function to tokenize sentences
def tokenize_sentences(sentences):
      tokenized_sentences = []
      for sentence in sentences:
          tokens = [token.text for token in nlp(sentence)]
          tokenized_sentences.append(tokens)
      return tokenized_sentences

  #Extract position of entitie(POE) from .pointer file
def extract_POE(file_path):
      POE_Entities=[]
      with open(file_path, 'r') as file:
          for each_line in file:
            POE_list=each_line.split('|')
            POE_set=set()
            for each_POE in POE_list:
              each_POE=each_POE.strip()

              content=each_POE.split(' ')
              POE_set.add((int(content[0]),int(content[1])))
              POE_set.add((int(content[2]),int(content[3])))
            POE_Entities.append(POE_set)

      return POE_Entities

def mark_positions(sequences,POE_list):
  relative_positions=[[],[],[]]
  if len(sequences)!=len(POE_list):
    print("Length of input do not match!")
    return
  for k in range(0,3):
    for i in range(0,len(sequences)):
      each_sequence=sequences[i]
      each_POE_List=POE_list[i]
      positions=each_sequence
      adding_position=True
      temp_list4set=list(each_POE_List)
      if k>=len(temp_list4set):
        relative_positions[k].append(np.zeros(len(positions)).astype(int).tolist())
      else:
        for j in range(0,len(each_sequence)):
          each_num=each_sequence[j]

          entity1_start=temp_list4set[k][0]
          entity1_end=temp_list4set[k][1]
          if j<entity1_start:
            positions[j]=-entity1_start+j
          elif entity1_start<j<entity1_end:
            positions[j]=0
          elif j>entity1_end and each_num!=0 :
            positions[j]=j-entity1_end
          else:
            positions[j]=0

        relative_positions[k].append(positions)


  return relative_positions[0],relative_positions[1],relative_positions[2]

def count_tokens(sentences, min_count=None):
      token_counts = {}  # Create an empty dictionary to store tokens and their counts

      # Iterate over each sentence
      for sentence in sentences:
          # Iterate over each token in the sentence
          for token in nlp(sentence):
            if str(token) not in ['.',',','(', ')', '-',"%", "'", ':']:
              # Add the token to the dictionary and update the count
              token_counts[str(token)] = token_counts.get(str(token), 0) + 1

      # Filter tokens by minimum count if specified
      if min_count is not None:
          token_counts = {token: count for token, count in token_counts.items() if count > min_count}

      # Sort the dictionary by value (token counts) in descending order
      token_counts = dict(sorted(token_counts.items(), key=lambda x: x[1], reverse=True))

      return token_counts


def convert_to_sequences(sentences, annotations, word_to_index, annotation_to_index):
      X = []
      y = []
      for i in range(len(sentences)):
          sentence = sentences[i]
          annotation = annotations[i]
          sequence = []  # Convert sentence to sequence of word indices
          for word in sentence.split():
            if word not in ['.',',','(', ')', '-',"%", "'", ':']:
              if word in word_to_index:
                  sequence.append(word_to_index[word])
              else:
                  sequence.append(0)  # Use 0 for out-of-vocabulary words
          X.append(sequence)
          # Convert annotation to multi-class label
          label = np.zeros(len(annotation_to_index)).tolist()  # Initialize with zeros for all classes
          for each_tup in annotation:
            label[annotation_to_index[each_tup[2]]] = 1.0  # Set the corresponding index to 1

          y.append(label)
      return X, np.array(y)


# Data Preprocessing

##Extracts Position of entity from *.pointer file, used for position-embedding models

In [5]:
train_POE_list=extract_POE('train.pointer')
test_POE_list=extract_POE('test.pointer')

In [6]:
# count=0
# for each_set in test:
#   if len(each_set)>3:
#     count+=1
# print(count)

## Extracts sentences and annotation from file, and set up relation to index mapping

In [7]:

  # Extracting sentences and Triplets
train_annotations, test_annotations = extract_triplets('train.tup'), extract_triplets('test.tup')
train_sentences, test_sentences = read_document('train.sent'), read_document('test.sent')

  # Read unique relations from relations.txt
relations = extract_relations('relations.txt')
unique_relations={}
for relation in relations:
          doc = nlp(relation)
          tokens = [token.text for token in doc]
          unique_relations[relation] = tokens

num_unique_relations = len(unique_relations)

  #Create a mapping between each relation and its index
relation_to_index = {relation.replace(" ", "_").replace('/', '_or_').replace('\n', ''): index for index, relation in enumerate(unique_relations)}
print(relation_to_index)

{'product_or_material_produced': 0, 'manufacturer': 1, 'distributed_by': 2, 'industry': 3, 'position_held': 4, 'original_broadcaster': 5, 'owned_by': 6, 'founded_by': 7, 'distribution_format': 8, 'headquarters_location': 9, 'stock_exchange': 10, 'currency': 11, 'parent_organization': 12, 'chief_executive_officer': 13, 'director_or_manager': 14, 'owner_of': 15, 'operator': 16, 'member_of': 17, 'employer': 18, 'chairperson': 19, 'platform': 20, 'subsidiary': 21, 'legal_form': 22, 'publisher': 23, 'developer': 24, 'brand': 25, 'business_division': 26, 'location_of_formation': 27, 'creator': 28}


## number of samples with increasing number of relations

In [8]:
# Analysis on training set
for i in range(1,10):
  count=0
  for each_label in train_annotations:
    if len(each_label)>i:
      count+=1
  print("There are {} samples with {} relation(s)".format(count,i))

There are 1355 samples with 1 relation(s)
There are 474 samples with 2 relation(s)
There are 236 samples with 3 relation(s)
There are 107 samples with 4 relation(s)
There are 52 samples with 5 relation(s)
There are 35 samples with 6 relation(s)
There are 25 samples with 7 relation(s)
There are 16 samples with 8 relation(s)
There are 10 samples with 9 relation(s)


In [9]:
# Analysis on testing set
for i in range(0,10):
  count=0
  for each_label in test_annotations:
    if len(each_label)>i:
      count+=1
  print("There are {} samples with {} relation(s)".format(count,i))

There are 1068 samples with 0 relation(s)
There are 293 samples with 1 relation(s)
There are 97 samples with 2 relation(s)
There are 30 samples with 3 relation(s)
There are 0 samples with 4 relation(s)
There are 0 samples with 5 relation(s)
There are 0 samples with 6 relation(s)
There are 0 samples with 7 relation(s)
There are 0 samples with 8 relation(s)
There are 0 samples with 9 relation(s)


## Data filtering

Removing samples from training set that comes with too many relations within a single sample

In [10]:
indices = [i for i, each_label in enumerate(train_annotations) if  len(each_label)>3]

print(indices)

train_annotations = [num for i, num in enumerate(train_annotations) if i not in indices]
train_sentences = [num for i, num in enumerate(train_sentences) if i not in indices]
# y_train_reduced = np.array([num for i, num in enumerate(y_train.tolist()) if i not in indices])

# print(len(X_train_reduced)==len(y_train_reduced))
# print([i for i, num in enumerate(X_train_reduced) if len(num)!=907])
# print([i for i, num in enumerate(y_train_reduced) if len(num)!=29])

[8, 12, 34, 35, 36, 129, 168, 269, 288, 294, 340, 380, 402, 404, 515, 538, 541, 546, 707, 708, 710, 765, 776, 777, 780, 812, 875, 968, 986, 1012, 1040, 1081, 1090, 1091, 1114, 1166, 1210, 1214, 1226, 1283, 1289, 1290, 1297, 1306, 1319, 1395, 1409, 1418, 1468, 1498, 1582, 1583, 1627, 1685, 1688, 1728, 1769, 1772, 1810, 1820, 1838, 1871, 1891, 1931, 1933, 1939, 1958, 1982, 1997, 2022, 2079, 2088, 2102, 2112, 2116, 2140, 2153, 2162, 2180, 2183, 2217, 2240, 2243, 2248, 2269, 2393, 2397, 2413, 2447, 2566, 2579, 2592, 2609, 2640, 2680, 2717, 2734, 2758, 2778, 2779, 2782, 2810, 2823, 2830, 2833, 2845, 2860, 2875, 2877, 2878, 2889, 2908, 2910, 2911, 2916, 2954, 2957, 2958, 2968, 2988, 3051, 3080, 3081, 3123, 3141, 3144, 3156, 3180, 3191, 3197, 3198, 3199, 3231, 3247, 3284, 3343, 3359, 3381, 3523, 3528, 3560, 3576, 3580, 3609, 3617, 3630, 3660, 3673, 3702, 3731, 3742, 3754, 3765, 3801, 3876, 3877, 3907, 3910, 3926, 3937, 3951, 4014, 4077, 4121, 4140, 4156, 4160, 4207, 4237, 4276, 4281, 4305, 43

## Finalize and generate word to index mapping

In [ ]:
#GPT



FILTER_MIN=1

  # Build vocabulary
vocabulary = count_tokens(train_sentences)
print(vocabulary)
filtered_vocabulary = {key: value for key, value in vocabulary.items() if value > FILTER_MIN}

  # Print filtered vocabulary
print(filtered_vocabulary)
  # Create word_to_index mapping
word_to_index = {word: index for index, (word, count) in enumerate(filtered_vocabulary.items(), start=1)}
print(word_to_index)

In [ ]:



#with tf.device('/GPU:0'):
  # Convert training data to sequences
X_train, y_train = convert_to_sequences(train_sentences, train_annotations, word_to_index, relation_to_index)

  # Convert testing data to sequences
X_test, y_test = convert_to_sequences(test_sentences, test_annotations, word_to_index, relation_to_index)


  # Hyperparameters
EMBEDDING_DIM = 300  # Dimension of word embeddings
NUM_CLASSES = num_unique_relations  # Number of classes for relation extraction
BATCH_SIZE = 128
EPOCHS = 14
MAX_SEQ_LENGTH = max(len(sentence) for sentence in X_train)
VOCABULARY_SIZE=len(word_to_index) + 1






In [ ]:

for i in range(1,10):
  count=0
  for each_label in y_train:
    if sum(each_label)>i:
      count+=1
  print(count)


In [ ]:
  # Padding sequences to ensure uniform length
X_train = pad_sequences(X_train, maxlen=MAX_SEQ_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_SEQ_LENGTH)


In [ ]:
# indices = [i for i, each_label in enumerate(y_train) if  sum(each_label)>3]

# print(indices)

# X_train_reduced = np.array([num for i, num in enumerate(X_train.tolist()) if i not in indices])

# y_train_reduced = np.array([num for i, num in enumerate(y_train.tolist()) if i not in indices])

# print(len(X_train_reduced)==len(y_train_reduced))
# print([i for i, num in enumerate(X_train_reduced) if len(num)!=907])
# print([i for i, num in enumerate(y_train_reduced) if len(num)!=29])

# Trainer Definition

In [ ]:
def train_model(X_train, y_train,batch_size=8,epoch=6,num_NN=128,dropout=0.6,is_saving=False,saving_acc=0.55,num_Dense=64):
  history_callback = tf.keras.callbacks.History()
  optimizer = tf.keras.optimizers.Adam()
  model = Sequential()
  model.add(Embedding(VOCABULARY_SIZE, output_dim=EMBEDDING_DIM, input_shape=(MAX_SEQ_LENGTH,)))
  model.add(Bidirectional(LSTM(num_NN,dropout=dropout)))
  model.add(Dense(num_Dense, activation='relu'))

  model.add(Dense(NUM_CLASSES, activation='softmax'))

        # Compile model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

      # Train model

  model.fit(X_train, y_train, batch_size=batch_size, epochs=epoch, validation_data=(X_test, y_test),callbacks=[history_callback])

  history = history_callback.history

  best_accuracy = np.max(history['val_accuracy'])

  if best_accuracy>saving_acc and is_saving:
    tf.saved_model.save(model, './batch_size={};epoch={};num_NN={};num_Dense={}'.format(batch_size,epoch,num_NN,num_Dense))





## Trainer Usage

In [ ]:
train_model(X_train, y_train,batch_size=8,epoch=14,num_NN=128,dropout=0.4,is_saving=True,saving_acc=0.50)

Epoch 1/14
563/563 [==============================] - 37s 59ms/step - loss: 0.1617 - accuracy: 0.1987 - val_loss: 0.1713 - val_accuracy: 0.1695
Epoch 2/14
563/563 [==============================] - 24s 43ms/step - loss: 0.1490 - accuracy: 0.2173 - val_loss: 0.1646 - val_accuracy: 0.1657
Epoch 3/14
563/563 [==============================] - 23s 41ms/step - loss: 0.1311 - accuracy: 0.3387 - val_loss: 0.1445 - val_accuracy: 0.3137
Epoch 4/14
563/563 [==============================] - 23s 41ms/step - loss: 0.0967 - accuracy: 0.5433 - val_loss: 0.1330 - val_accuracy: 0.4466
Epoch 5/14
563/563 [==============================] - 22s 40ms/step - loss: 0.0646 - accuracy: 0.6884 - val_loss: 0.1288 - val_accuracy: 0.4813
Epoch 6/14
563/563 [==============================] - 23s 40ms/step - loss: 0.0438 - accuracy: 0.7882 - val_loss: 0.1357 - val_accuracy: 0.4841
Epoch 7/14
563/563 [==============================] - 22s 39ms/step - loss: 0.0287 - accuracy: 0.8602 - val_loss: 0.1481 - val_accuracy:

# Predicter definition

**Model Predicter**

---


Given a sequence of sentence(s), word and relation to index mapping and maximum sequence length if not uning FINRed, returns a sequence of prediction. Paremeters with defaults can be safely left as blank when the model used the training set from FINRed, but otherwise must be configured as identical to those are used during training.


---
**model:** a model extracted by **tf.saved_model.load**, note that freshly trained model can't be fed into the function in this way, as the model recreated from save is different than fresh ones and therefore having different usage


---
**X:** a 1-dimension sequence of sentences to predict. No need for any vector conversion in advance, these are done within the function


---

**word_to_index,relation_to_index:** key-value pairs that maps word/relation to integers. These must be identical to those are used during the training. The relation to index sequence is flipped in the function for its reversed use to convert integer back to label.


---
**MAX_SEQ_LENGTH:** the maxima sequence length encountered during training, must be identical to the one used when training.




In [ ]:
def predict_with_model(model,X,word_to_index='default',relation_to_index='default',MAX_SEQ_LENGTH=589):#uning FINRed by default, the MAX_SEQ_LENGTH is 589

  if word_to_index=='default':
    word_to_index={'the': 1, 'and': 2, 'of': 3, 'to': 4, 'in': 5, 'a': 6, 'on': 7, 'for': 8, 'that': 9, 'with': 10, 'is': 11, 'as': 12, "'s": 13, 'its': 14, 'at': 15, 'by': 16, 'news': 17, 'said': 18, 'from': 19, '"': 20, 'has': 21, 'it': 22, 'The': 23, 'will': 24, '$': 25, 'we': 26, 'an': 27, 'company': 28, 'our': 29, 'was': 30, 'which': 31, 'Group': 32, 'are': 33, 'have': 34, 'year': 35, 'be': 36, '’s': 37, 'more': 38, 'new': 39, 'this': 40, 'after': 41, 'LSE': 42, 'billion': 43, 'Bank': 44, 'their': 45, 'market': 46, 'million': 47, 'up': 48, 'or': 49, 'than': 50, '2015': 51, 'percent': 52, 'business': 53, ';': 54, 'would': 55, 'also': 56, 'not': 57, 'been': 58, 'you': 59, 'I': 60, 'over': 61, 'retail': 62, 'his': 63, 'had': 64, 'all': 65, 'Reuters': 66, 'but': 67, 'first': 68, 'India': 69, 'other': 70, 'including': 71, 'last': 72, 'CEO': 73, 'July': 74, 'about': 75, 'sales': 76, 'quarter': 77, '“': 78, '”': 79, 'Inc.': 80, 'services': 81, 'one': 82, 'Other': 83, 'into': 84, 'Google': 85, 'were': 86, 'New': 87, 'In': 88, 'out': 89, 'U.S.': 90, 'per': 91, 'announced': 92, 'some': 93, 'chief': 94, 'they': 95, '--': 96, 'companies': 97, 'years': 98, 'he': 99, 'financial': 100, 'Microsoft': 101, 'like': 102, 'industry': 103, 'deal': 104, 'We': 105, 'China': 106, 'Inc': 107, 'when': 108, 'growth': 109, '–': 110, 'shares': 111, 'well': 112, 'two': 113, 'through': 114, 'while': 115, 'global': 116, 'customers': 117, 'group': 118, 'software': 119, 'executive': 120, 'world': 121, 'And': 122, 'OTC': 123, 'such': 124, 'can': 125, 'General': 126, 'prices': 127, 'now': 128, 'second': 129, '10': 130, 'cent': 131, 'Chief': 132, 'week': 133, 'bank': 134, '&': 135, 'based': 136, 'time': 137, '/': 138, 'could': 139, 'European': 140, 'largest': 141, 'most': 142, 'Corp': 143, 'Australia': 144, 'where': 145, 'investment': 146, 'products': 147, 'Airbus': 148, 'down': 149, 'today': 150, 'Prime': 151, 'Minister': 152, 'share': 153, 'A': 154, 'Samsung': 155, 'London': 156, 'who': 157, 'aircraft': 158, 'News': 159, 'Greece': 160, 'data': 161, 'three': 162, 'just': 163, 'American': 164, 'UK': 165, 'only': 166, 'part': 167, 'Executive': 168, 'there': 169, 'both': 170, 'price': 171, 'production': 172, 'profit': 173, 'month': 174, 'Glencore': 175, 'Wednesday': 176, 'International': 177, 'demand': 178, 'Monday': 179, '1': 180, 'maker': 181, 'US': 182, 'made': 183, 'expected': 184, 'online': 185, 'Holdings': 186, 'Watch': 187, 'reported': 188, 'Friday': 189, 'Tuesday': 190, 'stock': 191, 'So': 192, 'since': 193, 'Corporation': 194, 'June': 195, 'ago': 196, 'around': 197, 'Broker': 198, 'Aug': 199, 'agreement': 200, 'Thursday': 201, 'if': 202, 'Amazon': 203, 'back': 204, 'party': 205, 'do': 206, 'between': 207, 'plans': 208, 'leading': 209, 'mining': 210, 'Co': 211, 'firm': 212, 'long': 213, 'any': 214, '|': 215, '£': 216, 'Syriza': 217, 'buy': 218, 'technology': 219, 'solutions': 220, 'biggest': 221, 'government': 222, 'giant': 223, 'people': 224, 'markets': 225, 'across': 226, 'so': 227, 'very': 228, '2': 229, "n't": 230, 'top': 231, 'mobile': 232, 'businesses': 233, 'next': 234, 'off': 235, 'Corp.': 236, 'Financial': 237, 'strong': 238, 'Tsipras': 239, 'Reserve': 240, 'service': 241, 'revenue': 242, 'high': 243, 'oil': 244, 'major': 245, 'euro': 246, 'what': 247, 'support': 248, 'platform': 249, 'NYSE': 250, 'due': 251, 'end': 252, 'Capital': 253, 'Energy': 254, 'York': 255, 'Ltd': 256, 'Motors': 257, 'going': 258, 'how': 259, 'gas': 260, 'says': 261, 'bailout': 262, 'Greek': 263, 'chairman': 264, 'told': 265, 'before': 266, 'take': 267, '—': 268, 'make': 269, 'United': 270, 'include': 271, 'stake': 272, 'during': 273, 'Business': 274, 'Europe': 275, 'state': 276, 'Company': 277, 'country': 278, 'may': 279, 'Apple': 280, 'according': 281, 'Electronics': 282, 'British': 283, 'Alexis': 284, 'sector': 285, 'operating': 286, 'Finance': 287, 'results': 288, 'continue': 289, 'third': 290, 'As': 291, 'statement': 292, 'cost': 293, 'Insurance': 294, 'because': 295, 'banks': 296, 'your': 297, 'value': 298, 'lower': 299, 'under': 300, 'National': 301, 'finance': 302, '20': 303, 'e': 304, 'higher': 305, '?': 306, 'This': 307, 'see': 308, 'help': 309, 'rate': 310, 'It': 311, 'unit': 312, 'stores': 313, 'recently': 314, 'sold': 315, 'Mr': 316, 'trading': 317, '3': 318, 'Co.': 319, 'former': 320, 'digital': 321, 'set': 322, 'half': 323, 'them': 324, 'offer': 325, 'further': 326, 'close': 327, 'call': 328, 'insurance': 329, 'Plc': 330, 'cut': 331, 'months': 332, 'increase': 333, 'think': 334, 'Boeing': 335, '30': 336, 'against': 337, 'rose': 338, 'even': 339, 'number': 340, '5': 341, 'On': 342, 'brands': 343, '7': 344, 'operations': 345, 'owned': 346, '2014': 347, 'then': 348, 'investors': 349, 'us': 350, 'costs': 351, 'assets': 352, 'brand': 353, 'energy': 354, 'capital': 355, 'Windows': 356, 'Xetra': 357, 'Ltd.': 358, 'Articles': 359, 'debt': 360, 'among': 361, 'still': 362, 'much': 363, 'Stock': 364, '*': 365, 'earnings': 366, 'Services': 367, 'leader': 368, 'management': 369, 'potential': 370, 'commerce': 371, 'provider': 372, 'University': 373, 'September': 374, 'full': 375, 'EU': 376, 'being': 377, 'America': 378, 'plan': 379, 'acquisition': 380, 'Exchange': 381, 'Global': 382, 'John': 383, 'rival': 384, 'talks': 385, 'FTSE': 386, 'sell': 387, 'private': 388, 'term': 389, 'product': 390, 'way': 391, 'use': 392, '12': 393, 'August': 394, 'President': 395, 'eurozone': 396, 'pay': 397, 'working': 398, 'system': 399, 'many': 400, 'shareholders': 401, '100': 402, 'those': 403, 'get': 404, 'By': 405, 'water': 406, 'Director': 407, 'same': 408, 'sale': 409, 'report': 410, 'Markets': 411, 'merger': 412, 'Officer': 413, 'credit': 414, 'open': 415, 'already': 416, 'large': 417, "'re": 418, 'marketing': 419, 'net': 420, 'economy': 421, 'work': 422, 'day': 423, 'coffee': 424, 'early': 425, 'these': 426, 'key': 427, 'cash': 428, 'growing': 429, 'launch': 430, 'big': 431, 'better': 432, 'no': 433, 'home': 434, 'fell': 435, 'current': 436, 'money': 437, 'Starbucks': 438, 'move': 439, '4': 440, 'Times': 441, 'own': 442, 'founder': 443, 'run': 444, 'store': 445, 'With': 446, 'within': 447, 'terms': 448, 'experience': 449, 'development': 450, 'Vodafone': 451, 'Eurogroup': 452, 'come': 453, 'rates': 454, '6': 455, 'Oil': 456, 'public': 457, 'mine': 458, 'South': 459, 'plant': 460, 'really': 461, 'network': 462, 'good': 463, 'interest': 464, 'ability': 465, 'agreed': 466, 'Facebook': 467, 'Jul': 468, 'Rs': 469, 'Share': 470, '15': 471, 'should': 472, 'video': 473, 'look': 474, 'partnership': 475, 'each': 476, 'become': 477, 'venture': 478, 'five': 479, 'worth': 480, 'users': 481, 'countries': 482, 'launched': 483, 'consumer': 484, 'seen': 485, 'start': 486, 'low': 487, 'North': 488, '21': 489, 'international': 490, '50': 491, 'Athens': 492, '•': 493, 'meeting': 494, 'Retail': 495, 'Electric': 496, 'strategy': 497, 'Internet': 498, 'Canada': 499, 'French': 500, 'm': 501, 'card': 502, 'food': 503, 'earlier': 504, 'music': 505, 'commercial': 506, 'minutes': 507, 'selling': 508, 'joint': 509, 'Union': 510, 'following': 511, 'four': 512, 'future': 513, 'fund': 514, 'banking': 515, 'need': 516, 'team': 517, 'impact': 518, 'does': 519, 'equity': 520, 'building': 521, 'City': 522, 'gold': 523, 'related': 524, 'called': 525, 'Dutch': 526, 'owner': 527, 'director': 528, 'stocks': 529, 'LONDON': 530, 'deliver': 531, 'fresh': 532, 'project': 533, 'closed': 534, 'small': 535, 'projects': 536, 'PayPal': 537, 'For': 538, 'recent': 539, '’': 540, 'head': 541, 'Entertainment': 542, 'annual': 543, 'Red': 544, 'left': 545, '3D': 546, 'dollar': 547, 'used': 548, 'go': 549, 'content': 550, 'channel': 551, 'partners': 552, 'Management': 553, 'acquire': 554, '2020': 555, 'provide': 556, 'period': 557, 'offering': 558, 'retailer': 559, 'decline': 560, 'position': 561, 'media': 562, 'making': 563, 'latest': 564, 'policy': 565, 'payments': 566, 'IST': 567, 'Rentrak': 568, 'Jeroen': 569, 'Dijsselbloem': 570, 'several': 571, 'free': 572, 'program': 573, 'customer': 574, 'release': 575, 'Chinese': 576, 'power': 577, 'took': 578, 'Australian': 579, 'Gold': 580, 'her': 581, 'Mining': 582, 'Thomson': 583, 'saying': 584, 'loss': 585, 'Fiat': 586, 'manufacturing': 587, 'did': 588, 'car': 589, 'keep': 590, 'But': 591, 'six': 592, 'exchange': 593, 'local': 594, 'know': 595, "'ve": 596, 'real': 597, 'currently': 598, 'chain': 599, 'led': 600, 'lead': 601, '8': 602, 'AG': 603, 'natural': 604, 'Resources': 605, 'Twitter': 606, 'crore': 607, 'available': 608, 'conglomerate': 609, 'includes': 610, 'Our': 611, 'together': 612, 'access': 613, 'best': 614, 'say': 615, 'center': 616, 'increased': 617, 'whether': 618, 'control': 619, 'subsidiary': 620, 'Royal': 621, 'BST': 622, '...': 623, 'jobs': 624, 'president': 625, 'result': 626, 'added': 627, 'clients': 628, '9': 629, 'here': 630, 'eBay': 631, 'Limited': 632, 'put': 633, 'firms': 634, 'GM': 635, '2013': 636, 'M': 637, 'Street': 638, 'portfolio': 639, 'held': 640, 'Chrysler': 641, 'TV': 642, 'website': 643, '500': 644, 'different': 645, 'performance': 646, 'taking': 647, 'Los': 648, '28': 649, 'far': 650, 'forward': 651, 'name': 652, 'Market': 653, 'prime': 654, 'State': 655, 'copper': 656, 'Sydney': 657, 'right': 658, 'Asia': 659, 'likely': 660, 'nearly': 661, 'conference': 662, 'delivery': 663, 'n’t': 664, 'level': 665, 'few': 666, 'cloud': 667, 'supply': 668, 'ahead': 669, 'Angeles': 670, 'using': 671, 'Hong': 672, 'looking': 673, 'another': 674, 'decision': 675, 'Hat': 676, 'bid': 677, 'Delhi': 678, 'ministers': 679, 'comScore': 680, 'Sunday': 681, 'Technology': 682, 'governor': 683, 'bought': 684, 'Germany': 685, 'operator': 686, 'States': 687, 'vote': 688, '11': 689, 'SA': 690, 'example': 691, 'May': 692, 'significant': 693, 'given': 694, 'Japan': 695, 'At': 696, 'co': 697, 'change': 698, 'Kong': 699, 'Spotify': 700, 'Tata': 701, 'World': 702, 'Nasdaq': 703, 'days': 704, 'deals': 705, 'offers': 706, 'construction': 707, 'listed': 708, 'acquired': 709, 'quality': 710, 'got': 711, 'past': 712, '2016': 713, 'Network': 714, 'California': 715, 'signed': 716, 'Walmart': 717, 'expand': 718, 'office': 719, 'producer': 720, 'Equinix': 721, '29': 722, 'employees': 723, 'trade': 724, 'Swiss': 725, 'Stratasys': 726, 'Goldman': 727, 'Sachs': 728, 'Indian': 729, 'Foxtel': 730, 'give': 731, 'little': 732, 'important': 733, 'driven': 734, 'Investment': 735, 'seeing': 736, 'want': 737, 'weeks': 738, 'taken': 739, 'sports': 740, 'partner': 741, 'revenues': 742, 'saw': 743, 'Media': 744, 'information': 745, 'tax': 746, '17': 747, 'opportunities': 748, 'Mark': 749, 'Securities': 750, 'Airlines': 751, 'Shell': 752, '®': 753, 'Telstra': 754, 'Industries': 755, 'Berkshire': 756, 'continued': 757, 'analysts': 758, 'least': 759, 'national': 760, 'streaming': 761, 'social': 762, 'built': 763, 'line': 764, 'order': 765, 'strategic': 766, 'model': 767, 'place': 768, 'focus': 769, 'leaders': 770, 'matter': 771, 'again': 772, 'Direct': 773, '16': 774, 'Fox': 775, 'board': 776, 'Governor': 777, 'PLC': 778, 'Premier': 779, 'ShareCast': 780, '250': 781, 'Glenn': 782, 'Stevens': 783, 'RBI': 784, 'NEW': 785, 'concerns': 786, 'senior': 787, 'March': 788, 'officer': 789, 'While': 790, 'grocery': 791, 'providing': 792, 'search': 793, 'Health': 794, 'pressure': 795, 'NASDAQ': 796, 'systems': 797, 'class': 798, 'won': 799, 'steel': 800, '18': 801, 'possible': 802, 'Brian': 803, 'purchase': 804, '25': 805, 'forecast': 806, 'AP': 807, 'Petroleum': 808, 'David': 809, 'Saturday': 810, 'Ghana': 811, 'Value': 812, 'House': 813, 'One': 814, 'fiscal': 815, 'He': 816, 'base': 817, 'segment': 818, 'build': 819, 'process': 820, 'behind': 821, 'having': 822, 'review': 823, 'basis': 824, 'security': 825, 'play': 826, 'To': 827, 'Commission': 828, 'income': 829, 'April': 830, '19': 831, 'France': 832, 'Steel': 833, 'BG': 834, '…': 835, 'Hathaway': 836, 'health': 837, '13': 838, 'staff': 839, 'arm': 840, 'released': 841, 'addition': 842, 'believe': 843, 'orders': 844, 'Chevrolet': 845, 'innovative': 846, 'develop': 847, 'overall': 848, 'Intel': 849, 'applications': 850, 'units': 851, 'coming': 852, 'why': 853, 'television': 854, 'less': 855, 'research': 856, 'crisis': 857, 'reports': 858, 'total': 859, 'Sony': 860, 'issues': 861, 'family': 862, 'direct': 863, 'site': 864, 'changes': 865, 'hold': 866, 'package': 867, 'Hindustan': 868, 'Updated': 869, 'Britain': 870, 'rights': 871, 'Pacific': 872, 'BHP': 873, 'Anglo': 874, 'known': 875, 'range': 876, 'Fund': 877, 'life': 878, 'side': 879, 'players': 880, 'able': 881, 'advertising': 882, 'initiative': 883, 'member': 884, 'provided': 885, 'drive': 886, 'things': 887, 'space': 888, 'Development': 889, 'electronics': 890, 'remain': 891, 'approval': 892, 'infrastructure': 893, '14': 894, 'economic': 895, 'independent': 896, 'risk': 897, 'event': 898, 'seven': 899, 'index': 900, 'LLC': 901, 'drop': 902, 'amid': 903, 'Airways': 904, 'printing': 905, 'boss': 906, 'Fallers': 907, 'rise': 908, 'GE': 909, 'Press': 910, 'Rio': 911, 'YORK': 912, 'investor': 913, 'Chairman': 914, 'Sergio': 915, 'comes': 916, 'Express': 917, 'previously': 918, 'game': 919, 'Office': 920, 'central': 921, 'domestic': 922, 'form': 923, 'region': 924, 'expansion': 925, 'account': 926, 'Software': 927, 'corporate': 928, 'area': 929, 'Boston': 930, 'workers': 931, 'approximately': 932, ']': 933, 'Music': 934, '24': 935, 'opportunity': 936, 'app': 937, 'Bharti': 938, 'main': 939, 'every': 940, 'came': 941, 'general': 942, 'loan': 943, 'comment': 944, 'fall': 945, 'meet': 946, 'Air': 947, 'once': 948, 'reached': 949, 'must': 950, 'FedEx': 951, 'helped': 952, 'mins': 953, 'Photo': 954, 'Chancellor': 955, 'via': 956, 'Rajan': 957, 'SABMiller': 958, 'Marchionne': 959, 'record': 960, 'list': 961, 'late': 962, '31': 963, 'kind': 964, 'transaction': 965, 'Sky': 966, 'design': 967, 'show': 968, '[': 969, 'After': 970, 'Sales': 971, 'create': 972, 'E': 973, 'leadership': 974, 'members': 975, 'Technologies': 976, 'update': 977, 'Federal': 978, 'provides': 979, 'estate': 980, 'Korea': 981, 'case': 982, 'January': 983, 'Brazil': 984, 'point': 985, 'George': 986, 'short': 987, 'cuts': 988, 'Post': 989, 'nation': 990, 'Mattel': 991, 'Central': 992, 'Africa': 993, 'Amount': 994, '@': 995, 'Lonmin': 996, 'Systems': 997, 'Airtel': 998, 'Aug.': 999, 'Arun': 1000, 'Jaitley': 1001, 'Last': 1002, '2008': 1003, 'Morgan': 1004, 'loans': 1005, 'target': 1006, 'allow': 1007, '’re': 1008, 'relationship': 1009, 'Cloud': 1010, 'Baidu': 1011, 'enterprise': 1012, 'Center': 1013, 'my': 1014, 'various': 1015, 'environment': 1016, 'doing': 1017, 'Singapore': 1018, 'Life': 1019, 'needed': 1020, 'San': 1021, 'hit': 1022, 'despite': 1023, 'Trust': 1024, 'reduce': 1025, 'benefits': 1026, 'multi': 1027, 'PM': 1028, 'without': 1029, 'competition': 1030, 'others': 1031, 'near': 1032, 'conditions': 1033, 'popular': 1034, 'Barclays': 1035, 'reforms': 1036, 'milk': 1037, 'Shares': 1038, 'sources': 1039, 'raise': 1040, 'NasdaqGS': 1041, 'beer': 1042, 'agency': 1043, 'Buick': 1044, 'Visa': 1045, 'German': 1046, 'takeover': 1047, 'Credit': 1048, 'moving': 1049, 'engineering': 1050, 'role': 1051, 'obviously': 1052, 'adding': 1053, 'bring': 1054, 'continues': 1055, 'additional': 1056, 'situation': 1057, 'particularly': 1058, '365': 1059, 'success': 1060, 'positive': 1061, 'COVID-19': 1062, 'fourth': 1063, 'Michael': 1064, 'devices': 1065, 'win': 1066, 'Digital': 1067, 'posted': 1068, 'operation': 1069, 'regulatory': 1070, 'Century': 1071, 'Communications': 1072, 'vehicle': 1073, 'she': 1074, 'division': 1075, 'Foods': 1076, 'step': 1077, 'issue': 1078, 'contract': 1079, 'Melbourne': 1080, 'Real': 1081, 'return': 1082, 'Snapdeal': 1083, 'GMT': 1084, 'Raghuram': 1085, 'found': 1086, 'Qantas': 1087, 'AM': 1088, 'Line': 1089, 'Dubai': 1090, 'USD': 1091, 'Reliance': 1092, 'Billiton': 1093, 'SAA': 1094, 'Citigroup': 1095, 'closing': 1096, 'MasterCard': 1097, 'euros': 1098, 'parent': 1099, 'bit': 1100, 'deadline': 1101, '60': 1102, 'truck': 1103, 'owns': 1104, 'became': 1105, 'fuel': 1106, 'sustainable': 1107, 'Q2': 1108, 'prior': 1109, 'focused': 1110, 'delivered': 1111, 'started': 1112, 'benefit': 1113, 'Chicago': 1114, '70': 1115, 'buying': 1116, 'Disney': 1117, 'great': 1118, 'volume': 1119, '€': 1120, 'treatment': 1121, "'ll": 1122, 'amount': 1123, 'series': 1124, 'history': 1125, 'reach': 1126, 'core': 1127, 'Canadian': 1128, 'quarterly': 1129, 'offset': 1130, 'began': 1131, 'premium': 1132, 'First': 1133, 'received': 1134, 'October': 1135, 'launches': 1136, 'Paris': 1137, 'tough': 1138, 'filed': 1139, 'Council': 1140, 'raised': 1141, 'coal': 1142, 'Page': 1143, 'BP': 1144, 'According': 1145, '»': 1146, 'Risers': 1147, 'ATHENS': 1148, 'Arundhati': 1149, 'Bhattacharya': 1150, 'Martin': 1151, 'familiar': 1152, 'Jeff': 1153, 'Buffett': 1154, 'Italy': 1155, 'track': 1156, 'generation': 1157, 'air': 1158, 'soon': 1159, 'enable': 1160, 'might': 1161, 'needs': 1162, 'paper': 1163, 'pandemic': 1164, 'similar': 1165, 'spend': 1166, 'If': 1167, 'ever': 1168, '2019': 1169, 'phone': 1170, 'getting': 1171, 'Gartner': 1172, 'course': 1173, 'non': 1174, 'Public': 1175, 'majority': 1176, 'offered': 1177, 'towards': 1178, 'guidance': 1179, 'went': 1180, 'restaurant': 1181, 'currency': 1182, 'James': 1183, 'Plus': 1184, 'boost': 1185, 'Gas': 1186, 'dairy': 1187, 'Earnings': 1188, 'consumers': 1189, 'payment': 1190, 'itself': 1191, 'hours': 1192, 'commodity': 1193, '22': 1194, 'hrs': 1195, 'Banking': 1196, 'Smith': 1197, 'ICICI': 1198, 'Former': 1199, 'Related': 1200, 'estimates': 1201, 'Tinto': 1202, 'Warren': 1203, 'below': 1204, 'engine': 1205, 'him': 1206, 'Fed': 1207, 'Wall': 1208, 'outlook': 1209, 'increasing': 1210, 'teams': 1211, 'version': 1212, 'regional': 1213, 'hard': 1214, 'live': 1215, 'Enterprise': 1216, 'highly': 1217, 'founded': 1218, '40': 1219, 'reporting': 1220, 'clear': 1221, 'though': 1222, 'Auto': 1223, 'flagship': 1224, 'couple': 1225, 'fact': 1226, 'advantage': 1227, 'points': 1228, 'combined': 1229, 'fully': 1230, 'initial': 1231, 'producers': 1232, 'Deloitte': 1233, 'Power': 1234, 'single': 1235, 'issued': 1236, 'scale': 1237, 'push': 1238, 'weak': 1239, 'announcement': 1240, 'compared': 1241, 'average': 1242, 'That': 1243, 'About': 1244, 'produced': 1245, 'Day': 1246, 'invest': 1247, 'knowledge': 1248, '2011': 1249, 'airline': 1250, 'almost': 1251, 'Alan': 1252, 'industrial': 1253, 'Oracle': 1254, 'sign': 1255, 'minister': 1256, 'internet': 1257, 'headquarters': 1258, 'plc': 1259, 'factory': 1260, 'Bandhan': 1261, 'Sept': 1262, 'lenders': 1263, 'smartphone': 1264, 'Kolkata': 1265, 'Minerals': 1266, 'monetary': 1267, 'IT': 1268, 'notified': 1269, 'CBS': 1270, 'included': 1271, 'action': 1272, 'funds': 1273, 'newspaper': 1274, 'difficult': 1275, 'USA': 1276, 'effect': 1277, 'citing': 1278, 'retailers': 1279, 'match': 1280, 'along': 1281, 'beyond': 1282, 'platforms': 1283, 'pre': 1284, 'times': 1285, 'GMC': 1286, 'numbers': 1287, 'running': 1288, 'maybe': 1289, 'managed': 1290, 'grow': 1291, 'integration': 1292, 'integrated': 1293, 'actually': 1294, '27': 1295, 'common': 1296, 'lot': 1297, 'U.K.': 1298, 'paid': 1299, 'official': 1300, 'G': 1301, 'enough': 1302, 'advanced': 1303, 'cities': 1304, 'combination': 1305, 'named': 1306, 'planning': 1307, 'existing': 1308, 'options': 1309, 'starting': 1310, 'stay': 1311, 'momentum': 1312, 'away': 1313, '300': 1314, 'capacity': 1315, 'toy': 1316, 'remains': 1317, 'suppliers': 1318, 'carrier': 1319, 'budget': 1320, 'Coty': 1321, 'However': 1322, 'equipment': 1323, 'shareholder': 1324, 'Alcoa': 1325, 'ore': 1326, 'Osborne': 1327, 'Standard': 1328, 'Eurozone': 1329, 'View': 1330, 'miner': 1331, 'creditors': 1332, '  ': 1333, 'Lee': 1334, 'Zuckerberg': 1335, 'African': 1336, 'holds': 1337, 'hedge': 1338, 'Bloomberg': 1339, 'Larry': 1340, 'SBI': 1341, 'cement': 1342, 'A1JAGV': 1343, 'Barrick': 1344, 'Releases': 1345, 'Stanley': 1346, 'channels': 1347, 'manager': 1348, 'later': 1349, 'property': 1350, 'inflation': 1351, 'planned': 1352, 'spokesman': 1353, 'talk': 1354, 'user': 1355, 'margin': 1356, 'LinkedIn': 1357, 'produce': 1358, 'Uniper': 1359, 'efforts': 1360, 'grew': 1361, 'Azure': 1362, 'paying': 1363, 'larger': 1364, 'add': 1365, 'especially': 1366, 'done': 1367, 'peer': 1368, 'analysis': 1369, 'operate': 1370, 'post': 1371, 'developing': 1372, 'Quadrant': 1373, 'recovery': 1374, 'press': 1375, 'legal': 1376, 'presence': 1377, 'Private': 1378, 'Telecom': 1379, 'features': 1380, 'iPhone': 1381, 'wide': 1382, 'increases': 1383, 'expect': 1384, 'date': 1385, 'Investors': 1386, 'ensure': 1387, 'valued': 1388, 'significantly': 1389, 'source': 1390, 'losses': 1391, 'transfer': 1392, '26': 1393, 'accounts': 1394, 'exit': 1395, 'job': 1396, 'Johnson': 1397, 'cars': 1398, 'cap': 1399, 'weaker': 1400, 'met': 1401, 'house': 1402, 'required': 1403, 'iShares': 1404, 'Flipkart': 1405, 'BG.L': 1406, 'rising': 1407, 'Antony': 1408, 'Jenkins': 1409, 'expects': 1410, 'Euro': 1411, 'Report': 1412, 'warned': 1413, 'Virgin': 1414, 'Westpac': 1415, 'C&T': 1416, 'ruling': 1417, 'radical': 1418, 'parliamentary': 1419, 'twitter': 1420, 'Joyce': 1421, '2012': 1422, 'Lafarge': 1423, 'Seven': 1424, 'Estate': 1425, 'Bezos': 1426, 'cable': 1427, 'entity': 1428, 'manufacturer': 1429, 'charges': 1430, 'bankruptcy': 1431, 'HSBC': 1432, 'accounting': 1433, 'computer': 1434, 'until': 1435, 'networks': 1436, 'games': 1437, 'array': 1438, 'size': 1439, 'secure': 1440, 'capabilities': 1441, 'December': 1442, 'Fortum': 1443, 'category': 1444, 'view': 1445, 'manage': 1446, 'created': 1447, 'approach': 1448, 'mid': 1449, 'competitive': 1450, '2006': 1451, 'probably': 1452, 'publicly': 1453, 'expenses': 1454, 'negotiations': 1455, 'respectively': 1456, 'entire': 1457, 'parts': 1458, 'expanding': 1459, 'Magic': 1460, 'Tokyo': 1461, 'distribution': 1462, 'Mr.': 1463, 'safety': 1464, 'Party': 1465, 'Now': 1466, 'law': 1467, 'exports': 1468, 'tech': 1469, 'Mexico': 1470, 'All': 1471, 'powerful': 1472, 'turn': 1473, 'improve': 1474, 'ended': 1475, 'scheduled': 1476, 'Industry': 1477, 'fleet': 1478, 'developed': 1479, 'measures': 1480, 'IBM': 1481, 'sells': 1482, 'vehicles': 1483, 'managing': 1484, 'financing': 1485, 'goods': 1486, 'Russian': 1487, 'automotive': 1488, 'falling': 1489, 'operates': 1490, 'Codelco': 1491, 'Home': 1492, 'Bouygues': 1493, 'face': 1494, 'profits': 1495, 'Experience': 1496, 'Be': 1497, 'Barbie': 1498, 'rebellion': 1499, 'austerity': 1500, 'parliament': 1501, 'Modi': 1502, 'leftist': 1503, 'problems': 1504, 'Getty': 1505, 'executives': 1506, 'Partners': 1507, 'Exchequer': 1508, 'cutting': 1509, 'officials': 1510, 'stakes': 1511, '‘': 1512, 'Oxford': 1513, 'receive': 1514, 'levels': 1515, 'automaker': 1516, '35': 1517, 'estimated': 1518, 'collection': 1519, 'Court': 1520, 'JPMorgan': 1521, 'Chase': 1522, 'solution': 1523, 'entertainment': 1524, 'primarily': 1525, 'plus': 1526, 'exclusive': 1527, 'strength': 1528, 'industries': 1529, 'marketplace': 1530, 'mean': 1531, 'opened': 1532, 'Nike': 1533, 'primary': 1534, 'continuing': 1535, 'technologies': 1536, 'facing': 1537, 'original': 1538, 'discuss': 1539, 'designed': 1540, 'Online': 1541, 'trend': 1542, 'IPO': 1543, 'mentioned': 1544, 'November': 1545, "'m": 1546, 'East': 1547, 'round': 1548, 'check': 1549, 'response': 1550, 'self': 1551, 'Water': 1552, 'extended': 1553, 'mainly': 1554, 'deliveries': 1555, 'throughout': 1556, 'challenges': 1557, 'Tim': 1558, 'progress': 1559, 'states': 1560, 'longer': 1561, 'spending': 1562, 'community': 1563, 'Under': 1564, 'proposed': 1565, 'battery': 1566, 'Uber': 1567, 'traded': 1568, 'moved': 1569, 'Foundation': 1570, 'seek': 1571, 'Bob': 1572, 'Solutions': 1573, 'Coffee': 1574, 'Shanghai': 1575, '23': 1576, 'Spanish': 1577, 'Agency': 1578, 'decided': 1579, 'fashion': 1580, 'reason': 1581, 'gains': 1582, 'logistics': 1583, 'smaller': 1584, 'uranium': 1585, 'established': 1586, 'committee': 1587, 'trust': 1588, 'Ford': 1589, 'city': 1590, 'Peter': 1591, 'Coal': 1592, 'expectations': 1593, 'Sports': 1594, 'Evraz': 1595, 'Images': 1596, 'DI': 1597, 'Russia': 1598, 'infusion': 1599, 'Deutsche': 1600, 'decade': 1601, 'Index': 1602, 'Healthcare': 1603, 'elections': 1604, 'Birla': 1605, 'Amsterdam': 1606, 'Brothers': 1607, 'separate': 1608, 'cards': 1609, 'plants': 1610, 'introduced': 1611, 'mail': 1612, 'newly': 1613, '2009': 1614, 'compete': 1615, 'serve': 1616, 'versus': 1617, 'models': 1618, 'backed': 1619, 'address': 1620, 'me': 1621, 'auto': 1622, 'nine': 1623, 'means': 1624, 'front': 1625, 'directly': 1626, 'expanded': 1627, 'particular': 1628, 'COVID': 1629, 'items': 1630, 'quickly': 1631, 'always': 1632, 'Dynamics': 1633, 'vice': 1634, 'Liberty': 1635, 'client': 1636, 'branded': 1637, 'makes': 1638, 'asset': 1639, 'offerings': 1640, 'More': 1641, 'Data': 1642, 'uses': 1643, 'Association': 1644, 'ongoing': 1645, 'Cook': 1646, 'funding': 1647, 'volumes': 1648, 'ecosystem': 1649, 'Department': 1650, 'zone': 1651, 'book': 1652, 'outside': 1653, 'Investor': 1654, 'campaign': 1655, 'groups': 1656, '200': 1657, 'Siemens': 1658, 'located': 1659, 'Petrobras': 1660, 'processes': 1661, 'lost': 1662, 'apps': 1663, 'question': 1664, 'active': 1665, 'wing': 1666, 'Property': 1667, 'airport': 1668, 'remaining': 1669, 'location': 1670, 'Chile': 1671, 'February': 1672, 'sugar': 1673, 'de': 1674, 'Hotel': 1675, 'overnight': 1676, 'fast': 1677, 'Nations': 1678, '1.5': 1679, 'investments': 1680, 'resources': 1681, 'pricing': 1682, 'acquisitions': 1683, 'When': 1684, 'try': 1685, 'present': 1686, 'Vice': 1687, 'flight': 1688, 'HDFC': 1689, 'Big': 1690, 'makers': 1691, 'Beijing': 1692, 'care': 1693, 'smartphones': 1694, 'd': 1695, 'holding': 1696, 'Associated': 1697, 'tonnes': 1698, 'too': 1699, 'Fonterra': 1700, 'Carphone': 1701, 'Airport': 1702, 'gambling': 1703, '+': 1704, 'referendum': 1705, 'jumped': 1706, '0': 1707, 'ET': 1708, 'Bill': 1709, 'Centrica': 1710, 'Journal': 1711, 'S&P': 1712, 'White': 1713, 'Trump': 1714, 'Battery': 1715, 'Yellen': 1716, 'Kenya': 1717, 'MSCI': 1718, 'TechMARK': 1719, 'Harvey': 1720, 'court': 1721, 'Voyager': 1722, 'Alphabet': 1723, 'England': 1724, 'analyst': 1725, 'operators': 1726, 'Italian': 1727, 'improving': 1728, 'eight': 1729, 'transport': 1730, 'largely': 1731, 'allows': 1732, 'lending': 1733, 'stop': 1734, 'shopping': 1735, 'aviation': 1736, 'pace': 1737, 'bringing': 1738, 'facility': 1739, 'statements': 1740, 'let': 1741, 'helping': 1742, 'exposure': 1743, 'Banks': 1744, 'driving': 1745, 'betting': 1746, 'P&G': 1747, 'factors': 1748, 'placed': 1749, 'closely': 1750, 'Buy': 1751, 'subscribers': 1752, 'showed': 1753, 'entered': 1754, 'becoming': 1755, 'restructuring': 1756, 'positioned': 1757, 'areas': 1758, 'option': 1759, 'locations': 1760, 'thing': 1761, 'balance': 1762, 'testing': 1763, 'Toronto': 1764, 'Commercial': 1765, 'creating': 1766, 'activity': 1767, 'No': 1768, 'person': 1769, 'am': 1770, 'There': 1771, 'final': 1772, 'iOS': 1773, 'protect': 1774, 'Seattle': 1775, 'emergency': 1776, 'minimal': 1777, 'hand': 1778, 'Universal': 1779, 'Pictures': 1780, 'federal': 1781, '42': 1782, '400': 1783, 'brought': 1784, 'Alibaba': 1785, 'changing': 1786, 'Industrial': 1787, 'Conference': 1788, 'Innovation': 1789, 'Nintendo': 1790, 'League': 1791, 'Spain': 1792, 'Even': 1793, 'old': 1794, 'trying': 1795, 'West': 1796, 'controls': 1797, 'Washington': 1798, 'purchased': 1799, 'adjusted': 1800, 'luxury': 1801, 'pipeline': 1802, 'complex': 1803, 'Video': 1804, 'furniture': 1805, 'BlackRock': 1806, 'Earlier': 1807, 'Shenzhen': 1808, 'considering': 1809, 'Companies': 1810, '2010': 1811, 'morning': 1812, 'struggling': 1813, 'Detroit': 1814, 'Mary': 1815, 'Barra': 1816, 'Dixons': 1817, 'failed': 1818, 'AFP': 1819, 'announcing': 1820, 'Narendra': 1821, 'Comments': 1822, 'Mobile': 1823, 'Malaysia': 1824, 'speaks': 1825, 'Balfour': 1826, 'Beatty': 1827, 'rejected': 1828, 'summit': 1829, 'economist': 1830, 'Merkel': 1831, 'billionaire': 1832, 'programme': 1833, 'Lloyd': 1834, 'lowest': 1835, 'Club': 1836, 'PeopleSoft': 1837, 'commodities': 1838, 'Fairfax': 1839, 'Janet': 1840, 'gain': 1841, 'Volkswagen': 1842, 'opposition': 1843, 'Tech': 1844, 'Mahindra': 1845, 'foreign': 1846, 'Foxconn': 1847, 'flights': 1848, 'Uchumi': 1849, 'details': 1850, 'wants': 1851, 'stronger': 1852, 'figures': 1853, 'previous': 1854, 'programs': 1855, 'appointed': 1856, 'effectively': 1857, 'clothing': 1858, 'whole': 1859, 'Toyota': 1860, 'margins': 1861, 'greater': 1862, 'flow': 1863, 'chip': 1864, 'Number': 1865, 'analytics': 1866, 'Medical': 1867, 'filing': 1868, 'machine': 1869, 'Some': 1870, 'generate': 1871, 'committed': 1872, 'technical': 1873, 'They': 1874, 'served': 1875, 'You': 1876, 'wanted': 1877, 'gaming': 1878, 'pounds': 1879, 'planes': 1880, 'operated': 1881, 'raising': 1882, 'farmers': 1883, 'Comcast': 1884, 'announce': 1885, 'profitable': 1886, 'tools': 1887, 'King': 1888, 'declined': 1889, 'complete': 1890, 'requirements': 1891, 'Motor': 1892, 'disclosed': 1893, 'investing': 1894, 'What': 1895, 'maintain': 1896, 'SAP': 1897, 'giving': 1898, 'savings': 1899, 'roughly': 1900, 'red': 1901, 'Zealand': 1902, 'excited': 1903, 'Avon': 1904, 'organizations': 1905, 'effort': 1906, 'invested': 1907, 'file': 1908, 'proposal': 1909, 'decades': 1910, 'Q1': 1911, 'coverage': 1912, 'partially': 1913, 'FOX': 1914, 'certain': 1915, 'Francisco': 1916, 'political': 1917, 'K': 1918, 'Star': 1919, 'Year': 1920, 'X': 1921, 'followed': 1922, 'green': 1923, 'cocoa': 1924, 'Panasonic': 1925, 'Northern': 1926, 'anti': 1927, 'withstand': 1928, 'failure': 1929, 'materials': 1930, 'implement': 1931, 'host': 1932, 'branch': 1933, 'hour': 1934, 'transactions': 1935, 'Skype': 1936, 'Mac': 1937, 'movie': 1938, 'described': 1939, 'confidence': 1940, 'rest': 1941, 'standard': 1942, 'Wella': 1943, 'Queensland': 1944, 'Goldcorp': 1945, 'stage': 1946, 'upgrade': 1947, 'discount': 1948, 'rivals': 1949, 'crash': 1950, 'Tullow': 1951, 'Trading': 1952, 'whose': 1953, 'Asset': 1954, 'T': 1955, 'Sierra': 1956, 'Consumer': 1957, 'Girl': 1958, 'ups': 1959, 'B': 1960, 'Reddit': 1961, 'Source': 1962, 'photo': 1963, 'night': 1964, 'survey': 1965, 'rescue': 1966, 'Las': 1967, 'broker': 1968, 'Associates': 1969, 'Worldwide': 1970, 'approved': 1971, 'Asian': 1972, 'vendors': 1973, '2005': 1974, 'Hill': 1975, 'Platinum': 1976, 'brokerage': 1977, 'Western': 1978, 'Ten': 1979, 'nod': 1980, 'split': 1981, 'wrote': 1982, 'Chartered': 1983, 'fallen': 1984, 'dropped': 1985, 'R': 1986, 'Sept.': 1987, 'Android': 1988, 'Chanda': 1989, 'Kochhar': 1990, 'caused': 1991, 'Lehman': 1992, 'poor': 1993, 'stations': 1994, 'agreements': 1995, 'civil': 1996, 'Among': 1997, 'joined': 1998, 'massive': 1999, 'Jamie': 2000, 'looks': 2001, 'manufacturers': 2002, 'generally': 2003, 'flying': 2004, 'travel': 2005, 'mix': 2006, 'pushing': 2007, 'Max': 2008, 'consortium': 2009, 'NBC': 2010, 'everything': 2011, 'How': 2012, 'upcoming': 2013, 'drivers': 2014, 'sort': 2015, 'extra': 2016, 'traffic': 2017, 'Teams': 2018, '10,000': 2019, 'field': 2020, 'select': 2021, 'talent': 2022, 'These': 2023, 'instead': 2024, 'bigger': 2025, 'associated': 2026, 'multiple': 2027, 'audit': 2028, 'Columbia': 2029, 'Program': 2030, 'Brands': 2031, 'facilities': 2032, 'toward': 2033, 'recognized': 2034, 'application': 2035, '2021': 2036, 'Security': 2037, 'Information': 2038, 'necessary': 2039, 'Nick': 2040, 'bad': 2041, 'meat': 2042, 'volatility': 2043, 'leave': 2044, 'heritage': 2045, 'yesterday': 2046, 'initiatives': 2047, 'Puerto': 2048, 'Rico': 2049, 'electric': 2050, 'Q3': 2051, 'competitors': 2052, 'steam': 2053, 'follow': 2054, 'quite': 2055, 'dividend': 2056, 'utility': 2057, 'request': 2058, 'skills': 2059, 'airlines': 2060, 'completed': 2061, 'attract': 2062, 'cannabis': 2063, 'body': 2064, 'relief': 2065, 'healthcare': 2066, 'structure': 2067, 'Since': 2068, 'yet': 2069, 'shows': 2070, 'Meanwhile': 2071, 'successful': 2072, 'highest': 2073, 'regarding': 2074, 'above': 2075, 'purchases': 2076, 'Economic': 2077, 'device': 2078, 'innovation': 2079, 'chocolate': 2080, 'declining': 2081, 'Land': 2082, 'resume': 2083, 'Relations': 2084, 'comments': 2085, 'steps': 2086, 'variety': 2087, 'Maps': 2088, 'RSA': 2089, 'communications': 2090, 'successfully': 2091, 'efficiency': 2092, 'An': 2093, 'Daimler': 2094, 'Audi': 2095, 'student': 2096, 'Price': 2097, 'Satya': 2098, 'Michigan': 2099, 'however': 2100, 'joining': 2101, 'Service': 2102, 'shift': 2103, 'panel': 2104, 'R6C1.DE': 2105, 'enabling': 2106, 'Japanese': 2107, 'reporters': 2108, 'plane': 2109, 'BRUSSELS': 2110, 'fees': 2111, 'exploration': 2112, 'Pharmaceuticals': 2113, 'revealed': 2114, '#': 2115, 'Moneylife': 2116, 'IANS': 2117, 'union': 2118, 'Santander': 2119, 'DETROIT': 2120, 'Man': 2121, 'LMI': 2122, 'Acacia': 2123, 'Cheil': 2124, 'Secretary': 2125, 'Donald': 2126, 'metals': 2127, 'Resorts': 2128, 'Vegas': 2129, 'asked': 2130, 'FM': 2131, 'election': 2132, 'Copyright': 2133, 'claims': 2134, 'iron': 2135, 'fixed': 2136, 'backing': 2137, 'DATE': 2138, 'regulators': 2139, 'find': 2140, 'employee': 2141, 'insurer': 2142, 'Murdoch': 2143, 'C$': 2144, 'fee': 2145, 'tie': 2146, 'HD': 2147, 'export': 2148, 'Moynihan': 2149, 'giants': 2150, 'cents': 2151, 'Suzuki': 2152, 'BSE': 2153, 'InBev': 2154, 'relatively': 2155, '2007': 2156, 'processing': 2157, 'documents': 2158, 'immediate': 2159, 'positions': 2160, 'Dimon': 2161, 'confirmed': 2162, 'department': 2163, 'ready': 2164, 'feel': 2165, 'involved': 2166, 'Netflix': 2167, 'closer': 2168, 'ways': 2169, 'unique': 2170, 'sharing': 2171, 'processor': 2172, 'critical': 2173, 'halved': 2174, 'District': 2175, 'perhaps': 2176, 'accelerate': 2177, 'Best': 2178, 'land': 2179, 'C': 2180, 'symbol': 2181, 'slide': 2182, 'hired': 2183, 'recorded': 2184, 'appears': 2185, 'Thailand': 2186, 'Venmo': 2187, 'takes': 2188, 'nothing': 2189, 'hotel': 2190, 'providers': 2191, 'consumption': 2192, 'pleased': 2193, 'turnaround': 2194, 'Park': 2195, 'title': 2196, 'certainly': 2197, 'professional': 2198, 'presented': 2199, 'granted': 2200, 'specific': 2201, 'sectors': 2202, 'slightly': 2203, 'protection': 2204, 'rather': 2205, 'seems': 2206, '2017': 2207, 'negative': 2208, 'Amazon.com': 2209, 'gained': 2210, 'EBITDA': 2211, '47': 2212, '’ve': 2213, 'preferred': 2214, 'transition': 2215, 'talking': 2216, 'dozens': 2217, 'slowdown': 2218, 'allowing': 2219, 'huge': 2220, 'A350': 2221, 'policies': 2222, 'highlighted': 2223, '90': 2224, 'keeping': 2225, 'places': 2226, 'buyers': 2227, 'partnered': 2228, 'gave': 2229, 'Intuit': 2230, 'uncertainty': 2231, '737': 2232, 'registered': 2233, 'Wells': 2234, 'Fargo': 2235, 'figure': 2236, 'beginning': 2237, 'film': 2238, 'broadband': 2239, 'connect': 2240, 'mostly': 2241, 'BMW': 2242, 'activities': 2243, 'Channel': 2244, 'Pinterest': 2245, 'prospects': 2246, 'ads': 2247, 'supplier': 2248, 'everyone': 2249, 'extremely': 2250, 'allowed': 2251, 'Just': 2252, 'relations': 2253, 'millions': 2254, 'profile': 2255, 'assembly': 2256, 'revised': 2257, 'Advanced': 2258, 'ending': 2259, 'note': 2260, 'Xiaomi': 2261, 'settlement': 2262, 'HT': 2263, 'Primark': 2264, 'Milan': 2265, 'GVC': 2266, 'Centre': 2267, 'Brussels': 2268, 'suspended': 2269, 'KSE': 2270, 'Dick': 2271, 'ACA': 2272, 'Nissan': 2273, 'SE': 2274, 'pound': 2275, 'faced': 2276, 'charge': 2277, 'housing': 2278, 'forced': 2279, 'Seoul': 2280, 'Yahoo': 2281, 'MENAFN': 2282, 'Mumbai': 2283, 'festival': 2284, 'widely': 2285, 'special': 2286, 'Taiwan': 2287, 'NV': 2288, 'DIVIDEND': 2289, 'attempt': 2290, 'Obama': 2291, 'Irish': 2292, 'MUMBAI': 2293, 'ERA': 2294, 'Nadella': 2295, 'Review': 2296, 'E2V': 2297, 'Al': 2298, 'Premium': 2299, 'S': 2300, 'strike': 2301, 'Sun': 2302, 'PARIS': 2303, 'brewing': 2304, '13:30': 2305, 'UDG': 2306, 'startup': 2307, 'travelling': 2308, '©': 2309, 'Anheuser': 2310, 'Busch': 2311, 'adviser': 2312, 'liquidity': 2313, 'broader': 2314, 'Walgreens': 2315, 'UBS': 2316, '2002': 2317, 'targets': 2318, 'introduce': 2319, 'Despite': 2320, 'mobility': 2321, 'interests': 2322, 'sending': 2323, 'Netherlands': 2324, 'PC': 2325, 'pickup': 2326, 'Pizza': 2327, 'speed': 2328, 'potentially': 2329, 'roles': 2330, 'blue': 2331, 'expertise': 2332, 'worked': 2333, 'Over': 2334, 'alignment': 2335, 'Next': 2336, 'personal': 2337, 'Board': 2338, 'either': 2339, 'individual': 2340, 'Cigna': 2341, 'Costco': 2342, 'relative': 2343, 'determine': 2344, 'events': 2345, 'middle': 2346, 'dependence': 2347, 'difference': 2348, 'lack': 2349, 'Li': 2350, 'exciting': 2351, 'procedures': 2352, 'Abu': 2353, 'Dhabi': 2354, 'merchants': 2355, 'LG': 2356, 'combine': 2357, 'Quality': 2358, 'School': 2359, 'improvement': 2360, 'Building': 2361, 'easy': 2362, 'reflects': 2363, 'Two': 2364, 'test': 2365, 'indicated': 2366, 'color': 2367, 'Square': 2368, 'Target': 2369, 'effects': 2370, 'generated': 2371, 'opening': 2372, 'viewing': 2373, 'operational': 2374, 'Get': 2375, 'increasingly': 2376, 'selected': 2377, 'maintenance': 2378, 'material': 2379, 'hot': 2380, 'school': 2381, 'audio': 2382, 'vast': 2383, 'immediately': 2384, 'web': 2385, 'phones': 2386, 'represents': 2387, 'quarters': 2388, '21st': 2389, 'storage': 2390, 'footprint': 2391, 'telecom': 2392, 'fruit': 2393, 'Access': 2394, 'summer': 2395, 'collapse': 2396, 'normal': 2397, 'prominent': 2398, '34': 2399, 'excess': 2400, 'study': 2401, 'remained': 2402, 'fastest': 2403, 'award': 2404, 'winning': 2405, 'struck': 2406, 'Eni': 2407, '36': 2408, '37': 2409, 'fair': 2410, 'questions': 2411, 'Fortune': 2412, 'Committee': 2413, 'understand': 2414, 'extend': 2415, 'Lake': 2416, 'subsidiaries': 2417, 'risks': 2418, 'Aviation': 2419, 'She': 2420, 'routes': 2421, 'internal': 2422, 'monthly': 2423, 'concern': 2424, 'ranked': 2425, 'shoppers': 2426, 'season': 2427, 'attractive': 2428, 'A330': 2429, 'Patrick': 2430, 'wind': 2431, 'worldwide': 2432, 'living': 2433, 'representatives': 2434, 'TNT': 2435, 'ownership': 2436, 'acquiring': 2437, 'Total': 2438, 'launching': 2439, 'CRH': 2440, ' ': 2441, 'contracts': 2442, 'Platform': 2443, 'Commerce': 2444, 'kitchen': 2445, 'From': 2446, 'Correspondent': 2447, 'Dudley': 2448, 'slumped': 2449, 'Telegraph': 2450, 'activist': 2451, 'Procter': 2452, 'Gamble': 2453, 'Strong': 2454, 'Research': 2455, 'straight': 2456, 'iconic': 2457, 'doll': 2458, 'p.m.': 2459, 'TheStreet': 2460, 'Equity': 2461, 'Monetary': 2462, 'Money': 2463, 'Costa': 2464, 'Myer': 2465, 're': 2466, 'Jones': 2467, 'emerging': 2468, 'TLW.L': 2469, 'wine': 2470, '150': 2471, 'discovered': 2472, 'Fidelity': 2473, 'reform': 2474, 'Team': 2475, 'Egypt': 2476, 'sent': 2477, 'McFarlane': 2478, 'unlikely': 2479, 'Treasury': 2480, 'bill': 2481, 'parties': 2482, 'Angela': 2483, 'Tobacco': 2484, 'overseas': 2485, 'Nigeria': 2486, 'Hilton': 2487, 'affiliates': 2488, 'broadcaster': 2489, 'missed': 2490, 'sanctions': 2491, 'Blankfein': 2492, 'Berlin': 2493, 'serving': 2494, 'formed': 2495, 'Verizon': 2496, 'Unilever': 2497, 'Roy': 2498, 'Chairperson': 2499, 'Wesfarmers': 2500, 'St': 2501, 'unveiled': 2502, 'Horizon': 2503, 'Metals': 2504, 'Care': 2505, 'Nine': 2506, 'Managing': 2507, 'lender': 2508, 'Nomura': 2509, 'Emirates': 2510, 'women': 2511, 'secretary': 2512, 'Realogy': 2513, 'Professional': 2514, 'Kenyan': 2515, 'Ge': 2516, 'NCC': 2517, 'Head': 2518, 'telecoms': 2519, 'Presto': 2520, 'discussion': 2521, 'Worldpay': 2522, 'Commonwealth': 2523, 'AAL.L': 2524, 'Marketing': 2525, 'daily': 2526, 'Coinbase': 2527, 'compensation': 2528, 'Nation': 2529, 'lifted': 2530, 'BBL': 2531, 'ticket': 2532, 'Rate': 2533, 'Galaxy': 2534, 'Ivan': 2535, 'Glasenberg': 2536, 'Must': 2537, 'refiner': 2538, 'rupee': 2539, 'Robert': 2540, 'sharply': 2541, 'Opel': 2542, 'Time': 2543, 'homes': 2544, 'broadcast': 2545, 'urged': 2546, 'weigh': 2547, 'closures': 2548, 'shared': 2549, 'Paul': 2550, 'owing': 2551, 'workforce': 2552, 'Xbox': 2553, 'sense': 2554, 'idea': 2555, 'privacy': 2556, 'cases': 2557, 'traditional': 2558, 'percentage': 2559, 'thousands': 2560, 'During': 2561, 'managers': 2562, '2018': 2563, 'Maersk': 2564, 'road': 2565, 'Bureau': 2566, 'environmental': 2567, 'consider': 2568, 'diversified': 2569, 'engaged': 2570, 'discussions': 2571, 'ex': 2572, 'concerned': 2573, 'FireEye': 2574, 'shop': 2575, 'expense': 2576, 'problem': 2577, 'confident': 2578, 'Although': 2579, 'specialty': 2580, 'Live': 2581, 'subscription': 2582, 'competitor': 2583, 'respect': 2584, 'heavily': 2585, 'function': 2586, 'cover': 2587, 'force': 2588, 'happened': 2589, 'happen': 2590, 'impressive': 2591, 'Walker': 2592, 'never': 2593, 'sized': 2594, 'picture': 2595, 'offline': 2596, 'satellite': 2597, 'hybrid': 2598, 'Point': 2599, 'recognition': 2600, 'commitment': 2601, 'magazine': 2602, 'Dr.': 2603, 'reserves': 2604, 'supporting': 2605, 'goals': 2606, 'functionality': 2607, 'tool': 2608, 'Government': 2609, 'impacted': 2610, 'output': 2611, 'driver': 2612, 'Partnership': 2613, 'Southern': 2614, 'controlled': 2615, 'broad': 2616, 'modest': 2617, 'restaurants': 2618, 'calls': 2619, 'phase': 2620, 'Morris': 2621, 'brings': 2622, 'Focus': 2623, '80': 2624, 'transformation': 2625, 'and/or': 2626, 'requests': 2627, 'entering': 2628, '75': 2629, 'moment': 2630, 'clearly': 2631, 'studio': 2632, 'names': 2633, 'compliance': 2634, 'visit': 2635, 'Dallas': 2636, 'ground': 2637, 'underway': 2638, 'noted': 2639, 'affiliate': 2640, '787': 2641, 'stream': 2642, 'resulting': 2643, 'designs': 2644, 'passengers': 2645, 'Manager': 2646, 'temporary': 2647, 'closure': 2648, 'Latin': 2649, 'qualified': 2650, 'answer': 2651, 'audience': 2652, 'jointly': 2653, 'cargo': 2654, 'Logistics': 2655, 'boosted': 2656, 'pick': 2657, 'developers': 2658, 'degree': 2659, 'reducing': 2660, 'cross': 2661, 'declines': 2662, 'licensing': 2663, 'bonds': 2664, 'downturn': 2665, 'Airbnb': 2666, 'Volvo': 2667, 'often': 2668, 'commitments': 2669, 'something': 2670, 'welcome': 2671, 'OpenShift': 2672, 'Following': 2673, 'Lufthansa': 2674, 'calling': 2675, 'Deputy': 2676, 'training': 2677, 'surge': 2678, 'performing': 2679, 'Woolworths': 2680, 'minimum': 2681, 'bottom': 2682, 'aimed': 2683, 'damages': 2684, 'People': 2685, 'smart': 2686, 'Annual': 2687, 'fifth': 2688, 'inside': 2689, 'children': 2690, 'Leader': 2691, 'works': 2692, 'Dentsu': 2693, 'alcohol': 2694, 'replace': 2695, 'sharp': 2696, 'AirAsia': 2697, 'TOKYO': 2698, 'Nairobi': 2699, 'ride': 2700, 'surprise': 2701, 'Zacks': 2702, 'TLW': 2703, 'slump': 2704, 'SSYS': 2705, 'Income': 2706, 'Greeks': 2707, 'Justice': 2708, 'WASHINGTON': 2709, 'BJP': 2710, 'lawmakers': 2711, 'consulting': 2712, 'Gates': 2713, 'dollars': 2714, 'centre': 2715, 'man': 2716, 'dispute': 2717, 'Silverado': 2718, 'drinks': 2719, 'COMMENT': 2720, 'Richard': 2721, 'Banco': 2722, 'PMO.L': 2723, 'attention': 2724, 'Thomas': 2725, 'Key': 2726, 'Show': 2727, 'Top': 2728, 'Investments': 2729, '120': 2730, 'Hacking': 2731, 'Hartzer': 2732, 'hosted': 2733, 'ANZ': 2734, 'climbed': 2735, 'session': 2736, 'Alliance': 2737, 'Scotiabank': 2738, 'securities': 2739, 'Manufacturing': 2740, 'Russell': 2741, 'AFL': 2742, 'agree': 2743, 'dolls': 2744, 'Chilean': 2745, 'Paytm': 2746, 'celebration': 2747, 'ArcelorMittal': 2748, 'Wynn': 2749, 'Sands': 2750, 'Philip': 2751, 'challenge': 2752, 'SAN': 2753, 'FRANCISCO': 2754, 'Better': 2755, 'Geographic': 2756, 'Distribution': 2757, 'Thu': 2758, 'Korean': 2759, 'Sainsbury': 2760, 'Launch': 2761, '3.5': 2762, 'Institute': 2763, 'Partner': 2764, 'PMO': 2765, 'visual': 2766, 'ANNOUNCEMENTS': 2767, 'professor': 2768, 'Bwin': 2769, 'accepted': 2770, '1.4': 2771, 'interim': 2772, 'Josh': 2773, 'Earnest': 2774, 'chips': 2775, 'Alstom': 2776, '700': 2777, 'Barack': 2778, 'Support': 2779, 'Fri': 2780, 'Social': 2781, 'Duracell': 2782, 'Macquarie': 2783, 'Rupert': 2784, 'southern': 2785, 'Stuttgart': 2786, '49': 2787, 'Norman': 2788, 'trucks': 2789, 'wage': 2790, 'Marine': 2791, 'interview': 2792, '>': 2793, 'earn': 2794, 'pct': 2795, 'Grexit': 2796, 'destinations': 2797, 'Perth': 2798, 'AMP': 2799, 'Freddie': 2800, 'Wanda': 2801, 'editor': 2802, 'Miles': 2803, 'Additional': 2804, 'USAA': 2805, 'SDL': 2806, 'KCOM': 2807, 'Bangkok': 2808, 'PMI': 2809, 'wholly': 2810, 'inaugurates': 2811, 'inaugurate': 2812, '5,934': 2813, 'Walkman': 2814, 'Alitalia': 2815, 'Nexus': 2816, 'Amlin': 2817, 'Steve': 2818, 'sought': 2819, 'sites': 2820, 'M&A': 2821, 'Adam': 2822, 'carry': 2823, 'merge': 2824, 'spin': 2825, 'gainers': 2826, 'quick': 2827, 'permanent': 2828, 'institutions': 2829, 'chains': 2830, 'halted': 2831, 'ministry': 2832, '12,000': 2833, 'Web': 2834, 'leaving': 2835, '48': 2836, 'Halliburton': 2837, 'setting': 2838, 'signs': 2839, 'TJX': 2840, '2022': 2841, 'wo': 2842, 'advertisers': 2843, 'HBO': 2844, 'absolutely': 2845, 'outdoor': 2846, 'proven': 2847, 'Chris': 2848, 'connected': 2849, 'strategies': 2850, 'ID': 2851, 'talked': 2852, 'showing': 2853, 'consistent': 2854, "'d": 2855, 'patients': 2856, 'Analytics': 2857, 'thinking': 2858, 'shifted': 2859, 'remember': 2860, 'enhance': 2861, 'CRM': 2862, 'poker': 2863, 'adoption': 2864, 'anyone': 2865, '130': 2866, 'miles': 2867, 'sure': 2868, 'Special': 2869, 'pure': 2870, 'regions': 2871, 'trends': 2872, 'Black': 2873, 'views': 2874, 'partnering': 2875, 'merchant': 2876, 'print': 2877, 'goes': 2878, 'funded': 2879, 'sponsored': 2880, 'standpoint': 2881, 'County': 2882, 'bulk': 2883, 'standing': 2884, 'supported': 2885, 'properties': 2886, 'limited': 2887, 'fewer': 2888, 'changed': 2889, 'Joe': 2890, 'volatile': 2891, 'achieve': 2892, 'sub': 2893, 'offices': 2894, 'outstanding': 2895, 'components': 2896, 'stretch': 2897, 'ITV': 2898, 'effective': 2899, 'insights': 2900, 'relevant': 2901, 'regulations': 2902, 'developer': 2903, 'globally': 2904, 'Authority': 2905, 'contractor': 2906, 'centers': 2907, 'Zillow': 2908, 'ask': 2909, 'houses': 2910, 'collaboration': 2911, 'soft': 2912, 'Republic': 2913, '33': 2914, 'branches': 2915, 'premier': 2916, 'wholesale': 2917, 'putting': 2918, 'awards': 2919, 'communication': 2920, 'lease': 2921, 'MetLife': 2922, 'Mexican': 2923, 'contact': 2924, '1.6': 2925, 'silver': 2926, 'Fire': 2927, '2.7': 2928, 'extension': 2929, 'mind': 2930, 'shipments': 2931, 'Blue': 2932, 'Jimmy': 2933, '0.4': 2934, 'room': 2935, 'QuickBooks': 2936, 'consecutive': 2937, 'hoping': 2938, 'Production': 2939, 'SVOD': 2940, 'returns': 2941, 'light': 2942, '66': 2943, 'returning': 2944, 'Division': 2945, 'feature': 2946, 'Simon': 2947, 'Tom': 2948, 'arrangement': 2949, 'secured': 2950, 'reaching': 2951, 'delays': 2952, 'nationwide': 2953, 'Enel': 2954, 'medical': 2955, 'artists': 2956, 'prepared': 2957, 'reality': 2958, 'starts': 2959, 'achieving': 2960, '900': 2961, 'creation': 2962, 'older': 2963, 'Natural': 2964, 'Chair': 2965, 'Call': 2966, 'train': 2967, 'residential': 2968, 'Precision': 2969, 'deposits': 2970, 'NBCUniversal': 2971, 'alone': 2972, 'wallet': 2973, 'eventually': 2974, 'although': 2975, 'afternoon': 2976, 'consolidation': 2977, 'cause': 2978, 'appliance': 2979, 'retain': 2980, 'rapid': 2981, 'plays': 2982, 'culture': 2983, 'read': 2984, 'moves': 2985, 'practices': 2986, 'opens': 2987, 'casino': 2988, 'deposit': 2989, 'Texas': 2990, 'improved': 2991, 'categories': 2992, 'Canfor': 2993, 'cheaper': 2994, 'Tencent': 2995, 'station': 2996, 'Frankfurt': 2997, 'jets': 2998, 'compact': 2999, 'comprehensive': 3000, 'offshore': 3001, 'Linux': 3002, 'roll': 3003, 'Gazprom': 3004, 'Ocado': 3005, 'hundreds': 3006, 'freight': 3007, 'Ground': 3008, 'tied': 3009, 'discussed': 3010, 'KPMG': 3011, 'Nordstrom': 3012, 'tourism': 3013, 'factor': 3014, 'interested': 3015, 'underlying': 3016, 'strengthen': 3017, 'education': 3018, 'Pro': 3019, 'grant': 3020, '1.3': 3021, 'interconnection': 3022, 'fight': 3023, 'drops': 3024, 'aluminium': 3025, 'Mario': 3026, 'weekend': 3027, 'meant': 3028, 'Italia': 3029, 'THE': 3030, 'logo': 3031, 'Rank': 3032, 'prevent': 3033, 'resolution': 3034, 'bonus': 3035, 'leasing': 3036, 'Iran': 3037, 'Coca': 3038, 'Cola': 3039, 'Products': 3040, 'Stoxx': 3041, 'tracks': 3042, 'tea': 3043, 'vegetable': 3044, 'Automobiles': 3045, 'commission': 3046, 'aid': 3047, 'proposals': 3048, 'Kuala': 3049, 'Lumpur': 3050, 'Food': 3051, 'Thodey': 3052, 'Burberry': 3053, 'BRBY': 3054, 'warning': 3055, 'Vedanta': 3056, 'military': 3057, 'voters': 3058, 'ones': 3059, 'race': 3060, 'Realty': 3061, 'float': 3062, 'famous': 3063, 'seeks': 3064, 'tumbled': 3065, '2000': 3066, 'battle': 3067, 'Bratz': 3068, 'favour': 3069, 'Hasbro': 3070, 'SEC': 3071, '51': 3072, 'holdings': 3073, 'chair': 3074, 'Paladin': 3075, 'AAL': 3076, 'Homes': 3077, 'computers': 3078, 'experts': 3079, 'Harvard': 3080, 'SYDNEY': 3081, 'licence': 3082, 'Trade': 3083, 'DFS': 3084, 'rules': 3085, 'Rimini': 3086, 'Suite': 3087, '32': 3088, 'fly': 3089, 'DLG': 3090, 'Islamic': 3091, 'founders': 3092, 'Sergey': 3093, 'Brin': 3094, 'rebels': 3095, 'voting': 3096, 'Personal': 3097, 'SSE': 3098, 'EX': 3099, 'Banamex': 3100, 'turned': 3101, 'pays': 3102, 'Dr': 3103, 'Rolls': 3104, 'Royce': 3105, 'speech': 3106, 'Accra': 3107, 'Williams': 3108, 'claim': 3109, 'ETF': 3110, 'Ventures': 3111, 'victory': 3112, 'Antofagasta': 3113, 'SEOUL': 3114, 'table': 3115, 'chairperson': 3116, 'insisted': 3117, 'Iron': 3118, 'Flight': 3119, 'Challenger': 3120, 'owners': 3121, 'GLEN': 3122, 'wife': 3123, 'basic': 3124, 'elected': 3125, 'additive': 3126, 'Johnston': 3127, 'prospect': 3128, 'investigation': 3129, 'licences': 3130, 'Betfair': 3131, 'Kaz': 3132, 'KAZ.L': 3133, 'KAZ': 3134, 'books': 3135, 'Philippines': 3136, 'modules': 3137, 'cooperation': 3138, 'spent': 3139, 'Yum': 3140, 'Tue': 3141, 'Construction': 3142, 'Growth': 3143, 'commissioner': 3144, 'insurers': 3145, 'climate': 3146, 'brokers': 3147, 'Sprint': 3148, '1.15': 3149, 'Maruti': 3150, 'Dalian': 3151, 'Cards': 3152, 'S6': 3153, 'Holding': 3154, 'lift': 3155, 'lakh': 3156, 'Vanguard': 3157, 'alternative': 3158, 'Mich.': 3159, 'NMC': 3160, 'Paddy': 3161, 'threatened': 3162, 'Warner': 3163, 'priority': 3164, 'registration': 3165, 'upgraded': 3166, 'valuations': 3167, 'reflect': 3168, 'bell': 3169, 'writing': 3170, 'replacing': 3171, 'HOUSTON': 3172, 'valuation': 3173, 'ordered': 3174, 'suit': 3175, 'Depot': 3176, 'PepsiCo': 3177, 'door': 3178, 'Chrome': 3179, 'appeals': 3180, 'transportation': 3181, 'decrease': 3182, 'personnel': 3183, 'Customer': 3184, 'Copenhagen': 3185, 'hydrogen': 3186, 'published': 3187, 'linear': 3188, 'consolidated': 3189, 'proposition': 3190, 'speak': 3191, 'virtual': 3192, 'deep': 3193, 'pulp': 3194, 'learning': 3195, 'Philadelphia': 3196, 'interesting': 3197, 'relationships': 3198, 'speaking': 3199, 'remote': 3200, 'Salesforce': 3201, 'participating': 3202, 'begins': 3203, 'Before': 3204, 'measure': 3205, 'beta': 3206, 'cell': 3207, 'segments': 3208, 'page': 3209, 'AMC': 3210, 'proprietary': 3211, 'capture': 3212, 'leveraging': 3213, 'Looking': 3214, 'Professor': 3215, 'Foreign': 3216, 'preparing': 3217, 'Dell': 3218, 'faster': 3219, 'simply': 3220, 'click': 3221, 'lucrative': 3222, 'defined': 3223, 'IP': 3224, 'enabled': 3225, 'converting': 3226, 'V': 3227, 'begin': 3228, 'milestone': 3229, 'upon': 3230, 'Penguin': 3231, 'Up': 3232, 'publishing': 3233, 'submitted': 3234, 'pickups': 3235, 'hearing': 3236, 'installed': 3237, 'Natalie': 3238, 'Avenue': 3239, 'cosmetics': 3240, 'Tire': 3241, 'Sears': 3242, 'respective': 3243, 'join': 3244, 'Enterprises': 3245, 'Games': 3246, 'achieved': 3247, 'mile': 3248, 'notice': 3249, 'affected': 3250, 'Connecticut': 3251, 'art': 3252, 'detailed': 3253, 'anticipated': 3254, 'equivalent': 3255, 'advisory': 3256, 'demands': 3257, 'Transportation': 3258, 'Card': 3259, 'charitable': 3260, 'organization': 3261, 'advise': 3262, 'transform': 3263, 'Crown': 3264, 'renewable': 3265, 'regulated': 3266, 'standards': 3267, 'Illinois': 3268, 'excellent': 3269, 'resolve': 3270, 'gives': 3271, '95': 3272, 'grown': 3273, 'lifestyle': 3274, 'NVIDIA': 3275, 'memory': 3276, 'wireless': 3277, 'reduced': 3278, 'Cross': 3279, 'legacy': 3280, 'Choo': 3281, 'flat': 3282, 'accessories': 3283, '1.2': 3284, 'standalone': 3285, 'constant': 3286, 'happy': 3287, 'defense': 3288, 'reductions': 3289, 'engagement': 3290, 'Monster': 3291, 'efficient': 3292, 'pet': 3293, 'inventory': 3294, 'aspects': 3295, 'availability': 3296, 'innovate': 3297, 'Forum': 3298, 'focusing': 3299, 'actual': 3300, 'Grand': 3301, 'Space': 3302, 'versions': 3303, 'completely': 3304, 'enjoyed': 3305, 'aims': 3306, 'covers': 3307, 'guest': 3308, 'hub': 3309, 'replaced': 3310, 'reflecting': 3311, 'cool': 3312, 'recovered': 3313, 'contribute': 3314, 'appetite': 3315, 'listing': 3316, 'signing': 3317, 'linked': 3318, 'restrictions': 3319, 'route': 3320, 'seats': 3321, 'passenger': 3322, 'Today': 3323, 'latter': 3324, 'Warburg': 3325, 'threat': 3326, 'ounces': 3327, '2.6': 3328, 'physical': 3329, 'Mercedes': 3330, 'Lexus': 3331, 'wider': 3332, 'Cameco': 3333, 'disruptions': 3334, 'goodwill': 3335, 'tariffs': 3336, 'timing': 3337, 'fluctuations': 3338, 'Aphria': 3339, 'grade': 3340, 'Port': 3341, 'Jack': 3342, '’ll': 3343, 'spoke': 3344, 'rooms': 3345, '8.5': 3346, 'tobacco': 3347, 'message': 3348, 'presentation': 3349, 'officially': 3350, 'Nucor': 3351, 'divisions': 3352, 'reduction': 3353, 'grades': 3354, 'tap': 3355, 'Country': 3356, 'Van': 3357, 'middleware': 3358, 'DNA': 3359, 'integrating': 3360, 'Sanofi': 3361, 'falls': 3362, 'Susan': 3363, 'club': 3364, 'Minnesota': 3365, 'estimate': 3366, 'Also': 3367, 'diverse': 3368, 'installation': 3369, 'Small': 3370, 'attack': 3371, 'valuable': 3372, 'story': 3373, 'alongside': 3374, 'governing': 3375, 'decisions': 3376, 'taxes': 3377, 'Any': 3378, 'Those': 3379, 'Play': 3380, 'CFO': 3381, 'Aegis': 3382, 'Shop': 3383, 'dominant': 3384, 'grid': 3385, 'electricity': 3386, 'raises': 3387, 'Iwata': 3388, '\ue071': 3389, 'Super': 3390, 'machines': 3391, 'death': 3392, 'Not': 3393, 'article': 3394, 'Elon': 3395, 'Musk': 3396, 'Basin': 3397, 'Another': 3398, 'beauty': 3399, 'carrying': 3400, 'crashing': 3401, 'draconian': 3402, 'Will': 3403, 'Gulf': 3404, 'US$': 3405, 'LNG': 3406, 'sees': 3407, 'builds': 3408, 'Sonic': 3409, 'IAG': 3410, '888': 3411, 'beating': 3412, 'crude': 3413, 'decide': 3414, 'welcomed': 3415, 'Congress': 3416, 'Bharatiya': 3417, 'Janata': 3418, 'annually': 3419, 'seeking': 3420, 'mutiny': 3421, 'Walter': 3422, 'Inns': 3423, '1,585.53p': 3424, '003160.KS': 3425, 'P': 3426, 'Christopher': 3427, 'Carol': 3428, 'VDNRF': 3429, '4WD': 3430, 'crown': 3431, 'Myntra': 3432, 'charged': 3433, 'affect': 3434, 'Corporate': 3435, 'letter': 3436, 'realize': 3437, 'Automotive': 3438, 'cr': 3439, 'costly': 3440, 'Lynch': 3441, 'candidates': 3442, 'career': 3443, 'toys': 3444, 'Prior': 3445, 'departure': 3446, 'Force': 3447, 'lose': 3448, 'Macau': 3449, 'sank': 3450, 'Place': 3451, 'Tesco': 3452, 'supermarket': 3453, 'Dodge': 3454, 'Popular': 3455, 'Hamburg': 3456, 'boosting': 3457, 'Admiral': 3458, 'mortgage': 3459, 'rental': 3460, 'SUVs': 3461, 'labor': 3462, 'human': 3463, 'Store': 3464, 'Ben': 3465, '86': 3466, 'revolt': 3467, 'MSFT': 3468, 'hackers': 3469, 'Farnell': 3470, 'Aramex': 3471, '4.5': 3472, 'XP': 3473, 'Both': 3474, 'LMI.L': 3475, 'St.': 3476, '210': 3477, 'Lockheed': 3478, 'Coatings': 3479, 'Technip': 3480, 'star': 3481, 'Taylor': 3482, 'reliable': 3483, 'PTI': 3484, 'RIO.L': 3485, 'bond': 3486, 'contained': 3487, 'agrees': 3488, 'Tunisia': 3489, 'ECONOMIC': 3490, '09:30': 3491, 'PAYMENT': 3492, 'Bond': 3493, 'slash': 3494, 'Tanzania': 3495, 'Senate': 3496, 'surged': 3497, 'counter': 3498, '12.4': 3499, 'Part': 3500, 'Carney': 3501, 'Elliott': 3502, 'founding': 3503, '4.1': 3504, 'Xstrata': 3505, 'Kelly': 3506, 'REA': 3507, 'Brambles': 3508, 'Metcash': 3509, 'South32': 3510, 'Editing': 3511, 'Rahul': 3512, 'govt': 3513, 'A320': 3514, 'Equinox': 3515, 'appliances': 3516, 'killed': 3517, 'Massachusetts': 3518, 'pension': 3519, 'Victoria': 3520, 'simplified': 3521, 'HKSE': 3522, 'Purchase': 3523, 'rallied': 3524, 'CENTURY': 3525, 'hopes': 3526, 'YouTube': 3527, 'Print': 3528, 'Videocon': 3529, 'RIO': 3530, 'Ophir': 3531, 'OPGYF': 3532, 'OPHR': 3533, 'Biomedica': 3534, 'OXB.L': 3535, 'OXB': 3536, 'Skyepharma': 3537, 'SKP': 3538, 'rally': 3539, 'police': 3540, 'Porsche': 3541, 'Arab': 3542, 'jet': 3543, 'Clark': 3544, 'firing': 3545, 'Attorney': 3546, 'miners': 3547, 'ECB': 3548, 'Orange': 3549, 'principle': 3550, 'ratings': 3551, 'Control': 3552, 'ERP': 3553, 'Kathmandu': 3554, 'string': 3555, 'finances': 3556, 'Jennifer': 3557, 'Car': 3558, 'Google+': 3559, 'empire': 3560, 'Democracy': 3561, 'expressed': 3562, 'Saudi': 3563, 'RTI': 3564, 'Form': 3565, 'Local': 3566, 'auction': 3567, 'watch': 3568, 'diesel': 3569, 'Phillips': 3570, 'forecasts': 3571, 'fines': 3572, 'CNBC': 3573, '<': 3574, 'presidential': 3575, 'F': 3576, 'bn': 3577, 'Project': 3578, 'cycling': 3579, 'SoulCycle': 3580, 'Cement': 3581, 'tablet': 3582, 'MD': 3583, 'Sinopec': 3584, 'producing': 3585, 'SunGard': 3586, 'connectivity': 3587, 'missing': 3588, 'default': 3589, 'hotels': 3590, 'Renault': 3591, 'actions': 3592, 'Fannie': 3593, 'Mae': 3594, 'Ambani': 3595, 'Porter': 3596, 'prospectus': 3597, '™': 3598, 'drachma': 3599, 'AO': 3600, 'denied': 3601, 'snap': 3602, 'Freeport': 3603, 'fellow': 3604, 'ABG.L': 3605, 'convert': 3606, 'ZocDoc': 3607, 'Jose': 3608, 'Boko': 3609, 'Haram': 3610, 'Abubakar': 3611, 'Shekau': 3612, '2.5': 3613, 'wages': 3614, 'Sharma': 3615, 'floated': 3616, 'Notes': 3617, 'Flint': 3618, 'piece': 3619, 'GX8.AS': 3620, 'MIUI': 3621, 'Dilip': 3622, 'pesa': 3623, 'ComScore': 3624, 'speculation': 3625, 'sister': 3626, 'Viacom': 3627, 'pull': 3628, 'specifically': 3629, 'wake': 3630, 'engineered': 3631, 'processors': 3632, 'veteran': 3633, 'loaded': 3634, 'declared': 3635, 'Discover': 3636, 'GE.N': 3637, 'Suisse': 3638, 'Talk': 3639, 'directors': 3640, 'judge': 3641, 'scandal': 3642, 'recession': 3643, 'Pepsi': 3644, 'apparel': 3645, 'Ireland': 3646, 'resuming': 3647, 'haul': 3648, 'label': 3649, 'pretty': 3650, 'Instacart': 3651, '73': 3652, 'lineup': 3653, 'Additionally': 3654, 'hire': 3655, 'Phil': 3656, 'Studios': 3657, 'double': 3658, 'digit': 3659, 'Zimbra': 3660, 'mission': 3661, 'packaging': 3662, 'Santa': 3663, 'stuff': 3664, 'Mandiant': 3665, 'Kroger': 3666, 'tenants': 3667, 'efficiently': 3668, 'DraftKings': 3669, 'lives': 3670, 'leads': 3671, 'Walt': 3672, 'Cleveland': 3673, 'Indians': 3674, 'environments': 3675, 'shape': 3676, 'promotion': 3677, 'license': 3678, 'gaining': 3679, 'integrate': 3680, 'Act': 3681, 'solid': 3682, '800': 3683, 'preliminary': 3684, 'encouraging': 3685, 'classic': 3686, '20th': 3687, 'Acquisition': 3688, 'flexibility': 3689, 'Chromebook': 3690, '7.2': 3691, 'connection': 3692, 'opposed': 3693, 'economics': 3694, 'Middle': 3695, 'controlling': 3696, 'Series': 3697, 'external': 3698, 'joins': 3699, 'incredibly': 3700, 'Well': 3701, 'experiences': 3702, 'productive': 3703, 'traditionally': 3704, 'bookings': 3705, 'Art': 3706, 'Burger': 3707, 'CD': 3708, 'Retirement': 3709, 'sheet': 3710, 'marked': 3711, 'Customers': 3712, 'Montreal': 3713, 'vessel': 3714, 'functions': 3715, 'Auckland': 3716, 'conducting': 3717, 'landmark': 3718, 'impairment': 3719, 'determined': 3720, 'receiving': 3721, 'suggesting': 3722, 'CBD': 3723, 'type': 3724, 'eligible': 3725, 'purchasing': 3726, 'campaigns': 3727, 'MBIA': 3728, 'Guarantee': 3729, 'motion': 3730, 'Atlantic': 3731, 'projected': 3732, 'topic': 3733, 'PlayStation': 3734, 'believed': 3735, 'radio': 3736, 'peers': 3737, 'scope': 3738, 'synergies': 3739, 'BC': 3740, 'likes': 3741, 'Swoosh': 3742, 'apartment': 3743, 'wins': 3744, 'Like': 3745, 'mention': 3746, 'hospitality': 3747, 'section': 3748, 'conservative': 3749, 'awareness': 3750, 'facts': 3751, 'Through': 3752, 'science': 3753, 'reduces': 3754, 'requires': 3755, '41': 3756, 'streams': 3757, 'releases': 3758, 'comprising': 3759, 'developments': 3760, 'modification': 3761, 'sponsor': 3762, 'lines': 3763, 'communities': 3764, 'upgrades': 3765, 'love': 3766, 'ad': 3767, 'compelling': 3768, 'harder': 3769, 'MSN': 3770, '140': 3771, 'delivering': 3772, 'Argentina': 3773, 'Cash': 3774, 'App': 3775, 'Paramount': 3776, 'shut': 3777, 'Versace': 3778, '39': 3779, 'Then': 3780, 'hardware': 3781, 'agent': 3782, 'Virginia': 3783, 'Northrop': 3784, '0.8': 3785, 'Metro': 3786, 'farmer': 3787, 'Jefferies': 3788, 'Mutual': 3789, 'Further': 3790, 'responded': 3791, 'winner': 3792, 'combining': 3793, 'purpose': 3794, 'NBA': 3795, '1st': 3796, 'consensus': 3797, 'rich': 3798, 'feedback': 3799, 'Payments': 3800, 'introducing': 3801, 'reasons': 3802, 'Indonesia': 3803, 'BA': 3804, 'recall': 3805, 'SFR': 3806, 'meaning': 3807, 'Saipem': 3808, 'expecting': 3809, 'creative': 3810, 'aware': 3811, 'stakeholders': 3812, 'presents': 3813, 'therefore': 3814, 'Infrastructure': 3815, 'Nord': 3816, 'yield': 3817, 'component': 3818, 'facilitate': 3819, 'ambition': 3820, 'Honda': 3821, 'computing': 3822, 'disruption': 3823, 'productivity': 3824, 'Rover': 3825, 'Embraer': 3826, 'alliance': 3827, 'Carlos': 3828, 'sustainability': 3829, 'wondering': 3830, 'repo': 3831, 'Pincus': 3832, 'landscape': 3833, 'fans': 3834, 'accelerating': 3835, 'suite': 3836, 'demonstrate': 3837, 'heater': 3838, '500,000': 3839, 'stand': 3840, 'trusted': 3841, 'Second': 3842, 'Agreement': 3843, 'Goals': 3844, 'consolidating': 3845, 'Valley': 3846, 'Given': 3847, 'tries': 3848, 'wish': 3849, 'true': 3850, 'heads': 3851, 'liability': 3852, 'Quarter': 3853, 'spot': 3854, 'Hollywood': 3855, 'concerning': 3856, 'pressures': 3857, 'avoid': 3858, 'intellectual': 3859, 'profitability': 3860, 'twice': 3861, 'Houston': 3862, 'enables': 3863, 'adds': 3864, 'responsibility': 3865, 'played': 3866, 'intends': 3867, 'gross': 3868, 'subscriber': 3869, 'shown': 3870, 'Lewis': 3871, 'map': 3872, 'partly': 3873, 'Antonio': 3874, 'boutique': 3875, 'safe': 3876, 'AudioCodes': 3877, 'ultimately': 3878, 'deployment': 3879, 'Altria': 3880, 'suggested': 3881, 'OEMs': 3882, 'Michelin': 3883, 'referring': 3884, 'Operations': 3885, 'corner': 3886, 'path': 3887, 'arms': 3888, 'types': 3889, 'slated': 3890, 'stable': 3891, 'defer': 3892, 'understanding': 3893, 'portion': 3894, 'powered': 3895, 'importance': 3896, 'edge': 3897, 'telco': 3898, 'stimulus': 3899, 'forgotten': 3900, 'daughter': 3901, 'digester': 3902, 'biogas': 3903, 'dealer': 3904, 'heat': 3905, 'leverage': 3906, 'packages': 3907, 'unless': 3908, 'fill': 3909, 'Sage': 3910, 'LLP': 3911, 'Carl': 3912, 'stood': 3913, 'Autodesk': 3914, 'choice': 3915, 'difficulties': 3916, 'shops': 3917, 'gets': 3918, 'player': 3919, 'aspect': 3920, 'Technical': 3921, 'materially': 3922, 'Malone': 3923, 'dominated': 3924, 'Working': 3925, 'scores': 3926, 'mines': 3927, 'Server': 3928, 'combat': 3929, 'supplies': 3930, 'Commitment': 3931, 'appointments': 3932, 'possibility': 3933, 'easier': 3934, 'bars': 3935, 'knows': 3936, 'Instagram': 3937, 'Note': 3938, 'solar': 3939, 'war': 3940, 'SEATTLE': 3941, '/PRNewswire/': 3942, 'jump': 3943, 'claiming': 3944, '4.7': 3945, 'Drahi': 3946, 'Your': 3947, 'expensive': 3948, 'OF': 3949, 'AA': 3950, 'bankers': 3951, 'Baker': 3952, 'rigs': 3953, 'exploring': 3954, 'lies': 3955, 'trillion': 3956, 'pushed': 3957, 'Clairol': 3958, 'recommendations': 3959, 'liquefied': 3960, 'Chevy': 3961, 'Bwin.party': 3962, 'heavy': 3963, 'IB': 3964, 'Banker': 3965, 'Anthem': 3966, 'edged': 3967, 'Gavin': 3968, 'concessions': 3969, 'Bengaluru': 3970, 'Three': 3971, 'Phone': 3972, 'struggled': 3973, 'Scentre': 3974, 'Atlanta': 3975, 'Administration': 3976, 'written': 3977, 'Workers': 3978, 'Kunal': 3979, 'Bahl': 3980, 'awarded': 3981, 'Jaresko': 3982, 'McDonald': 3983, 'SK': 3984, '1.9': 3985, 'schedule': 3986, 'Dan': 3987, 'High': 3988, 'Fisher': 3989, 'operative': 3990, 'Andy': 3991, 'Read': 3992, 'Major': 3993, 'limit': 3994, 'litigation': 3995, 'EMG': 3996, 'Allied': 3997, 'VED': 3998, 'Ethiopia': 3999, 'credibility': 4000, 'Costolo': 4001, 'aiming': 4002, 'startups': 4003, 'claimed': 4004, 'wary': 4005, 'Consulting': 4006, 'ExpressRoute': 4007, 'floor': 4008, 'Dorsey': 4009, '20,000': 4010, 'rent': 4011, 'young': 4012, 'telecommunications': 4013, 'Tusk': 4014, 'forth': 4015, 'MGA': 4016, 'meetings': 4017, 'Account': 4018, 'Penn': 4019, 'Picture': 4020, 'articles': 4021, 'Benz': 4022, 'anchor': 4023, 'defended': 4024, 'leaves': 4025, 'Content': 4026, 'steady': 4027, 'extending': 4028, 'substantially': 4029, 'father': 4030, 'Hancock': 4031, 'Housing': 4032, 'Story': 4033, 'Subscribe': 4034, 'typically': 4035, 'Collection': 4036, 'drink': 4037, 'AAPL.O': 4038, 'studios': 4039, 'Nokia': 4040, 'Ray': 4041, 'CITY': 4042, 'spokeswoman': 4043, 'Analysts': 4044, 'Tutor': 4045, 'Perini': 4046, 'Ono': 4047, 'ONO': 4048, 'functional': 4049, '3.1': 4050, 'Sale': 4051, 'Minneapolis': 4052, 'cigarette': 4053, 'electrical': 4054, 'numerous': 4055, 'baby': 4056, 'acting': 4057, 'hr': 4058, 'posting': 4059, 'N': 4060, 'Brown': 4061, 'suffered': 4062, 'AIF': 4063, 'Centamin': 4064, 'AAPL': 4065, 'al': 4066, 'Porat': 4067, 'depreciation': 4068, 'proved': 4069, 'ADM': 4070, 'Brisbane': 4071, 'TPG': 4072, 'a.m.': 4073, 'responsible': 4074, 'governments': 4075, 'Transurban': 4076, 'Qube': 4077, '100,000': 4078, 'CNA.L': 4079, 'GKN': 4080, 'RDSB': 4081, 'Mon': 4082, 'MPs': 4083, 'BRW1.DE': 4084, 'PayPoint': 4085, 'GB': 4086, 'Samuel': 4087, 'LP': 4088, 'Bharat': 4089, 'Bell': 4090, 'kilogram': 4091, 'metal': 4092, 'fears': 4093, 'Domino': 4094, 'regulator': 4095, 'UAW': 4096, 'premarket': 4097, 'apparently': 4098, 'Holcim': 4099, 'succession': 4100, 'Glass': 4101, 'write': 4102, 'Aurizon': 4103, 'Arrium': 4104, 'CSL': 4105, 'Spark': 4106, 'Mirvac': 4107, 'duty': 4108, 'Wholesale': 4109, 'Gandhi': 4110, 'Here': 4111, 'Save': 4112, 'Morrison': 4113, 'Wm': 4114, 'Supermarkets': 4115, 'MRW': 4116, 'catch': 4117, 'letters': 4118, 'Sao': 4119, 'Jan.': 4120, 'Abbott': 4121, 'attend': 4122, 'Kevin': 4123, 'topped': 4124, 'desktop': 4125, 'approve': 4126, '52': 4127, 'LafargeHolcim': 4128, 'email': 4129, 'Esure': 4130, 'motor': 4131, 'Alex': 4132, 'Aditya': 4133, 'Weir': 4134, 'WEIR': 4135, 'Trulia': 4136, 'EVR': 4137, 'E2V.L': 4138, 'Check': 4139, 'Senior': 4140, 'creator': 4141, 'Lisa': 4142, 'hurt': 4143, 'Movies': 4144, 'exchanges': 4145, 'representative': 4146, 'Jazztel': 4147, 'worries': 4148, 'documentation': 4149, 'Planning': 4150, 'entry': 4151, 'administration': 4152, 'spinoff': 4153, 'quoted': 4154, 'stepping': 4155, 'billions': 4156, 'born': 4157, 'Gerry': 4158, 'Duke': 4159, '1999': 4160, 'Xinhua': 4161, 'serious': 4162, 'hourly': 4163, 'hope': 4164, 'Westacott': 4165, 'imports': 4166, 'handling': 4167, 'Economy': 4168, 'Brazilian': 4169, 'votes': 4170, 'p': 4171, 'Engineering': 4172, 'electronic': 4173, 'SPECIAL': 4174, 'debate': 4175, 'Hyundai': 4176, '3.7': 4177, 'representing': 4178, 'Eastern': 4179, 'Tushar': 4180, 'Srivastava': 4181, 'employers': 4182, 'SpiceJet': 4183, 'Republican': 4184, 'Carmichael': 4185, 'Human': 4186, 'SANTIAGO': 4187, 'tycoon': 4188, '0.3': 4189, 'hike': 4190, 'indigenous': 4191, 'Stmt': 4192, 'trouble': 4193, 'RENT': 4194, 'Springer': 4195, '890': 4196, 'subject': 4197, 'Louis': 4198, 'ca': 4199, 'Several': 4200, 'Andrew': 4201, 'Quay': 4202, 'ruled': 4203, 'Jean': 4204, 'Juncker': 4205, 'CF': 4206, 'yuan': 4207, 'carried': 4208, 'Saxo': 4209, 'Danish': 4210, 'Aussie': 4211, 'buys': 4212, '-led': 4213, 'Greg': 4214, 'Novelli': 4215, 'Kotak': 4216, 'VCT': 4217, 'asking': 4218, '04': 4219, 'advice': 4220, 'm.': 4221, 'prolonged': 4222, 'corruption': 4223, 'violated': 4224, 'carriers': 4225, 'Why': 4226, 'Football': 4227, 'Pvt': 4228, 'damage': 4229, 'EMG.L': 4230, 'DVD': 4231, 'Tyson': 4232, 'Ansell': 4233, 'AmEx': 4234, 'tight': 4235, 'MS': 4236, 'imported': 4237, 'Schaeuble': 4238, 'Dow': 4239, 'locally': 4240, 'investigating': 4241, 'Caterpillar': 4242, 'PSU': 4243, 'reportedly': 4244, 'MOSCOW': 4245, 'Kimberly': 4246, 'sedan': 4247, 'Shanghvi': 4248, 'Fino': 4249, 'PayTech': 4250, 'Depository': 4251, 'Instruments': 4252, 'plunged': 4253, 'Kettering': 4254, 'paise': 4255, 'SoftBank': 4256, 'Week': 4257, 'privately': 4258, 'ASX': 4259, 'Vogogo': 4260, 'luggage': 4261, 'Haier': 4262, 'Sep': 4263, 'WhatsApp': 4264, 'Kwid': 4265, 'Corus': 4266, 'Pesa': 4267, 'Jobs': 4268, 'weight': 4269, 'simple': 4270, 'treat': 4271, 'merged': 4272, 'stricken': 4273, 'stability': 4274, 'Analog': 4275, 'Devices': 4276, 'Rite': 4277, 'Aid': 4278, 'son': 4279, 'Primary': 4280, 'recommended': 4281, 'instant': 4282, 'messaging': 4283, 'graduate': 4284, 'chance': 4285, 'Alabama': 4286, 'disputes': 4287, 'Arizona': 4288, 'Diet': 4289, 'Wal': 4290, 'Mart': 4291, 'Austria': 4292, 'Luxembourg': 4293, 'commissions': 4294, 'trending': 4295, 'benefiting': 4296, 'enhanced': 4297, 'integrations': 4298, 'VIP': 4299, 'SUV': 4300, 'Cadillac': 4301, 'Mobility': 4302, 'benefited': 4303, 'enhancing': 4304, 'Zoom': 4305, 'Frank': 4306, 'Airports': 4307, 'draft': 4308, 'Xeon': 4309, 'examples': 4310, 'Village': 4311, 'Tesla': 4312, 'Fashion': 4313, 'guess': 4314, 'frozen': 4315, 'Eric': 4316, 'exactly': 4317, 'fine': 4318, 'Harris': 4319, 'tried': 4320, 'hall': 4321, 'heart': 4322, 'partnerships': 4323, 'Colorado': 4324, 'FanDuel': 4325, 'unmatched': 4326, 'nature': 4327, 'guys': 4328, 'audiences': 4329, 'promote': 4330, 'gradually': 4331, 'perfect': 4332, 'Zulily': 4333, 'traction': 4334, 'Navigator': 4335, 'groceries': 4336, 'households': 4337, 'tire': 4338, 'circa': 4339, 'ARM': 4340, 'architecture': 4341, 'EV': 4342, 'acceptance': 4343, '3.4': 4344, 'automation': 4345, 'Southeast': 4346, 'discussing': 4347, 'establishment': 4348, 'thank': 4349, 'VMware': 4350, 'shot': 4351, 'professionals': 4352, 'procedure': 4353, 'Braintree': 4354, 'medium': 4355, 'code': 4356, 'craft': 4357, 'heavyweight': 4358, 'Love': 4359, 'watched': 4360, 'perspective': 4361, 'voice': 4362, 'mixed': 4363, 'intelligence': 4364, 'Whopper': 4365, 'shopper': 4366, 'loyalty': 4367, 'Ahold': 4368, 'Davis': 4369, 'VA': 4370, 'injection': 4371, 'hopeful': 4372, 'trial': 4373, 'Non': 4374, 'Fifth': 4375, '’d': 4376, 'experienced': 4377, 'mortar': 4378, 'servicing': 4379, 'mass': 4380, 'delay': 4381, 'unparalleled': 4382, 'downtown': 4383, 'cater': 4384, 'Ontario': 4385, 'residents': 4386, 'principal': 4387, 'BBC': 4388, 'universities': 4389, 'Q': 4390, 'pending': 4391, 'engineers': 4392, 'happens': 4393, '55': 4394, 'values': 4395, 'Northeast': 4396, 'crazy': 4397, 'resource': 4398, 'oversupply': 4399, 'watching': 4400, 'lockdown': 4401, 'packaged': 4402, 'challenging': 4403, 'HarperCollins': 4404, 'Q4': 4405, 'Going': 4406, 'affiliated': 4407, 'deploying': 4408, 'evolved': 4409, 'updated': 4410, 'assumptions': 4411, 'folks': 4412, 'matters': 4413, 'pursuant': 4414, 'Pfizer': 4415, 'Sciences': 4416, 'promised': 4417, 'licenses': 4418, 'Dragon': 4419, '65': 4420, 'airplane': 4421, 'zero': 4422, 'graphite': 4423, 'treated': 4424, 'Madison': 4425, 'penetration': 4426, 'dynamics': 4427, 'visibility': 4428, 'Israel': 4429, 'ratios': 4430, 'ranges': 4431, 'puts': 4432, 'vendor': 4433, 'require': 4434, 'slate': 4435, 'farms': 4436, 'outlets': 4437, '2.8': 4438, 'seemed': 4439, 'Factor': 4440, 'felt': 4441, 'headcount': 4442, 'Anthony': 4443, 'distribute': 4444, 'adjust': 4445, 'substantial': 4446, 'preview': 4447, 'reopen': 4448, 'Kors': 4449, 'decreased': 4450, 'disclosure': 4451, 'Let': 4452, 'Ensure': 4453, 'strategically': 4454, 'finally': 4455, 'sixth': 4456, 'server': 4457, 'Of': 4458, 'Box': 4459, 'engage': 4460, 'Java': 4461, 'comprised': 4462, 'exceeded': 4463, 'Fresh': 4464, 'lived': 4465, 'Circle': 4466, 'BOSTON': 4467, 'pilot': 4468, 'distributor': 4469, 'La': 4470, 'brewery': 4471, 'approvals': 4472, 'limits': 4473, 'lifetime': 4474, 'updates': 4475, 'entitled': 4476, 'College': 4477, 'WWE': 4478, 'style': 4479, 'Trent': 4480, 'elements': 4481, 'demonstrating': 4482, 'targeted': 4483, 'certified': 4484, 'reviews': 4485, 'TSYS': 4486, 'fantastic': 4487, 'ASEAN': 4488, '747': 4489, 'reopened': 4490, 'heading': 4491, 'A320neo': 4492, 'dedicated': 4493, 'franchise': 4494, 'EchoPark': 4495, 'merchandise': 4496, 'rapidly': 4497, 'tested': 4498, 'Altice': 4499, 'restart': 4500, 'organic': 4501, 'Gamesa': 4502, 'worst': 4503, 'Pool': 4504, 'Euronext': 4505, 'fertilizer': 4506, 'status': 4507, 'apply': 4508, 'ranking': 4509, 'Carolina': 4510, 'Transavia': 4511, 'negotiation': 4512, 'strikes': 4513, 'turning': 4514, 'deferred': 4515, 'downstream': 4516, 'protein': 4517, 'Sofitel': 4518, 'picks': 4519, 'My': 4520, 'Theresa': 4521, 'wealth': 4522, 'Welcome': 4523, 'Origin': 4524, 'aggressively': 4525, 'Florida': 4526, 'drawn': 4527, 'tractor': 4528, 'etc': 4529, 'challenged': 4530, 'spaces': 4531, '2030': 4532, 'reflected': 4533, 'Mastercard': 4534, 'debit': 4535, 'carbon': 4536, 'Change': 4537, 'Sustainable': 4538, 'Frozen': 4539, 'dealers': 4540, 'Transit': 4541, 'contain': 4542, 'R&D': 4543, 'drove': 4544, 'drilling': 4545, 'discipline': 4546, 'Uranium': 4547, 'Nebraska': 4548, 'participants': 4549, 'Entergy': 4550, 'Arkansas': 4551, 'filings': 4552, 'executed': 4553, 'ratio': 4554, 'Ship': 4555, 'anywhere': 4556, 'pursuing': 4557, 'consolidates': 4558, 'Commissioner': 4559, 'windows': 4560, 'indicates': 4561, 'Expert': 4562, 'north': 4563, 'south': 4564, 'timely': 4565, 'tells': 4566, 'doubled': 4567, 'regard': 4568, 'unveiling': 4569, 'admitted': 4570, 'Pulp': 4571, 'Right': 4572, 'extraordinary': 4573, 'everybody': 4574, 'mainstream': 4575, 'mills': 4576, 'congratulate': 4577, 'import': 4578, 'lighting': 4579, 'programming': 4580, 'finished': 4581, 'consumed': 4582, 'fraud': 4583, 'Demand': 4584, 'PACCAR': 4585, 'Green': 4586, 'Furniture': 4587, 'runs': 4588, 'workloads': 4589, 'contribution': 4590, 'interaction': 4591, 'occurred': 4592, 'historical': 4593, 'learn': 4594, 'explained': 4595, 'quiet': 4596, 'Jordan': 4597, 'please': 4598, 'fulfill': 4599, 'firewall': 4600, 'advised': 4601, 'Freight': 4602, 'Meeting': 4603, 'bar': 4604, 'handle': 4605, 'forces': 4606, 'FX': 4607, 'rocket': 4608, 'convenience': 4609, 'thus': 4610, 'Brand': 4611, 'stated': 4612, 'collectively': 4613, 'hair': 4614, 'robust': 4615, 'suffering': 4616, 'Drive': 4617, 'exercise': 4618, 'TVs': 4619, 'circle': 4620, 'spite': 4621, 'recording': 4622, 'otherwise': 4623, 'laws': 4624, 'else': 4625, 'relating': 4626, 'Advertising': 4627, 'Lloyds': 4628, 'strengthens': 4629, 'nickel': 4630, 'advance': 4631, 'bullish': 4632, '50,000': 4633, 'movies': 4634, 'Television': 4635, 'Redmi': 4636, 'tablets': 4637, 'Macondo': 4638, 'plunging': 4639, 'Z': 4640, 'Icahn': 4641, 'slow': 4642, 'Midnight': 4643, 'PT': 4644, 'woes': 4645, 'died': 4646, 'poised': 4647, 'arriving': 4648, 'pioneered': 4649, 'Its': 4650, 'employed': 4651, 'spread': 4652, 'Kažimír': 4653, 'reluctantly': 4654, 'Deutschland': 4655, 'subsequently': 4656, 'struggle': 4657, 'payout': 4658, 'Billionaire': 4659, 'bills': 4660, 'SpaceX': 4661, 'liftoff': 4662, 'emphasis': 4663, 'shifting': 4664, 'sealed': 4665, 'Science': 4666, 'Fresnillo': 4667, 'exclusively': 4668, 'Leasing': 4669, 'promises': 4670, 'swung': 4671, 'equities': 4672, 'M&S': 4673, 'S&T': 4674, 'usually': 4675, 'membership': 4676, 'Knight': 4677, 'Mason': 4678, 'criticism': 4679, 'minute': 4680, 'save': 4681, 'Whitbread': 4682, 'Chesky': 4683, 'LOS': 4684, 'ANGELES': 4685, 'accused': 4686, 'negotiate': 4687, 'prompting': 4688, 'Safety': 4689, '458': 4690, 'chapter': 4691, 'juices': 4692, 'confectionery': 4693, 'JLR': 4694, 'Mid': 4695, 'attended': 4696, 'Jim': 4697, 'depressed': 4698, 'Hot': 4699, 'Wheels': 4700, 'Albertsons': 4701, 'exporters': 4702, 'meanwhile': 4703, 'ETI': 4704, 'Murray': 4705, 'William': 4706, 'Stephen': 4707, 'Situations': 4708, 'FCSS': 4709, 'IPO.L': 4710, 'Minds': 4711, 'ALM': 4712, 'Sudan': 4713, 'Kumar': 4714, 'Xterra': 4715, 'jewel': 4716, 'tip': 4717, 'Boots': 4718, 'Fremont': 4719, 'Six': 4720, 'credible': 4721, 'thanks': 4722, 'ditch': 4723, 'describing': 4724, '74': 4725, 'Accenture': 4726, 'WMT': 4727, 'breaking': 4728, 'Sir': 4729, 'Exchanges': 4730, 'bridge': 4731, 'Pao': 4732, 'swaps': 4733, 'Semiconductor': 4734, 'Controls': 4735, 'outcome': 4736, 'competitiveness': 4737, 'Budget': 4738, '43': 4739, 'violent': 4740, 'protests': 4741, 'sovereign': 4742, 'awaited': 4743, 'shipping': 4744, 'One97': 4745, 'Ohio': 4746, 'picking': 4747, 'Reynolds': 4748, 'Altera': 4749, 'employment': 4750, 'Ian': 4751, 'preferences': 4752, 'Done': 4753, 'Evans': 4754, 'Mike': 4755, 'negotiating': 4756, 'Founded': 4757, '1.8': 4758, 'richest': 4759, 'Mobil': 4760, 'BNP': 4761, 'Paribas': 4762, 'Full': 4763, 'Sign': 4764, 'WSJ': 4765, 'browser': 4766, '!': 4767, 'Marissa': 4768, 'Mayer': 4769, 'entities': 4770, 'SL': 4771, 'Young': 4772, 'Curio': 4773, 'Economics': 4774, 'restore': 4775, 'considered': 4776, 'depth': 4777, 'Rival': 4778, 'broadcasting': 4779, 'Shine': 4780, 'Imperial': 4781, 'Fluor': 4782, 'REUTERS': 4783, 'DE': 4784, 'hailing': 4785, 'Elsewhere': 4786, 'Cathay': 4787, 'PFL.L': 4788, 'express': 4789, 'unions': 4790, 'guidelines': 4791, 'analytical': 4792, 'auctions': 4793, '4,000': 4794, 'sit': 4795, 'DC.L': 4796, 'Warehouse': 4797, 'PCs': 4798, 'Taste': 4799, 'Touring': 4800, 'Is': 4801, 'J': 4802, 'Spencer': 4803, 'Kingdom': 4804, 'Copper': 4805, 'repeated': 4806, 'Charles': 4807, 'lawmaker': 4808, 'Fair': 4809, 'JD': 4810, 'Hyperion': 4811, 'Ruth': 4812, 'scenario': 4813, 'headcounts': 4814, 'yen': 4815, 'CHICAGO': 4816, 'booking': 4817, '6,000': 4818, 'Statoil': 4819, 'earned': 4820, 'Labor': 4821, 'Gina': 4822, 'Rinehart': 4823, 'APA': 4824, 'signalled': 4825, 'OZ': 4826, 'household': 4827, '+1.30': 4828, 'BT': 4829, 'Matthey': 4830, 'easyJet': 4831, 'Ashtead': 4832, 'SBRY': 4833, 'L': 4834, 'elite': 4835, 'terror': 4836, 'Mail': 4837, 'FINAL': 4838, 'TEP.L': 4839, 'deciding': 4840, 'essentially': 4841, 'farm': 4842, 'solids': 4843, 'Ukrainian': 4844, 'Uganda': 4845, 'Apparel': 4846, 'Great': 4847, 'Select': 4848, 'Tema': 4849, '6.5': 4850, 'troubled': 4851, 'hoped': 4852, 'marking': 4853, 'Wales': 4854, 'container': 4855, 'Rubber': 4856, 'Greencross': 4857, 'CSR': 4858, 'Aurelia': 4859, 'Orica': 4860, 'Hardie': 4861, 'Recall': 4862, 'Boral': 4863, 'CBA': 4864, 'Goodman': 4865, 'QBE': 4866, 'Adelaide': 4867, 'Brighton': 4868, 'DUET': 4869, 'AGL': 4870, 'Ramsay': 4871, 'File': 4872, 'aisle': 4873, 'Tax': 4874, '1994': 4875, 'complaints': 4876, 'Carlyle': 4877, 'AWD': 4878, 'block': 4879, 'Meggitt': 4880, '+0.64': 4881, 'chicken': 4882, 'MoneyGram': 4883, 'competing': 4884, 'equal': 4885, 'Core': 4886, 'Paolo': 4887, 'Cinven': 4888, 'Fortescue': 4889, 'franchising': 4890, 'Gardens': 4891, 'Coldwell': 4892, 'EDT': 4893, 'age': 4894, 'Southwest': 4895, 'Plan': 4896, 'Ashley': 4897, 'publisher': 4898, 'Quote': 4899, 'sterling': 4900, 'Printing': 4901, 'produces': 4902, 'Reg': 4903, 'Persimmon': 4904, 'PSN': 4905, 'Pearson': 4906, 'Shire': 4907, 'BLT': 4908, 'PAY': 4909, '+0.79': 4910, 'NCC.L': 4911, 'Spirent': 4912, 'SPMYY': 4913, 'SPT': 4914, '8431.TWO': 4915, 'Follow': 4916, 'Society': 4917, 'Action': 4918, 'Affairs': 4919, 'InterContinental': 4920, 'Hotels': 4921, 'Color': 4922, 'pence': 4923, 'Eye': 4924, 'Dusseldorf': 4925, 'platinum': 4926, 'mini': 4927, 'deterioration': 4928, 'd2h': 4929, 'bouquet': 4930, 'English': 4931, 'sentiment': 4932, 'IKEA': 4933, 'hurdle': 4934, 'Ledger': 4935, 'custom': 4936, 'Assist': 4937, 'Earn': 4938, 'journalists': 4939, 'A0F5DE': 4940, 'clean': 4941, 'Bain': 4942, 'lows': 4943, 'ExxonMobil': 4944, 'holders': 4945, '8.9': 4946, 'Skyharbour': 4947, 'extensive': 4948, 'Mines': 4949, 'settle': 4950, 'Ikea': 4951, 'Buenos': 4952, 'Aires': 4953, 'modern': 4954, 'retreat': 4955, 'Subway': 4956, 'Jared': 4957, 'Fogle': 4958, 'AB': 4959, 'Scotland': 4960, 'Sumitomo': 4961, 'Stifel': 4962, 'download': 4963, 'c': 4964, 'centres': 4965, 'Silicon': 4966, 'useful': 4967, 'buildings': 4968, 'tank': 4969, 'Axel': 4970, 'aluminum': 4971, 'turbine': 4972, 'completion': 4973, 's': 4974, 'OMERS': 4975, 'losing': 4976, 'SGE.L': 4977, '+0.67': 4978, 'believes': 4979, 'deputy': 4980, 'antitrust': 4981, 'Stan': 4982, 'Osaka': 4983, 'insurgency': 4984, 'Envision': 4985, 'Jeep': 4986, 'Cherokee': 4987, 'Beer': 4988, 'exploit': 4989, 'TCG': 4990, 'laundering': 4991, 'Jan': 4992, 'dip': 4993, 'Wood': 4994, 'Moore': 4995, 'Chemicals': 4996, 'rig': 4997, 'inner': 4998, 'selection': 4999, 'Branson': 5000, 'Qatar': 5001, 'briefing': 5002, 'Tie': 5003, 'Champion': 5004, 'intense': 5005, 'fired': 5006, 'Justin': 5007, 'Cable': 5008, 'francs': 5009, 'duties': 5010, 'headwinds': 5011, '2004': 5012, '2.3': 5013, 'beat': 5014, 'preserve': 5015, 'engines': 5016, 'KR': 5017, 'Publishing': 5018, 'st': 5019, 'crossover': 5020, 'northern': 5021, '6.1': 5022, 'Xoom': 5023, 'remittance': 5024, 'predicts': 5025, 'slipped': 5026, 'reputation': 5027, 'constantly': 5028, 'fire': 5029, 'PTT': 5030, 'Cameron': 5031, 'iiNet': 5032, 'Million': 5033, 'Evolution': 5034, 'Claude': 5035, 'Officials': 5036, 'repurchase': 5037, 'council': 5038, 'Naftogaz': 5039, 'IMF': 5040, 'debts': 5041, 'Macartney': 5042, 'teleconference': 5043, 'unclear': 5044, 'Draghi': 5045, 'Farm': 5046, 'sex': 5047, 'paves': 5048, 'VP': 5049, 'stressed': 5050, 'triggered': 5051, 'Oct.': 5052, 'reliant': 5053, 'playing': 5054, 'Heinz': 5055, 'Kraft': 5056, 'PG': 5057, 'Electronic': 5058, 'Jio': 5059, 'Dennis': 5060, 'headlines': 5061, 'boom': 5062, 'Citi': 5063, 'Plaza': 5064, 'ranging': 5065, 'minority': 5066, 'A380': 5067, 'specialist': 5068, 'Diamonds': 5069, 'goal': 5070, 'Bruce': 5071, 'AOL': 5072, 'worker': 5073, 'Clarkson': 5074, 'ban': 5075, 'Payment': 5076, 'floating': 5077, 'salary': 5078, 'approached': 5079, 'trader': 5080, 'Infosys': 5081, 'Leone': 5082, 'PNB': 5083, 'GS': 5084, 'Ryanair': 5085, 'evaporated': 5086, 'Elizabeth': 5087, 'BrightHouse': 5088, 'Dassault': 5089, 'depend': 5090, 'Heavy': 5091, 'Assembly': 5092, 'UBM': 5093, 'SDL.L': 5094, 'KCOM.L': 5095, 'IShares': 5096, 'Gov': 5097, '10YR': 5098, 'UCITS': 5099, 'IEGM': 5100, 'RM': 5101, 'Amtrak': 5102, 'entrepreneurs': 5103, 'NGC': 5104, '5.1': 5105, 'Castparts': 5106, 'Adani': 5107, '600': 5108, 'beef': 5109, 'Assurance': 5110, 'condom': 5111, 'resurfaced': 5112, 'Dean': 5113, 'AfDB': 5114, 'failing': 5115, 'Leadership': 5116, 'Daily': 5117, 'Equipment': 5118, 'Pichai': 5119, 'Thompson': 5120, 'Dealing': 5121, 'Prepare': 5122, 'Vimpelcom': 5123, 'Hutchison': 5124, 'feed': 5125, 'proactive': 5126, 'HPCL': 5127, 'BPCL': 5128, 'IOC': 5129, 'slip': 5130, 'Borshoff': 5131, 'Offers': 5132, 'Mashreq': 5133, 'genius': 5134, 'Bhd': 5135, 'McMoRan': 5136, 'MTN': 5137, 'enjoy': 5138, 'Shantilal': 5139, 'Pharma': 5140, 'Vijay': 5141, 'Shekhar': 5142, 'Torotrak': 5143, 'Supply': 5144, 'OS': 5145, 'Edge': 5146, 'Nuvo': 5147, 'Cholamandalam': 5148, 'Consort': 5149, 'CSRMF': 5150, 'diamond': 5151, 'drones': 5152, 'historic': 5153, 'cycle': 5154, 'telcos': 5155, 'successor': 5156, 'KFC': 5157, 'SAB': 5158, 'Surface': 5159, '106': 5160, 'EAFE': 5161, 'ACWI': 5162, 'Vale': 5163, 'lowered': 5164, 'cited': 5165, 'Chocolate': 5166, 'Jackson': 5167, 'GOOG': 5168, '3.47': 5169, 'WSO2': 5170, 'BMO': 5171, 'Ahead': 5172, 'Tropics': 5173, 'curable': 5174, 'stockholders': 5175, '6P': 5176, 'Chromecast': 5177, 'ReachLocal': 5178, 'LEH.N': 5179, 'execution': 5180, 'buyout': 5181, 'Bear': 5182, 'Stearns': 5183, 'authority': 5184, 'publish': 5185, 'succeed': 5186, 'beaten': 5187, 'Apps': 5188, 'Gmail': 5189, 'word': 5190, 'refused': 5191, 'send': 5192, 'deepening': 5193, 'console': 5194, 'Staples': 5195, 'disappointing': 5196, 'Cos': 5197, 'Stores': 5198, 'Buckle': 5199, 'adopted': 5200, 'publishers': 5201, 'grocers': 5202, 'fundamentally': 5203, 'Belgium': 5204, 'foods': 5205, 'sticking': 5206, 'Symantec': 5207, 'Yukon': 5208, 'Aimia': 5209, 'Together': 5210, 'Back': 5211, 'Ocean': 5212, 'Wind': 5213, 'Interactive': 5214, 'Or': 5215, 'SaaS': 5216, 'Scalable': 5217, 'Klabin': 5218, 'contributed': 5219, 'System': 5220, 'ramp': 5221, 'Yes': 5222, 'FDA': 5223, 'Restoration': 5224, 'extent': 5225, 'Publix': 5226, 'whom': 5227, 'notion': 5228, 'town': 5229, 'Speaking': 5230, 'serves': 5231, 'tend': 5232, 'Outlook': 5233, 'coupled': 5234, 'proving': 5235, 'deeper': 5236, 'cultivation': 5237, 'Kugou': 5238, 'evolves': 5239, 'encouraged': 5240, 'fundamental': 5241, 'occupied': 5242, 'referred': 5243, 'database': 5244, 'Moreover': 5245, 'insulin': 5246, 'peptide': 5247, 'feet': 5248, 'charging': 5249, 'Broadband': 5250, 'heard': 5251, 'ticker': 5252, '330': 5253, '225': 5254, 'Closing': 5255, 'financials': 5256, '150,000': 5257, 'entirely': 5258, 'Dunlop': 5259, 'behalf': 5260, 'P2P': 5261, 'QR': 5262, 'register': 5263, 'neck': 5264, 'button': 5265, 'screen': 5266, 'seamlessly': 5267, 'mill': 5268, 'idle': 5269, 'renewed': 5270, 'powers': 5271, 'Cognos': 5272, 'swoosh': 5273, 'Labs': 5274, 'Employees': 5275, 'Event': 5276, 'Insights': 5277, 'highlighting': 5278, 'Downtown': 5279, 'publication': 5280, 'accommodate': 5281, 'referenced': 5282, 'picked': 5283, 'deli': 5284, 'bus': 5285, 'Revlon': 5286, 'agnostic': 5287, 'proud': 5288, 'desire': 5289, '350': 5290, 'drugs': 5291, 'assistance': 5292, 'meals': 5293, 'KT': 5294, 'dates': 5295, 'stories': 5296, 'adjustment': 5297, 'opting': 5298, 'regulation': 5299, 'fit': 5300, 'SurveyMonkey': 5301, 'modernize': 5302, 'transmission': 5303, 'ph': 5304, 'metro': 5305, 'suburban': 5306, 'Pennsylvania': 5307, 'happening': 5308, 'pharmacies': 5309, 'DePuy': 5310, 'Synthes': 5311, 'allocation': 5312, 'spring': 5313, '88': 5314, 'curious': 5315, 'aggressive': 5316, 'hitting': 5317, 'enforcement': 5318, 'apace': 5319, 'album': 5320, 'artist': 5321, 'cautious': 5322, 'incredible': 5323, 'B2B': 5324, 'generating': 5325, 'mark': 5326, 'doubt': 5327, 'whereas': 5328, 'walk': 5329, 'grab': 5330, 'sometimes': 5331, 'highlight': 5332, 'underperforming': 5333, 'rule': 5334, 'metrics': 5335, 'holiday': 5336, 'fairly': 5337, 'prudent': 5338, 'retention': 5339, 'targeting': 5340, 'Mylan': 5341, 'Upjohn': 5342, 'reverse': 5343, 'Viatris': 5344, 'virtually': 5345, 'Good': 5346, 'favor': 5347, 'spectrum': 5348, 'seat': 5349, 'Varian': 5350, 'minus': 5351, 'submit': 5352, 'scientific': 5353, 'multiyear': 5354, 'theatrical': 5355, 'films': 5356, 'Expansion': 5357, 'avenue': 5358, 'swap': 5359, 'basically': 5360, 'economically': 5361, 'reasonable': 5362, 'narrow': 5363, 'streamline': 5364, 'simplify': 5365, 'Master': 5366, 'transitioning': 5367, 'box': 5368, 'essential': 5369, 'contributions': 5370, 'mitigate': 5371, 'foundation': 5372, 'waste': 5373, 'Peru': 5374, 'thirds': 5375, 'hear': 5376, 'evaluating': 5377, 'tenant': 5378, 'RV': 5379, 'inclusion': 5380, 'Peacock': 5381, 'Go': 5382, 'Stora': 5383, 'Enso': 5384, 'intended': 5385, 'blast': 5386, 'possibly': 5387, 'globe': 5388, 'Esurance': 5389, 'Allstate': 5390, 'somewhat': 5391, 'spacecraft': 5392, 'Grumman': 5393, 'formally': 5394, 'delayed': 5395, 'DTC': 5396, 'Hulu': 5397, 'animation': 5398, 'Double': 5399, 'Rockstar': 5400, 'refer': 5401, 'friend': 5402, 'Fi': 5403, 'Huawei': 5404, '767': 5405, 'Still': 5406, 'Raymond': 5407, 'annuity': 5408, 'bah': 5409, 'ingredients': 5410, 'Furthermore': 5411, 'Tabrecta': 5412, 'royalties': 5413, 'subsequent': 5414, 'Novartis': 5415, 'Dead': 5416, 'featuring': 5417, 'Role': 5418, 'discover': 5419, 'standout': 5420, 'Overall': 5421, '25th': 5422, 'Mafia': 5423, 'renowned': 5424, 'Laser': 5425, 'cabin': 5426, 'Midwest': 5427, 'saving': 5428, 'manual': 5429, 'Issuer': 5430, 'capability': 5431, 'retirement': 5432, 'Barcelona': 5433, 'guests': 5434, 'utilizing': 5435, 'Police': 5436, 'Americas': 5437, 'Scientific': 5438, 'Téléfoot': 5439, 'Sport': 5440, 'Mozambique': 5441, 'backlog': 5442, 'Vivendi': 5443, 'Editis': 5444, 'aligned': 5445, 'evidence': 5446, 'optimize': 5447, 'dynamic': 5448, 'dealing': 5449, '1,300': 5450, 'postponed': 5451, 'colleagues': 5452, 'Jaguar': 5453, '71': 5454, 'Bombardier': 5455, 'manufactured': 5456, '72': 5457, 'implemented': 5458, 'lab': 5459, 'Institutional': 5460, 'macroeconomic': 5461, 'graphic': 5462, 'schedules': 5463, 'briefly': 5464, 'touch': 5465, '1.7': 5466, 'Interxion': 5467, 'bundles': 5468, 'favorable': 5469, 'Tier': 5470, 'attracted': 5471, 'incremental': 5472, 'offsetting': 5473, '53': 5474, 'compute': 5475, 'prestigious': 5476, 'resilient': 5477, 'depending': 5478, 'population': 5479, 'digits': 5480, 'dining': 5481, 'Telefónica': 5482, 'maintaining': 5483, 'bundle': 5484, 'Maestro': 5485, 'renewables': 5486, 'Climate': 5487, '5.3': 5488, 'ice': 5489, 'Corridor': 5490, 'respond': 5491, 'configuration': 5492, 'expectation': 5493, 'strongest': 5494, 'instance': 5495, 'profiles': 5496, 'wallets': 5497, 'rewards': 5498, 'detail': 5499, 'Matt': 5500, 'pointed': 5501, 'Denver': 5502, 'undertaken': 5503, 'interruptions': 5504, 'Louisiana': 5505, 'formula': 5506, 'technological': 5507, 'breaches': 5508, 'defects': 5509, 'replacement': 5510, 'LIBOR': 5511, 'loading': 5512, 'radar': 5513, 'Reform': 5514, 'careers': 5515, 'Howard': 5516, 'Sirius': 5517, 'Wizz': 5518, 'tourists': 5519, 'Third': 5520, 'papers': 5521, 'Pay': 5522, 'distancing': 5523, 'Hyatt': 5524, 'Chesapeake': 5525, 'Bay': 5526, 'adapt': 5527, 'roadmap': 5528, 'seller': 5529, 'sellers': 5530, 'vaccines': 5531, 'diluted': 5532, 'EPS': 5533, 'CapEx': 5534, 'shifts': 5535, 'satisfy': 5536, 'adverse': 5537, 'dramatic': 5538, 'doubling': 5539, 'Because': 5540, 'deployed': 5541, 'brokered': 5542, 'accelerated': 5543, 'Analyst': 5544, 'wave': 5545, 'encourages': 5546, 'Thanks': 5547, 'uncertain': 5548, 'doors': 5549, 'Manhattan': 5550, 'Certification': 5551, 'Leading': 5552, 'adopting': 5553, 'ensuring': 5554, 'Berkeley': 5555, 'A319': 5556, 'A320s': 5557, 'powder': 5558, 'steelmaking': 5559, 'Chapter': 5560, 'AutoNation': 5561, 'corresponding': 5562, 'Ted': 5563, 'burn': 5564, 'gear': 5565, 'retired': 5566, 'bottles': 5567, 'WarnerMedia': 5568, 'Citrix': 5569, 'adopt': 5570, 'ignored': 5571, 'Protection': 5572, 'E5': 5573, 'manufactures': 5574, 'definitely': 5575, 'Kenworth': 5576, 'Deal': 5577, 'DAF': 5578, 'Initiative': 5579, 'Exit': 5580, 'intensity': 5581, 'Sabadell': 5582, 'regards': 5583, 'sourcing': 5584, 'looked': 5585, 'indoor': 5586, 'ETFs': 5587, 'Swedish': 5588, 'distributors': 5589, 'Mikhail': 5590, 'reaction': 5591, 'breweries': 5592, 'watches': 5593, 'laid': 5594, 'reorganization': 5595, 'vegetables': 5596, '1,000': 5597, 'bull': 5598, 'signature': 5599, 'routing': 5600, 'tips': 5601, 'Ongoing': 5602, 'column': 5603, 'chart': 5604, 'Indofood': 5605, 'noodles': 5606, 'noodle': 5607, 'comfortably': 5608, 'portfolios': 5609, 'principally': 5610, 'relation': 5611, 'CAD': 5612, 'Aerospace': 5613, 'GoPro': 5614, 'Free': 5615, 'definition': 5616, 'encourage': 5617, 'lay': 5618, 'DHL': 5619, 'learned': 5620, 'pork': 5621, 'deliverables': 5622, 'Difficulty': 5623, 'agriculture': 5624, 'deepen': 5625, 'Meituan': 5626, 'upgrading': 5627, '446': 5628, 'actively': 5629, 'H': 5630, 'soared': 5631, 'Jo': 5632, 'Dollar': 5633, '56': 5634, 'follows': 5635, 'consent': 5636, 'violation': 5637, 'copyright': 5638, 'dispose': 5639, 'families': 5640, 'assess': 5641, 'identity': 5642, 'wall': 5643, 'kinds': 5644, 'Adobe': 5645, 'Journey': 5646, 'enterprises': 5647, 'Ads': 5648, 'outperforming': 5649, 'removed': 5650, 'SQL': 5651, 'vertical': 5652, 'ambitious': 5653, 'appropriate': 5654, 'rail': 5655, 'sections': 5656, 'formation': 5657, 'display': 5658, 'iTunes': 5659, 'chemical': 5660, 'sensors': 5661, 'Pixel': 5662, 'Computer': 5663, 'implementing': 5664, '250,000': 5665, 'Mitsubishi': 5666, 'petroleum': 5667, 'dairies': 5668, '7.3': 5669, 'royalty': 5670, 'calculated': 5671, 'Golar': 5672, 'turbines': 5673, 'charter': 5674, 'lowering': 5675, 'vibrant': 5676, 'Cincinnati': 5677, 'agents': 5678, 'digitally': 5679, 'sneak': 5680, 'Satoru': 5681, 'U': 5682, 'argues': 5683, 'count': 5684, 'cyber': 5685, 'strides': 5686, 'scheme': 5687, 'Numericable': 5688, 'Matador': 5689, 'Shareholders': 5690, 'migrants': 5691, 'ABF.L': 5692, 'Bupa': 5693, 'ousted': 5694, 'Oilfield': 5695, 'Hughes': 5696, 'photograph': 5697, 'novel': 5698, 'Silver': 5699, 'Newmont': 5700, 'kick': 5701, 'Parliament': 5702, 'Summit': 5703, 'SPD': 5704, 'Having': 5705, 'pro': 5706, 'upside': 5707, 'Societe': 5708, 'Generale': 5709, 'FR0000130809': 5710, '898.3': 5711, 'Breaking': 5712, 'Recruiting': 5713, 'Works': 5714, 'SCA': 5715, 'Publishers': 5716, 'GmbH': 5717, 'FBI': 5718, 'hack': 5719, '\\r': 5720, 'Iger': 5721, 'Roberts': 5722, 'CBRE': 5723, 'willing': 5724, 'Norwegian': 5725, 'Hydro': 5726, 'Switzerland': 5727, 'MW': 5728, 'tourist': 5729, 'Fast': 5730, 'Uniqlo': 5731, 'Traffic': 5732, 'Ferrari': 5733, 'Sat': 5734, 'mineral': 5735, 'jam': 5736, 'honey': 5737, 'FCA': 5738, 'Rosevear': 5739, 'TMFMarlowe': 5740, 'JP': 5741, '620': 5742, '349': 5743, 'Autoplay': 5744, 'Off': 5745, 'settings': 5746, 'faces': 5747, 'Atlassian': 5748, 'mandate': 5749, 'Teavana': 5750, 'humanitarian': 5751, 'citizens': 5752, 'shock': 5753, 'helps': 5754, 'Members': 5755, 'Diageo': 5756, 'jewelry': 5757, 'ringing': 5758, 'Finnish': 5759, '8.8': 5760, 'Little': 5761, 'Field': 5762, 'footsteps': 5763, 'Buys': 5764, 'Strategies': 5765, '5,000': 5766, 'Bailey': 5767, 'Caledonia': 5768, 'CLDN': 5769, '2,429.00p': 5770, '38': 5771, 'Drax': 5772, 'DRX': 5773, 'Debenhams': 5774, '535.00p': 5775, 'BBY': 5776, 'ABGLF': 5777, 'Citizen': 5778, 'Lab': 5779, 'widespread': 5780, 'words': 5781, 'anything': 5782, 'visitors': 5783, 'roped': 5784, 'advisor': 5785, 'memo': 5786, 'benchmark': 5787, 'solidify': 5788, 'EMEA': 5789, 'formal': 5790, 'IRS': 5791, '7.5': 5792, 'sudden': 5793, 'Funding': 5794, 'accounted': 5795, 'fix': 5796, 'Brewing': 5797, 'Competition': 5798, 'homeowners': 5799, 'port': 5800, 'east': 5801, 'Audio': 5802, 'turmoil': 5803, 'repair': 5804, 'Heathrow': 5805, 'west': 5806, 'eco': 5807, 'demanded': 5808, 'Caribbean': 5809, 'BNS': 5810, 'spinning': 5811, 'Cisco': 5812, 'A1W6XZ': 5813, '1995': 5814, 'bosses': 5815, 'FB': 5816, 'Clarke': 5817, 'Natera': 5818, 'methods': 5819, 'Bosch': 5820, 'Delphi': 5821, 'Freescale': 5822, 'TTM': 5823, 'crackdown': 5824, 'taxpayer': 5825, 'listening': 5826, 'lawsuit': 5827, 'nations': 5828, 'Word': 5829, 'premiums': 5830, 'afford': 5831, 'explored': 5832, 'outcomes': 5833, 'Merrill': 5834, 'parents': 5835, 'MAT': 5836, 'discounts': 5837, 'websites': 5838, 'Sutton': 5839, 'probe': 5840, 'FIFA': 5841, 'SHARES': 5842, 'initially': 5843, 'succeeded': 5844, 'gay': 5845, 'gender': 5846, 'asks': 5847, 'Highveld': 5848, 'Zuma': 5849, 'borrowing': 5850, 'repayment': 5851, 'consequences': 5852, '028260.KS': 5853, 'Penfolds': 5854, 'normally': 5855, 'Wine': 5856, 'Lane': 5857, 'Advisors': 5858, 'Neighborhood': 5859, 'listings': 5860, 'Lang': 5861, 'mother': 5862, 'builder': 5863, 'Chevron': 5864, 'Murphy': 5865, 'ConocoPhillips': 5866, 'GT': 5867, 'Anne': 5868, 'Coast': 5869, 'Colombia': 5870, 'List': 5871, '0.1': 5872, 'Va.': 5873, 'End': 5874, 'Amid': 5875, '07:11': 5876, 'Immelt': 5877, 'Jeffrey': 5878, 'Beverage': 5879, 'Reports': 5880, '6.6': 5881, 'capitalization': 5882, 'Afren': 5883, 'fields': 5884, 'CFPB': 5885, 'Reaching': 5886, 'Twentieth': 5887, 'Film': 5888, 'Endemol': 5889, 'Integral': 5890, 'Logista': 5891, 'Aviva': 5892, '8.3': 5893, 'convention': 5894, 'mood': 5895, 'rock': 5896, 'steep': 5897, 'arrangements': 5898, 'Pemex': 5899, 'longstanding': 5900, '-2.79': 5901, 'tightening': 5902, 'pledged': 5903, 'organisation': 5904, 'Alaska': 5905, 'landing': 5906, 'seemingly': 5907, 'benign': 5908, 'DI6.BE': 5909, 'TPC': 5910, 'agencies': 5911, '8.0': 5912, 'participates': 5913, 'forecasting': 5914, 'perform': 5915, 'Skills': 5916, 'Education': 5917, 'pursue': 5918, 'committees': 5919, 'bear': 5920, '07/13': 5921, 'FINANCE': 5922, 'withdraw': 5923, 'withdrawal': 5924, 'promoting': 5925, 'Marks': 5926, 'bet': 5927, 'scrap': 5928, 'booming': 5929, 'breaks': 5930, 'tour': 5931, 'Railways': 5932, 'N.Y.': 5933, 'Barclaycard': 5934, 'BW)(NV': 5935, 'RIMINI': 5936, 'STREET': 5937, 'Largest': 5938, 'BusinessObjects': 5939, 'ORCL': 5940, 'Siebel': 5941, 'Edwards': 5942, 'Database': 5943, 'Jerry': 5944, 'Nobel': 5945, 'Nippon': 5946, 'MSFT.O': 5947, 'Archer': 5948, 'Calif.': 5949, 'Cup': 5950, 'Teachers': 5951, 'Open': 5952, 'Micron': 5953, 'span': 5954, 'pop': 5955, 'Swift': 5956, '1989': 5957, 'drastically': 5958, 'Lions': 5959, 'scene': 5960, 'Austrian': 5961, 'Phoenix': 5962, 'Lines': 5963, 'maintained': 5964, '110,000': 5965, 'Parker': 5966, '+1.85': 5967, '+1.80': 5968, 'Inmarsat': 5969, '+1.28': 5970, 'STJ': 5971, '-2.10': 5972, 'Randgold': 5973, 'Merlin': 5974, 'AHT.L': 5975, 'AHT': 5976, 'stars': 5977, 'cuisine': 5978, '4.8': 5979, 'Assets': 5980, 'Record': 5981, 'Kingfisher': 5982, 'Mortgage': 5983, 'Lending': 5984, 'Planet': 5985, 'Sons': 5986, 'paint': 5987, '119': 5988, 'attributed': 5989, 'debated': 5990, 'JOHANNESBURG': 5991, 'Saharan': 5992, 'agreeing': 5993, 'soaring': 5994, 'Rwanda': 5995, 'Republicans': 5996, 'Ex': 5997, 'Im': 5998, 'Tsinghua': 5999, 'PartyGaming': 6000, 'maiden': 6001, 'band': 6002, 'accepting': 6003, 'photos': 6004, 'HCMLF': 6005, 'pool': 6006, 'bail': 6007, 'ANFGF': 6008, 'looms': 6009, 'shortly': 6010, 'pave': 6011, 'heirs': 6012, '46': 6013, 'Areva': 6014, 'gift': 6015, 'voluntary': 6016, 'IDFC': 6017, 'ZTE': 6018, 'anxiety': 6019, 'downside': 6020, 'sluggish': 6021, 'SEEK': 6022, 'majors': 6023, 'Asciano': 6024, 'GPT': 6025, 'Fletcher': 6026, 'Areas': 6027, 'Cover': 6028, 'Federation': 6029, 'Iluka': 6030, 'Newcrest': 6031, 'WorleyParsons': 6032, 'Dulux': 6033, 'thought': 6034, 'pensions': 6035, 'childcare': 6036, 'liquor': 6037, 'custody': 6038, 'Biocon': 6039, 'monetisation': 6040, 'Kiran': 6041, '5.5': 6042, 'convince': 6043, 'suffer': 6044, 'revamped': 6045, 'noting': 6046, 'CIT': 6047, 'OneWest': 6048, 'fail': 6049, 'voted': 6050, 'Ideas': 6051, 'district': 6052, 'featured': 6053, 'governance': 6054, 'spokesperson': 6055, 'SKY.L': 6056, 'SKY': 6057, 'Reed': 6058, 'REL': 6059, '+0.76': 6060, 'TSCO': 6061, 'TW': 6062, 'trap': 6063, 'teaming': 6064, 'stays': 6065, 'Voice': 6066, 'Walsh': 6067, 'hikes': 6068, 'BERK34.SA': 6069, 'CVC': 6070, 'theatre': 6071, 'Stage': 6072, 'SOCAR': 6073, 'Azerbaijan': 6074, 'DESFA': 6075, 'DEPA': 6076, 'Oct': 6077, 'followers': 6078, 'convinced': 6079, 'consultant': 6080, 'FOXA': 6081, 'RLGY': 6082, 'involving': 6083, 'dividends': 6084, 'Judge': 6085, 'meets': 6086, 'gap': 6087, 'optimism': 6088, 'disappointed': 6089, 'ADM.L': 6090, 'appoints': 6091, 'specializing': 6092, 'prompted': 6093, 'residence': 6094, 'interior': 6095, 'SunEdison': 6096, 'mounted': 6097, 'Dawn': 6098, 'Twenty': 6099, 'fallout': 6100, 'weakened': 6101, 'DTS': 6102, 'inched': 6103, 'Movers': 6104, 'RDSA': 6105, 'smile': 6106, 'S7E.DE': 6107, 'SHP': 6108, 'ANTO.L': 6109, 'Materials': 6110, '+2.26': 6111, '-2.08': 6112, 'CEY': 6113, 'INVGF': 6114, 'involvement': 6115, 'slashed': 6116, 'destination': 6117, 'Coke': 6118, 'Pimco': 6119, 'Allianz': 6120, 'Dreamliner': 6121, 'walks': 6122, 'Founder': 6123, 'ICT': 6124, 'Me': 6125, 'IHG': 6126, 'Stella': 6127, 'Madigan': 6128, 'carmaker': 6129, 'CNH': 6130, 'TOC.DU': 6131, 'downgrade': 6132, 'opinion': 6133, 'evolving': 6134, 'SEF': 6135, 'LCH.Clearnet': 6136, '2001': 6137, 'Mercury': 6138, 'LS': 6139, 'behemoth': 6140, 'tougher': 6141, 'importing': 6142, 'NDAQ': 6143, 'taxation': 6144, 'commissioning': 6145, 'ABC': 6146, 'weighed': 6147, 'Job': 6148, 'Include': 6149, 'Financials': 6150, 'Sector': 6151, 'Monitor': 6152, 'Work': 6153, 'Requirements': 6154, 'revival': 6155, 'RIA': 6156, 'Novosti': 6157, 'shocked': 6158, 'Denis': 6159, 'inquiry': 6160, 'midnight': 6161, 'A$': 6162, 'MYOB': 6163, 'returned': 6164, 'hosts': 6165, 'SBUX': 6166, 'Globe': 6167, 'gather': 6168, 'Euromoney': 6169, 'GLOBE': 6170, 'NEWSWIRE': 6171, 'Saskatchewan': 6172, 'Joint': 6173, 'dozen': 6174, 'recalls': 6175, 'exposed': 6176, 'DDAIY': 6177, 'supporter': 6178, 'KBW': 6179, 'Agricultural': 6180, 'Mitsui': 6181, 'Rail': 6182, 'receives': 6183, 'railways': 6184, 'retreated': 6185, '3.6': 6186, 'Marc': 6187, '8.1': 6188, 'homewares': 6189, 'Secret': 6190, 'factories': 6191, 'DIS': 6192, 'flagged': 6193, 'newest': 6194, 'differ': 6195, 'Metal': 6196, 'alumina': 6197, 'unfavorable': 6198, 'raw': 6199, 'Arabia': 6200, 'i': 6201, 'commercialization': 6202, 'FILE': 6203, 'Headquarters': 6204, 'ACI': 6205, '+2.11': 6206, 'Consolidated': 6207, 'SGE': 6208, 'break': 6209, 'sheets': 6210, 'technique': 6211, 'faulty': 6212, 'Town': 6213, 'slowing': 6214, 'Govt': 6215, 'Sinha': 6216, 'bidding': 6217, 'highs': 6218, 'CommonBond': 6219, 'interface': 6220, 'triple': 6221, 'bloc': 6222, '-2.86': 6223, 'Witness': 6224, 'lend': 6225, 'Kalam': 6226, 'wiring': 6227, 'Based': 6228, 'employer': 6229, 'hires': 6230, 'minimise': 6231, 'drug': 6232, 'Regeneron': 6233, 'KKR': 6234, 'Random': 6235, 'Challenge': 6236, 'Marathon': 6237, 'insight': 6238, 'Discovery': 6239, 'Castleton': 6240, 'Henderson': 6241, '125.24p': 6242, 'announcements': 6243, 'economists': 6244, 'Brokers': 6245, 'ZARE': 6246, 'Wipro': 6247, 'Micro': 6248, '2.9': 6249, 'Broadcom': 6250, 'Oyj': 6251, 'sets': 6252, 'conducted': 6253, 'tomorrow': 6254, 'LVMH': 6255, 'Broadcasting': 6256, 'similarly': 6257, 'poorly': 6258, 'Monash': 6259, 'penalty': 6260, 'inspiration': 6261, 'Community': 6262, 'Effective': 6263, 'Inclusion': 6264, '4.3': 6265, 'Class': 6266, 'headed': 6267, 'Chuquicamata': 6268, 'automobile': 6269, 'Bitcoin': 6270, 'Indoor': 6271, 'fitness': 6272, 'Fitness': 6273, 'DDD': 6274, 'branding': 6275, 'dominance': 6276, 'widen': 6277, 'Poor': 6278, 'Arabtec': 6279, 'DUBAI': 6280, 'bourse': 6281, 'rout': 6282, 'Masters': 6283, 'umbrella': 6284, 'Launches': 6285, '67': 6286, 'Continuum': 6287, 'PowerPoint': 6288, 'orbit': 6289, 'noise': 6290, 'PFL': 6291, 'Squawk': 6292, '549': 6293, 'chemicals': 6294, 'Restaurant': 6295, 'Leaders': 6296, 'consultancy': 6297, 'MAJOR': 6298, 'ultra': 6299, 'convenient': 6300, 'SCOR': 6301, 'travellers': 6302, 'tickets': 6303, 'absorb': 6304, 'intent': 6305, 'muscles': 6306, 'Madrid': 6307, 'sensitive': 6308, 'cancer': 6309, 'displayed': 6310, 'ROME': 6311, 'MasterCraft': 6312, 'Engine': 6313, 'athletes': 6314, 'quit': 6315, 'overwhelmingly': 6316, 'rupture': 6317, 'identified': 6318, 'twin': 6319, 'BARC.L': 6320, 'Hague': 6321, 'Vuitton': 6322, 'boepd': 6323, 'bitcoin': 6324, 'Maximum': 6325, 'baht': 6326, '1.98': 6327, '05': 6328, 'IndiGo': 6329, 'ease': 6330, 'square': 6331, 'Gone': 6332, 'colocation': 6333, '5.2': 6334, 'Circular': 6335, 'Sims': 6336, 'Search': 6337, 'Gibson': 6338, 'helicopters': 6339, 'begun': 6340, 'Franklin': 6341, 'Leung': 6342, 'settled': 6343, 'Transocean': 6344, 'Deepwater': 6345, 'OCI': 6346, 'Properties': 6347, 'wrong': 6348, 'unpopular': 6349, 'lawsuits': 6350, 'Ukraine': 6351, 'Vladimir': 6352, 'helicopter': 6353, 'backgrounds': 6354, 'Parts': 6355, 'nuclear': 6356, '69': 6357, 'Miners': 6358, 'Sampoerna': 6359, 'Wire': 6360, 'Though': 6361, 'Anderson': 6362, 'EMV': 6363, 'absence': 6364, 'enacted': 6365, 'tweet': 6366, 'Henry': 6367, 'journalist': 6368, 'Moscow': 6369, '239': 6370, 'writedown': 6371, 'Coles': 6372, 'JB': 6373, 'Mukesh': 6374, '3,500': 6375, 'Hywood': 6376, 'regular': 6377, 'reject': 6378, 'planemaker': 6379, 'questionable': 6380, 'Old': 6381, 'bag': 6382, 'fantasy': 6383, 'Moody': 6384, 'magnitude': 6385, 'QUARTERLY': 6386, 'INTERNATIONAL': 6387, 'Dividend': 6388, 'AGMS': 6389, 'Golden': 6390, 'River': 6391, 'LED': 6392, 'CNN': 6393, 'flows': 6394, 'background': 6395, 'licensed': 6396, 'obtained': 6397, 'Sam': 6398, 'rolled': 6399, 'husband': 6400, 'Dave': 6401, 'NYSEArca': 6402, 'FBGX': 6403, 'amounts': 6404, 'Lazard': 6405, 'Daniel': 6406, 'ambassador': 6407, 'trimmed': 6408, 'Trader': 6409, 'AUTO': 6410, 'TEP': 6411, 'AOWDF': 6412, 'lounge': 6413, 'baggage': 6414, 'wherever': 6415, 'Ashitey': 6416, 'Mile': 6417, '1.50': 6418, 'beverages': 6419, 'shoe': 6420, 'Anadarko': 6421, 'whopping': 6422, 'Forbes': 6423, 'Snapchat': 6424, 'contractors': 6425, 'Age': 6426, 'Interbank': 6427, 'Opera': 6428, 'ports': 6429, 'pump': 6430, 'Timothy': 6431, 'Mayopoulos': 6432, 'Layton': 6433, 'prepare': 6434, 'Keith': 6435, 'fueled': 6436, 'Vista': 6437, 'Vertex': 6438, 'prepares': 6439, 'landlord': 6440, 'Ticker': 6441, 'OTTAWA': 6442, 'pictured': 6443, 'UAE': 6444, 'bin': 6445, 'Peabody': 6446, '1.56': 6447, 'ties': 6448, 'Doll': 6449, 'girl': 6450, '57': 6451, 'diversify': 6452, 'turbulent': 6453, 'Alpha': 6454, 'Hellenic': 6455, 'Bajaj': 6456, 'L&T': 6457, 'seconds': 6458, 'Insider': 6459, 'CLSA': 6460, 'Commenting': 6461, 'vision': 6462, 'waiting': 6463, 'Motorcycle': 6464, 'JRG': 6465, '+1.76': 6466, 'NCC6.BO': 6467, 'corridor': 6468, 'tunnel': 6469, 'ordinary': 6470, 'Clinton': 6471, '14.5': 6472, 'boy': 6473, 'geographical': 6474, 'renamed': 6475, 'Bridge': 6476, '6.8': 6477, 'automatically': 6478, 'Tube': 6479, 'Wolfgang': 6480, 'EUREX': 6481, "O'Leary": 6482, 'Nomad': 6483, 'Tribune': 6484, 'Billion': 6485, 'ship': 6486, 'APC': 6487, '290': 6488, 'fear': 6489, 'suggests': 6490, 'cattle': 6491, 'Truck': 6492, 'Scania': 6493, 'Paccar': 6494, 'glove': 6495, 'crashed': 6496, 'Composite': 6497, 'controversial': 6498, 'fluctuated': 6499, 'wrangle': 6500, 'bitter': 6501, 'RSS': 6502, '1876': 6503, 'weekly': 6504, 'blockbuster': 6505, 'Booking.com': 6506, 'priced': 6507, 'Colu': 6508, 'Ma': 6509, 'Harman': 6510, 'STAN': 6511, 'II': 6512, 'suspend': 6513, 'tonne': 6514, 'requirement': 6515, 'refined': 6516, 'reviewed': 6517, 'resulted': 6518, 'newspapers': 6519, 'Sarah': 6520, 'Mitchell': 6521, 'transfers': 6522, 'Interest': 6523, 'Client': 6524, 'principles': 6525, 'RGLD': 6526, 'Pueblo': 6527, 'Viejo': 6528, 'Ka': 6529, 'shing': 6530, 'kept': 6531, 'unchanged': 6532, 'Gujarat': 6533, 'PayNet': 6534, 'beach': 6535, 'Ken': 6536, 'lobbying': 6537, 'Yet': 6538, 'underscoring': 6539, 'Julie': 6540, 'Rice': 6541, 'indicating': 6542, 'Supreme': 6543, 'Jane': 6544, 'cold': 6545, 'Model': 6546, 'Tender': 6547, 'Early': 6548, 'Microblogging': 6549, 'bug': 6550, 'appear': 6551, 'Monitoring': 6552, 'act': 6553, 'Patriot': 6554, 'Brooks': 6555, 'Delgadillo': 6556, 'infuse': 6557, 'Citadel': 6558, 'Current': 6559, 'lauded': 6560, 'pass': 6561, '+0.45': 6562, 'ITGL': 6563, 'FDI': 6564, 'DELHI': 6565, 'Lavastorm': 6566, '750': 6567, 'Cobalt': 6568, 'refusing': 6569, 'promoters': 6570, 'Shillings': 6571, 'identify': 6572, 'KUALA': 6573, 'LUMPUR': 6574, 'plummeted': 6575, '877': 6576, 'Anil': 6577, 'pitch': 6578, 'Rs.5,000': 6579, 'Hockridge': 6580, 'Huggies': 6581, 'Wipes': 6582, 'Email': 6583, 'island': 6584, '+2.21': 6585, 'esure': 6586, 'Ola': 6587, 'undisclosed': 6588, 'Foxtons': 6589, 'OCDO': 6590, 'ALLWF': 6591, 'NMC.L': 6592, 'NMCX': 6593, 'Ricardo': 6594, 'RCDO': 6595, 'BATM': 6596, 'BTAVF': 6597, 'Piramal': 6598, 'gathering': 6599, 'Majesco': 6600, 'Re': 6601, 'sect': 6602, 'Chadian': 6603, 'Four': 6604, 'Blog': 6605, 'TRK': 6606, 'Bills': 6607, 'Aircraft': 6608, 'Firefox': 6609, 'female': 6610, 'Applications': 6611, 'Ministry': 6612, 'laboratory': 6613, 'edition': 6614, 'revise': 6615, 'Noor': 6616, 'Keio': 6617, 'LaCrosse': 6618, 'PU': 6619, 'Programme': 6620, 'OXIG': 6621, 'CSRT': 6622, 'Helicopters': 6623, 'Kmart': 6624, 'Stutchbury': 6625, 'fold': 6626, 'unfamiliar': 6627, 'transparency': 6628, 'U.S': 6629, 'archive': 6630, 'NPA': 6631, 'Cascada': 6632, 'Independent': 6633, 'Out': 6634, 'sends': 6635, 'Polyus': 6636, 'Petra': 6637, 'vowed': 6638, 'convertible': 6639, 'persuade': 6640, 'climb': 6641, 'Morning': 6642, 'Herald': 6643, 'Storyful': 6644, 'concluded': 6645, 'bicycle': 6646, 'Hero': 6647, 'billed': 6648, 'Dot': 6649, '0.14': 6650, 'SPDR': 6651, '0.33': 6652, 'Letters': 6653, 'El': 6654, '16.9': 6655, 'aided': 6656, 'showcasing': 6657, 'terminal': 6658, 'GM.N': 6659, '4.48': 6660, 'resigned': 6661, 'Bath': 6662, 'metric': 6663, 'Barron': 6664, 'Tchibo': 6665, 'Hole': 6666, 'PAP.L': 6667, 'pumpkin': 6668, 'beers': 6669, 'originating': 6670, 'HSS': 6671, 'Expands': 6672, 'broke': 6673, 'labels': 6674, 'Jonathan': 6675, 'Cultural': 6676, 'GLENCORE': 6677, 'camera': 6678, 'Asus': 6679, '09:00': 6680, 'Aberdeen': 6681, 'themselves': 6682, 'generations': 6683, 'Etihad': 6684, 'capex': 6685, 'lymphoma': 6686, 'burden': 6687, 'bps': 6688, 'brewer': 6689, 'Marshmallow': 6690, 'Miller': 6691, 'MS&AD': 6692, 'AML.L': 6693, 'Lanco': 6694, 'Infratech': 6695, 'Turing': 6696, 'Shkreli': 6697, 'Ingenico': 6698, 'floatation': 6699, 'Aerojet': 6700, 'measurement': 6701, 'proxy': 6702, 'understood': 6703, 'Nielsen': 6704, 'comp': 6705, '1.51': 6706, 'D': 6707, 'iPad': 6708, 'Machinery': 6709, 'weakening': 6710, 'NAHREP': 6711, 'Mi': 6712, 'Casa': 6713, 'Realtors': 6714, 'imbalance': 6715, 'Win': 6716, 'retransmission': 6717, 'Paulson': 6718, 'semiconductor': 6719, 'disastrous': 6720, 'AXP.N': 6721, 'COF.N': 6722, 'DFS.N': 6723, 'neutral': 6724, 'calendar': 6725, 'editions': 6726, 'Docs': 6727, 'editing': 6728, 'Lawrence': 6729, 'KBR': 6730, 'HAL.N': 6731, 'Appeals': 6732, 'ZURICH': 6733, 'withdrew': 6734, 'Alcatel': 6735, 'locked': 6736, '360': 6737, 'OfficeMax': 6738, 'ginseng': 6739, 'winners': 6740, 'discounters': 6741, 'Ross': 6742, 'overseeing': 6743, 'CVX': 6744, 'cookies': 6745, 'protecting': 6746, 'realized': 6747, 'quicker': 6748, 'clusters': 6749, 'entrenched': 6750, 'Secondly': 6751, 'JV': 6752, 'Escalade': 6753, 'Tahoe': 6754, 'execute': 6755, 'CIO': 6756, 'ROI': 6757, 'skilled': 6758, 'accessing': 6759, 'ideas': 6760, 'Telephonics': 6761, 'SAS': 6762, 'fuels': 6763, '2023': 6764, 'Vineyard': 6765, 'smoothly': 6766, 'incorporate': 6767, 'aim': 6768, 'Unipro': 6769, 'bolster': 6770, 'comprises': 6771, 'portal': 6772, 'UC': 6773, 'Baltimore': 6774, 'MedImmune': 6775, 'AstraZeneca': 6776, 'Intelie': 6777, 'Hardware': 6778, 'Gallery': 6779, 'Monica': 6780, 'Goods': 6781, 'Mall': 6782, 'Gucci': 6783, 'relocated': 6784, 'Outlets': 6785, 'Adidas': 6786, 'Falls': 6787, 'manages': 6788, 'physicians': 6789, 'irregular': 6790, 'advancing': 6791, 'conjunction': 6792, 'Scripts': 6793, '3,000': 6794, 'covering': 6795, 'opportunistic': 6796, 'proteins': 6797, 'insightful': 6798, 'bedding': 6799, 'difficulty': 6800, 'Hershey': 6801, 'Cavaliers': 6802, 'NFL': 6803, 'intention': 6804, 'seamless': 6805, 'efficiencies': 6806, 'associates': 6807, 'concept': 6808, 'karaoke': 6809, 'mapping': 6810, 'reposition': 6811, 'Noble': 6812, 'coopetition': 6813, 'structured': 6814, 'nice': 6815, 'amenities': 6816, 'WPT': 6817, 'Perhaps': 6818, '2024': 6819, 'automated': 6820, 'authorizations': 6821, 'journal': 6822, 'reminder': 6823, 'Marvell': 6824, 'TripAdvisor': 6825, 'HP': 6826, 'whereby': 6827, 'LiqTech': 6828, 'silicon': 6829, 'Geely': 6830, '78': 6831, 'auditors': 6832, 'Lymphoid': 6833, 'Irving': 6834, 'Cancer': 6835, 'cornerstone': 6836, 'Language': 6837, 'enter': 6838, 'Singtel': 6839, 'Hynix': 6840, 'Hostess': 6841, 'Directors': 6842, 'ADNOC': 6843, 'checkout': 6844, 'scan': 6845, 'Lion': 6846, 'syndication': 6847, 'internationally': 6848, 'domestically': 6849, 'aggregators': 6850, 'Performance': 6851, 'RPM': 6852, 'Delhaize': 6853, 'Joining': 6854, 'Huntington': 6855, 'Dana': 6856, 'Ana': 6857, 'Within': 6858, 'recognize': 6859, 'pivotal': 6860, 'alert': 6861, 'accuracy': 6862, 'vulnerability': 6863, 'Bilibili': 6864, 'rollout': 6865, 'Pick': 6866, 'Elle': 6867, 'Québec': 6868, 'Similar': 6869, 'Rhopressa': 6870, 'Rocklatan': 6871, 'indications': 6872, 'generic': 6873, 'Clean': 6874, 'congratulations': 6875, 'comparable': 6876, '305.9': 6877, 'Forever': 6878, 'omnichannel': 6879, 'brick': 6880, 'administrative': 6881, 'HR': 6882, 'coach': 6883, 'signals': 6884, 'restoration': 6885, 'contrast': 6886, 'demonstrated': 6887, 'cells': 6888, 'stabilize': 6889, 'priorities': 6890, 'park': 6891, 'retinal': 6892, 'disease': 6893, 'orphan': 6894, 'medicinal': 6895, 'designation': 6896, 'ADX-2191': 6897, 'protocol': 6898, 'exclusivity': 6899, 'collaborated': 6900, 'Church': 6901, '1,200': 6902, 'ends': 6903, 'Buddy': 6904, 'Outlet': 6905, 'Vitamin': 6906, 'Shoppe': 6907, 'OneWeb': 6908, 'Coronation': 6909, 'pause': 6910, 'Hub': 6911, 'creativity': 6912, 'native': 6913, 'charities': 6914, 'Listen': 6915, 'statutory': 6916, 'Regarding': 6917, 'finalized': 6918, 'overload': 6919, 'weighing': 6920, 'Epic': 6921, 'Fortnite': 6922, 'measured': 6923, 'reliability': 6924, 'sequentially': 6925, 'Utilities': 6926, 'PREPA': 6927, 'provisions': 6928, 'Jersey': 6929, 'pent': 6930, 'rebounded': 6931, 'CVS': 6932, 'borne': 6933, 'saved': 6934, 'Storage': 6935, 'weighted': 6936, 'barrel': 6937, 'masking': 6938, 'reception': 6939, 'Beechcraft': 6940, 'fruition': 6941, 'SiriusXM': 6942, 'Pandora': 6943, '407': 6944, 'bricks': 6945, 'commentary': 6946, 'Alteryx': 6947, 'B2C': 6948, 'Douglas': 6949, 'eased': 6950, 'slowly': 6951, 'disaster': 6952, 'hits': 6953, 'Ageas': 6954, 'Fortis': 6955, 'imagine': 6956, 'et': 6957, 'CRE': 6958, 'Heritage': 6959, 'Quarterly': 6960, 'Summary': 6961, 'Build': 6962, 'usage': 6963, '1.0': 6964, 'Routematch': 6965, '2,300': 6966, 'needing': 6967, 'extract': 6968, 'genomic': 6969, 'compression': 6970, 'implications': 6971, 'Healthineers': 6972, 'Reebok': 6973, 'THC': 6974, 'dramatically': 6975, 'interactions': 6976, 'chat': 6977, 'findings': 6978, 'analyses': 6979, 'monitor': 6980, 'swiftly': 6981, 'recommending': 6982, 'Amdocs': 6983, 'chosen': 6984, 'Lease': 6985, 'ourselves': 6986, 'Limoneira': 6987, 'Charitable': 6988, 'impacts': 6989, 'mechanism': 6990, 'equitable': 6991, 'thoughts': 6992, 'ViacomCBS': 6993, 'Roku': 6994, 'Showtime': 6995, 'Safeway': 6996, 'schools': 6997, 'Crave': 6998, 'merchandising': 6999, 'aggregation': 7000, 'Palo': 7001, 'Alto': 7002, 'patterns': 7003, 'rev': 7004, 'CUDA': 7005, 'transformed': 7006, 'Sonos': 7007, 'speaker': 7008, 'sound': 7009, 'beloved': 7010, 'procurement': 7011, 'Pittsburgh': 7012, 'databases': 7013, 'ships': 7014, 'fixtures': 7015, 'represent': 7016, 'CDW': 7017, 'orchestrated': 7018, 'venue': 7019, 'subscriptions': 7020, 'concentrated': 7021, 'walls': 7022, 'Kratos': 7023, 'uniquely': 7024, 'designated': 7025, 'gig': 7026, 'CPUs': 7027, 'complements': 7028, 'underscores': 7029, 'Shot': 7030, '5.9': 7031, 'Wi': 7032, 'offload': 7033, 'boat': 7034, 'therapy': 7035, 'item': 7036, 'negatively': 7037, 'Only': 7038, 'seed': 7039, 'amazing': 7040, 'none': 7041, 'conferences': 7042, 'RBC': 7043, 'Evercore': 7044, 'upper': 7045, 'aside': 7046, 'D.C.': 7047, 'programmers': 7048, 'Charter': 7049, 'Cox': 7050, 'Huntsville': 7051, 'Gondi': 7052, 'attributes': 7053, 'Monjuvi': 7054, 'Pemazyre': 7055, 'Lilly': 7056, 'fridge': 7057, '28th': 7058, 'tons': 7059, 'requested': 7060, 'fixes': 7061, 'Portland': 7062, 'Player': 7063, 'Switch': 7064, 'titles': 7065, 'combines': 7066, 'Roll7': 7067, 'shelf': 7068, 'Clinic': 7069, '1.1': 7070, 'reconciliation': 7071, 'reconcile': 7072, 'AppFolio': 7073, 'solely': 7074, 'Slide': 7075, 'fairness': 7076, 'Merchant': 7077, 'Strategic': 7078, 'Campus': 7079, 'performer': 7080, 'noteworthy': 7081, 'Chin': 7082, '777s': 7083, 'Vueling': 7084, 'TD': 7085, 'Internationally': 7086, 'MAX': 7087, 'echopark.com': 7088, 'outlays': 7089, 'Coupa': 7090, 'automate': 7091, 'unified': 7092, 'undergoing': 7093, 'trials': 7094, 'frankly': 7095, 'totaled': 7096, 'assume': 7097, 'repeating': 7098, 'contingencies': 7099, 'lion': 7100, 'ACCIONA': 7101, 'Giorgio': 7102, 'proper': 7103, 'Acura': 7104, 'adaptive': 7105, 'Wachovia': 7106, 'generators': 7107, 'Camry': 7108, 'KLM': 7109, 'remove': 7110, 'airplanes': 7111, 'pilots': 7112, 'Vinci': 7113, 'Energies': 7114, '79': 7115, '767s': 7116, '190': 7117, 'zinc': 7118, 'virus': 7119, 'intend': 7120, 'PCR': 7121, 'Roberto': 7122, 'Andrea': 7123, 'Anelise': 7124, 'Alberto': 7125, 'Governance': 7126, 'Compliance': 7127, 'divestments': 7128, 'grows': 7129, 'tell': 7130, 'concentrate': 7131, 'conglomerates': 7132, 'upsell': 7133, 'mechanical': 7134, 'baseline': 7135, 'Methanex': 7136, 'specialists': 7137, 'Remastered': 7138, 'critics': 7139, 'migration': 7140, 'Proofpoint': 7141, 'milling': 7142, 'Marfrig': 7143, 'submarkets': 7144, 'Partially': 7145, 'UGG': 7146, 'reset': 7147, 'assistant': 7148, 'boiler': 7149, 'demographic': 7150, 'structurally': 7151, 'anomaly': 7152, 'Fendt': 7153, 'Recovery': 7154, 'direction': 7155, 'stalled': 7156, 'deploy': 7157, 'protocols': 7158, 'filters': 7159, 'shaft': 7160, 'sinking': 7161, 'crew': 7162, 'Infra': 7163, 'towers': 7164, 'tale': 7165, 'parks': 7166, 'xFi': 7167, 'Globally': 7168, 'structural': 7169, 'disposal': 7170, 'Nestlé': 7171, 'cream': 7172, 'firearm': 7173, 'ammunition': 7174, 'tracking': 7175, '230': 7176, 'geographies': 7177, 'lists': 7178, 'Learning': 7179, 'Learn': 7180, 'resonate': 7181, 'obvious': 7182, 'centric': 7183, 'encompass': 7184, 'tremendous': 7185, 'Kazatomprom': 7186, 'dumping': 7187, 'auctioned': 7188, 'Such': 7189, 'limitation': 7190, 'pandemics': 7191, 'Dabir': 7192, 'Surfaces': 7193, 'customary': 7194, 'divestitures': 7195, 'judgments': 7196, 'thereunder': 7197, 'undergone': 7198, 'emerged': 7199, 'define': 7200, 'abroad': 7201, 'adult': 7202, 'LPs': 7203, 'simultaneous': 7204, 'Denso': 7205, 'limiting': 7206, 'Shopify': 7207, 'invasive': 7208, 'clarity': 7209, 'framework': 7210, 'stages': 7211, '98': 7212, '24th': 7213, 'contents': 7214, 'inception': 7215, 'marketplaces': 7216, 'performed': 7217, 'Hungary': 7218, 'originally': 7219, 'cancellation': 7220, 'Advisory': 7221, 'Delaware': 7222, 'consultants': 7223, 'indicate': 7224, 'bright': 7225, 'merely': 7226, 'exhibition': 7227, 'InfoGenesis': 7228, 'OnDemand': 7229, 'Quick': 7230, 'resorts': 7231, 'longtime': 7232, 'Cambridge': 7233, 'payouts': 7234, 'bakeries': 7235, 'bread': 7236, 'compromise': 7237, 'transferring': 7238, 'reaffirmed': 7239, '567': 7240, 'amounting': 7241, 'Peugeot': 7242, 'Corsa': 7243, 'remarkable': 7244, 'Free2Move': 7245, 'PSA': 7246, 'agility': 7247, 'pockets': 7248, 'uncertainties': 7249, 'Opportunities': 7250, 'involve': 7251, 'conflicting': 7252, 'validated': 7253, 'accompanying': 7254, 'educating': 7255, 'Coming': 7256, 'broadly': 7257, 'contributing': 7258, 'balances': 7259, 'migrated': 7260, 'smooth': 7261, 'Defense': 7262, 'resistance': 7263, 'link': 7264, 'foray': 7265, 'Iberia': 7266, 'LATAM': 7267, 'announces': 7268, 'necessarily': 7269, 'workflows': 7270, 'patient': 7271, 'resiliency': 7272, 'evaluate': 7273, 'leases': 7274, 'conveyed': 7275, 'arrested': 7276, 'locate': 7277, 'FrontStage': 7278, 'AI': 7279, 'recordable': 7280, 'injury': 7281, 'accomplishment': 7282, 'Award': 7283, 'parked': 7284, 'spreads': 7285, 'Separately': 7286, 'Hallmark': 7287, 'Christmas': 7288, 'PDP': 7289, 'Covestro': 7290, 'organize': 7291, 'refreshing': 7292, 'conversion': 7293, 'protest': 7294, 'detection': 7295, 'Onanon': 7296, 'relates': 7297, 'surrounding': 7298, 'sounds': 7299, '180': 7300, 'Peterbilt': 7301, 'Hydrogen': 7302, 'emission': 7303, 'Canton': 7304, 'synergistic': 7305, 'albeit': 7306, 'CDN': 7307, 'FY': 7308, 'spanning': 7309, 'Soft': 7310, 'dataset': 7311, 'periods': 7312, 'modularity': 7313, 'trick': 7314, 'Telenor': 7315, 'easing': 7316, 'Cdiscount': 7317, 'PJSC': 7318, 'departments': 7319, 'Export': 7320, 'Could': 7321, 'landfill': 7322, 'animal': 7323, 'choices': 7324, 'exceptional': 7325, 'impressed': 7326, 'Steelcase': 7327, 'parcel': 7328, 'Integration': 7329, 'adjustments': 7330, 'incentive': 7331, 'EMC': 7332, 'Genzyme': 7333, 'personally': 7334, 'bandwidth': 7335, 'unpredictable': 7336, 'careful': 7337, 'Lowe': 7338, 'provision': 7339, 'breadth': 7340, 'contains': 7341, 'mattress': 7342, 'NetSuite': 7343, 'bets': 7344, 'ratification': 7345, 'appointment': 7346, 'landlords': 7347, 'alluded': 7348, 'openings': 7349, 'practical': 7350, 'template': 7351, 'invoice': 7352, 'transitional': 7353, '8.7': 7354, '875': 7355, 'stands': 7356, '2.4': 7357, '110': 7358, 'lovers': 7359, 'productively': 7360, 'underwriting': 7361, 'arisen': 7362, 'Unity': 7363, 'reference': 7364, 'hamburger': 7365, 'Yaouré': 7366, 'Mine': 7367, 'Pure': 7368, 'task': 7369, 'Victorian': 7370, 'Pipeline': 7371, 'drawing': 7372, 'TX': 7373, 'Woodman': 7374, 'format': 7375, 'remind': 7376, 'Movement': 7377, 'Urban': 7378, 'maps': 7379, 'E3': 7380, 'dialogue': 7381, 'LN': 7382, 'Garden': 7383, 'Guangzhou': 7384, 'proceed': 7385, 'ILS': 7386, 'GPUs': 7387, 'acceleration': 7388, 'oriented': 7389, 'WeChat': 7390, '449': 7391, 'QVC': 7392, 'fragrance': 7393, 'Le': 7394, 'fragrances': 7395, 'historically': 7396, 'shipped': 7397, 'recurring': 7398, 'sensor': 7399, 'ecommerce': 7400, '0.85': 7401, '4.4': 7402, 'separately': 7403, 'domination': 7404, 'Corona': 7405, 'touched': 7406, 'strongly': 7407, 'formerly': 7408, '120,000': 7409, 'highlights': 7410, 'marketers': 7411, 'Forrester': 7412, 'Wave': 7413, 'Lead': 7414, 'Ability': 7415, 'Vision': 7416, 'Kingsoft': 7417, 'Winter': 7418, 'Olympics': 7419, 'unaffected': 7420, 'preparation': 7421, 'conversations': 7422, 'CE': 7423, 'applicable': 7424, 'Beach': 7425, 'swift': 7426, 'differently': 7427, 'propelled': 7428, 'VaynerMedia': 7429, 'relaunch': 7430, 'creates': 7431, 'recruit': 7432, 'earning': 7433, 'iGaming': 7434, 'displays': 7435, 'bundled': 7436, 'Revenue': 7437, 'junior': 7438, 'classes': 7439, 'friends': 7440, 'Zhengzhou': 7441, 'Sichuan': 7442, 'blocks': 7443, 'enrich': 7444, 'anniversary': 7445, 'ethanol': 7446, 'columns': 7447, 'USP': 7448, 'installing': 7449, 'tanks': 7450, 'gallons': 7451, 'micro': 7452, 'artificial': 7453, 'Aemetis': 7454, 'repay': 7455, 'predictability': 7456, '777': 7457, 'RF': 7458, 'Ravi': 7459, 'spill': 7460, 'Merger': 7461, 'Producers': 7462, 'Rusal': 7463, 'blasted': 7464, 'swelling': 7465, 'definitive': 7466, 'backer': 7467, 'Davidson': 7468, 'peek': 7469, 'bite': 7470, 'dies': 7471, 'tumor': 7472, 'Writer': 7473, 'Boy': 7474, 'condolences': 7475, 'notes': 7476, 'Wii': 7477, 'stylus': 7478, 'Slovakia': 7479, 'attacks': 7480, 'yields': 7481, 'progressive': 7482, 'rebuffed': 7483, 'CIL': 7484, 'absurd': 7485, 'accident': 7486, 'shale': 7487, 'unconventional': 7488, 'summary': 7489, 'addressing': 7490, 'Living': 7491, 'Sheldon': 7492, 'Kenton': 7493, 'PRESIDENT': 7494, 'EUROPEAN': 7495, 'smelter': 7496, 'supplied': 7497, 'softer': 7498, 'flash': 7499, 'wiped': 7500, 'subprime': 7501, 'Renaissance': 7502, 'recalled': 7503, 'Hummer': 7504, '12.5': 7505, 'perfume': 7506, 'projection': 7507, 'Endeavour': 7508, 'Buy).Want': 7509, 'Keystone': 7510, 'Enquest': 7511, 'carnage': 7512, 'celebrate': 7513, 'Avolon': 7514, 'Bohai': 7515, 'tender': 7516, 'Transaction': 7517, 'SBUX.SW': 7518, 'Sharply': 7519, 'Curtis': 7520, 'Swings': 7521, 'Loss': 7522, 'Drop': 7523, 'Sixth': 7524, 'TSMC': 7525, 'Against': 7526, 'subcompact': 7527, 'slashing': 7528, 'DC': 7529, 'PFSweb': 7530, 'stringent': 7531, 'sweeping': 7532, 'mailed': 7533, 'Dixon': 7534, 'EVR.L': 7535, '9.7': 7536, 'slower': 7537, 'Fit': 7538, 'Interviews': 7539, 'Interview': 7540, 'Fixed': 7541, 'BB': 7542, 'Career': 7543, 'networking': 7544, 'poll': 7545, 'Book': 7546, 'Schuster': 7547, 'von': 7548, 'Vontobel': 7549, 'cybersecurity': 7550, 'weaknesses': 7551, 'tackle': 7552, 'threats': 7553, 'creditor': 7554, '31.4': 7555, '1.75': 7556, 'certificate': 7557, 'committing': 7558, 'NCR': 7559, 'WTB': 7560, 'Millennium': 7561, 'MDG': 7562, 'seventh': 7563, 'Echo': 7564, 'Norsk': 7565, 'Aluminium': 7566, 'Rolled': 7567, 'captive': 7568, '2025': 7569, 'prosecutors': 7570, 'walked': 7571, 'wipe': 7572, 'JD.com': 7573, 'conflict': 7574, 'Highway': 7575, 'recalling': 7576, 'Speciale': 7577, 'FAO': 7578, 'exuberance': 7579, 'pumped': 7580, '16.2': 7581, '15.2': 7582, 'Motley': 7583, 'Fool': 7584, 'Shopo': 7585, 'individuals': 7586, 'Funds': 7587, '\ufeff': 7588, 'Dunkin': 7589, 'gaps': 7590, 'sooner': 7591, 'hanging': 7592, 'destroyed': 7593, 'strategist': 7594, 'NAND': 7595, 'Guinness': 7596, 'Johnnie': 7597, 'whisky': 7598, 'Bangladesh': 7599, 'Schulman': 7600, 'ENUR.DE': 7601, 'Friends': 7602, 'MEGA': 7603, 'theme': 7604, 'Shadow': 7605, 'Agriculture': 7606, 'Joel': 7607, 'Fitzgibbon': 7608, 'hurdles': 7609, 'Volt': 7610, 'BBRYF': 7611, 'Miss': 7612, 'Lynn': 7613, 'MYI': 7614, 'Sells': 7615, 'Datatec': 7616, 'Jens': 7617, 'Montanana': 7618, 'Fairweather': 7619, 'keen': 7620, 'accountancy': 7621, 'lawyers': 7622, 'Ascena': 7623, '11.1': 7624, 'BAFBF': 7625, 'Supergroup': 7626, 'SGP': 7627, 'HAS': 7628, '164.80p': 7629, 'BKG': 7630, 'Mitchells': 7631, 'Butlers': 7632, 'MAB.L': 7633, 'MAB': 7634, 'CHOO.L': 7635, 'CHOO': 7636, 'Oman': 7637, 'elsewhere': 7638, 'fighter': 7639, 'sacking': 7640, 'MAU': 7641, 'expenditure': 7642, 'KOLKATA': 7643, 'Ashok': 7644, 'Ghosh': 7645, 'financier': 7646, 'loved': 7647, 'Generation': 7648, 'vital': 7649, 'siblings': 7650, 'Spotlight': 7651, 'EQIX': 7652, 'REIT': 7653, 'proverbial': 7654, 'iceberg': 7655, 'fintech': 7656, 'UnitedHealth': 7657, 'FTSEurofirst': 7658, 'forthcoming': 7659, 'Mahua': 7660, 'Venkatesh': 7661, '08': 7662, 'Matthew': 7663, 'neighborhood': 7664, 'collaborating': 7665, 'RBS': 7666, 'condemns': 7667, 'misery': 7668, 'advocating': 7669, 'clinch': 7670, 'divided': 7671, 'Test': 7672, 'designer': 7673, 'CFOs': 7674, 'doubts': 7675, 'objective': 7676, '0900': 7677, 'stimulate': 7678, 'Babcock': 7679, 'BLND.L': 7680, 'urging': 7681, 'battled': 7682, 'waves': 7683, '86,000': 7684, 'trades': 7685, 'TSX': 7686, 'Pierre': 7687, 'Gramegna': 7688, 'Eddie': 7689, 'leaseback': 7690, 'deployments': 7691, 'connecting': 7692, 'Cramer': 7693, 'tripled': 7694, 'Release': 7695, 'OMX': 7696, 'genetic': 7697, 'Nelson': 7698, 'Murata': 7699, 'Infineon': 7700, 'Rockwell': 7701, 'Collins': 7702, '25,000': 7703, 'welfare': 7704, 'certainty': 7705, 'disinfectant': 7706, 'Nemours': 7707, 'Reckitt': 7708, 'Benckiser': 7709, 'buyer': 7710, 'A400': 7711, 'transporter': 7712, 'battles': 7713, 'instability': 7714, 'borders': 7715, 'unveil': 7716, '62': 7717, 'attitudes': 7718, 'sacked': 7719, 'handed': 7720, 'transferred': 7721, 'streets': 7722, 'tower': 7723, 'Aa3': 7724, 'rolling': 7725, 'unlimited': 7726, 'stint': 7727, 'Asda': 7728, 'Hortons': 7729, 'practiced': 7730, 'Bachelor': 7731, 'Arts': 7732, 'sits': 7733, 'Karen': 7734, 'processed': 7735, 'boasts': 7736, 'cleared': 7737, 'Sharp': 7738, 'BLOOMBERG': 7739, 'crunch': 7740, 'cranks': 7741, 'Vanadium': 7742, 'Jacob': 7743, 'WHEN': 7744, 'Critics': 7745, 'landfarms': 7746, 'cows': 7747, 'Blackrock': 7748, 'cheap': 7749, 'counts': 7750, 'Isaac': 7751, 'Larian': 7752, 'futures': 7753, 'clears': 7754, 'courses': 7755, 'Sean': 7756, 'crushing': 7757, 'Forecast': 7758, 'Shows': 7759, 'Rising': 7760, 'grip': 7761, 'Wolf': 7762, 'discounted': 7763, 'Estates': 7764, 'Hamilton': 7765, 'Hugo': 7766, 'Boss': 7767, 'territories': 7768, 'CITIC': 7769, 'Willis': 7770, 'Towers': 7771, 'Watson': 7772, 'implied': 7773, 'pie': 7774, 'crowdfunding': 7775, 'Prospecting': 7776, 'arguably': 7777, '13.9': 7778, 'clipped': 7779, 'mismanaged': 7780, 'Stratus': 7781, 'SXT': 7782, 'Exxon': 7783, 'Range': 7784, 'miss': 7785, 'Ferguson': 7786, 'Type': 7787, 'Cnova': 7788, 'Panama': 7789, 'supports': 7790, 'Rich': 7791, 'Their': 7792, 'integrates': 7793, 'checking': 7794, 'errors': 7795, 'BW)(VA': 7796, 'HILTON': 7797, 'WORLDWIDE': 7798, 'Reichshof': 7799, 'Opens': 7800, 'Historic': 7801, 'Joins': 7802, 'Mclean': 7803, 'considerable': 7804, 'wealthiest': 7805, 'Raises': 7806, 'accelerates': 7807, 'shrink': 7808, 'Camera': 7809, 'boys': 7810, 'Days': 7811, 'Channels': 7812, 'STAR': 7813, 'Should': 7814, 'Rise': 7815, 'Radio': 7816, 'FLR': 7817, 'setback': 7818, 'Legal': 7819, 'logged': 7820, 'Boulger': 7821, 'Charcol': 7822, 'encompasses': 7823, 'Etsy': 7824, 'TaskRabbit': 7825, 'Lyft': 7826, 'Centuries': 7827, 'organised': 7828, 'cheered': 7829, '0.06': 7830, 'Islands': 7831, 'MONTREAL': 7832, 'bomb': 7833, 'studying': 7834, 'DFM': 7835, 'ARMX': 7836, 'employing': 7837, 'records': 7838, 'sustained': 7839, 'contracting': 7840, 'layoffs': 7841, 'dismissal': 7842, 'routine': 7843, 'collect': 7844, 'commonly': 7845, 'abilities': 7846, 'Previous': 7847, 'BS': 7848, 'PYPL': 7849, 'sharper': 7850, 'rely': 7851, 'connects': 7852, 'crossroads': 7853, 'Wright': 7854, 'argue': 7855, 'attracting': 7856, 'WEEK': 7857, 'REVENUE': 7858, 'BONDS': 7859, 'AND': 7860, 'SERIES': 7861, 'MGR': 7862, 'SENIOR': 7863, 'HOUSING': 7864, 'INC': 7865, 'earliest': 7866, '10bn': 7867, 'ALTR': 7868, 'laptops': 7869, 'HAGUE': 7870, 'Anika': 7871, 'Toll': 7872, 'Sebring': 7873, 'Convertible': 7874, 'Than': 7875, 'WH': 7876, 'stellar': 7877, 'supermarkets': 7878, 'favourite': 7879, 'sun': 7880, 'Escondida': 7881, 'chapters': 7882, 'windfall': 7883, 'VZ': 7884, 'threw': 7885, 'Vittorio': 7886, 'Colao': 7887, 'vs.': 7888, 'receding': 7889, 'Academy': 7890, 'flexible': 7891, 'Med': 7892, 'holidays': 7893, 'Steven': 7894, 'scholar': 7895, 'theory': 7896, 'calculate': 7897, 'arrival': 7898, 'driverless': 7899, 'Knorr': 7900, 'Hellmann': 7901, 'A0NC7B': 7902, 'APAC': 7903, 'Akzo': 7904, 'Paints': 7905, 'PPG': 7906, 'Valspar': 7907, 'Jotun': 7908, 'Altima': 7909, '325,000': 7910, 'Daniels': 7911, 'Midland': 7912, 'Phenomenex': 7913, 'Soccer': 7914, 'DALLAS': 7915, 'Announces': 7916, 'Kosmos': 7917, 'Teacher': 7918, '650': 7919, 'metre': 7920, 'blades': 7921, '380': 7922, 'pulled': 7923, 'catalog': 7924, 'TATA': 7925, 'Vistara': 7926, 'A-320': 7927, 'cart': 7928, 'arrived': 7929, 'derailment': 7930, 'Germanwings': 7931, 'Innes': 7932, 'Willox': 7933, 'bridges': 7934, 'restrict': 7935, '8,000': 7936, 'Mongolia': 7937, 'Hans': 7938, 'reiterated': 7939, 'Travis': 7940, 'Perkins': 7941, 'Hammerson': 7942, 'HMSO': 7943, 'ISAT.L': 7944, 'Mondi': 7945, 'MNDI': 7946, '+1.16': 7947, '+1.14': 7948, '+1.10': 7949, '-2.04': 7950, 'JMAT.L': 7951, 'JMAT': 7952, 'RRS': 7953, 'EZJ': 7954, '1,649.00p': 7955, 'CA2': 7956, 'menus': 7957, 'Crenn': 7958, 'LON': 7959, 'Processing': 7960, 'Paypal': 7961, 'Explorer': 7962, 'US4': 7963, 'fortunes': 7964, 'leftists': 7965, 'TRADING': 7966, 'Electrocomponents': 7967, 'Figures': 7968, 'Yellow': 7969, 'Heath': 7970, 'veto': 7971, 'MPC': 7972, 'chaired': 7973, 'suggestions': 7974, '260': 7975, '280': 7976, 'thumping': 7977, 'conclusion': 7978, 'dated': 7979, 'showcase': 7980, 'Fraser': 7981, 'Hyde': 7982, '113': 7983, 'Krugman': 7984, 'collapsed': 7985, 'Chi': 7986, 'Manila': 7987, 'Minas': 7988, 'Turner': 7989, 'Wetherspoon': 7990, 'pub': 7991, 'Zambia': 7992, 'Cameroon': 7993, 'Dov': 7994, 'Charney': 7995, 'benchmarks': 7996, 'vocally': 7997, 'objected': 7998, 'highway': 7999, 'anyway': 8000, 'Unigroup': 8001, 'garage': 8002, 'entrepreneurial': 8003, '13.49': 8004, 'Tea': 8005, 'institution': 8006, 'GDP': 8007, 'GNA': 8008, 'MV': 8009, 'voyage': 8010, 'Graham': 8011, 'witnesses': 8012, 'steepest': 8013, 'Mission': 8014, 'Somalia': 8015, 'MANILA': 8016, 'Aboitiz': 8017, 'formalising': 8018, 'Philippine': 8019, 'dark': 8020, 'overhaul': 8021, 'Furious': 8022, 'organisations': 8023, 'slumping': 8024, 'rein': 8025, 'iHub': 8026, 'Investec': 8027, 'Jon': 8028, 'ISR': 8029, 'conventional': 8030, 'KLD': 8031, 'wings': 8032, 'weather': 8033, 'Look': 8034, 'somebody': 8035, 'Tristan': 8036, 'brew': 8037, 'Product': 8038, 'fan': 8039, 'promoted': 8040, 'intraday': 8041, 'Bombay': 8042, 'Qualcomm': 8043, 'schemes': 8044, 'Goodyear': 8045, 'Aristocrat': 8046, 'Leisure': 8047, 'Gryphon': 8048, 'Incitec': 8049, 'Pivot': 8050, 'Amcor': 8051, 'BlueScope': 8052, 'Cue': 8053, 'PacBrands': 8054, 'Sandfire': 8055, 'Tabcorp': 8056, 'Monadelphous': 8057, 'iSelect': 8058, 'Dexus': 8059, 'Resmed': 8060, 'TFS': 8061, 'NAB': 8062, 'Centres': 8063, 'Alexium': 8064, 'Eclipx': 8065, 'Independence': 8066, 'Alumina': 8067, 'Mesoblast': 8068, 'AWE': 8069, 'Fleetwood': 8070, 'Otto': 8071, 'McPherson': 8072, 'Yowie': 8073, 'Virtus': 8074, 'Spotless': 8075, 'Slater': 8076, 'Gordon': 8077, 'UGL': 8078, 'Sirtex': 8079, 'Ardent': 8080, 'BWP': 8081, 'ALS': 8082, 'DuluxGroup': 8083, 'Woodside': 8084, 'Programmed': 8085, 'Barbara': 8086, 'WOW': 8087, '\u200b': 8088, 'DS': 8089, 'Doug': 8090, 'input': 8091, 'Are': 8092, 'Worth': 8093, '240': 8094, 'Shopping': 8095, 'Syria': 8096, 'dangerous': 8097, 'Thai': 8098, 'Mazumdar': 8099, 'Shaw': 8100, 'plagued': 8101, 'predict': 8102, 'LCC': 8103, 'Chico': 8104, '85': 8105, 'Rubenstein': 8106, 'predictions': 8107, 'divorce': 8108, 'LT3': 8109, 'champions': 8110, '2003': 8111, '1bn': 8112, 'letting': 8113, 'Pet': 8114, 'Brother': 8115, 'Hanjin': 8116, '+3.43': 8117, 'MGGT': 8118, 'Elsevier': 8119, 'RUKEF': 8120, '+0.66': 8121, 'BATS': 8122, 'Wimpey': 8123, 'outgoing': 8124, 'rat': 8125, 'animals': 8126, 'surprisingly': 8127, 'caught': 8128, 'snake': 8129, 'bone': 8130, 'Apollo': 8131, 'Leon': 8132, 'Rothwell': 8133, 'Travel': 8134, 'wire': 8135, 'attorney': 8136, 'examining': 8137, 'Jess': 8138, 'illegal': 8139, 'unfair': 8140, '2pc': 8141, 'Leftist': 8142, 'biological': 8143, 'Labco': 8144, '4744.TWO': 8145, 'FR0010220475': 8146, 'Cuba': 8147, 'LUXEMBOURG': 8148, 'parliaments': 8149, 'Tony': 8150, 'Opposition': 8151, 'marginal': 8152, 'Gap': 8153, 'pharmacy': 8154, 'Corcoran': 8155, 'Sotheby': 8156, 'ZipRealty': 8157, 'Conn.': 8158, 'Geneva': 8159, 'finalize': 8160, 'headquartered': 8161, 'batteries': 8162, 'nil': 8163, 'VLKAY': 8164, 'honour': 8165, 'broadcasters': 8166, 'Com': 8167, 'upfront': 8168, 'reckons': 8169, 'Gary': 8170, 'Rex': 8171, 'flagging': 8172, 'Loeb': 8173, 'Vivint': 8174, 'Standards': 8175, 'revolution': 8176, 'Eden': 8177, 'EUROGROUP': 8178, 'JEROEN': 8179, 'DIJSSELBLOEM': 8180, 'YHOO': 8181, 'Highfield': 8182, 'Gerard': 8183, 'Sterling': 8184, 'contagion': 8185, 'th': 8186, '0.25': 8187, '3.25': 8188, 'assumed': 8189, 'Solidscape': 8190, 'Canal': 8191, 'applicants': 8192, 'acquires': 8193, '20p': 8194, '18.38': 8195, 'techMARK': 8196, '-0.38': 8197, '+4.08': 8198, '+1.86': 8199, '+1.75': 8200, '+1.42': 8201, 'TUI': 8202, 'Shs': 8203, 'Pakistan': 8204, 'slowed': 8205, '+0.46': 8206, '+0.32': 8207, '+0.31': 8208, '-4.03': 8209, '-3.37': 8210, '-2.66': 8211, 'ANTO': 8212, '-2.47': 8213, 'FRES': 8214, '394.50p': 8215, '+2.71': 8216, 'BET.L': 8217, 'Factory': 8218, '+1.98': 8219, 'Marston': 8220, 'Aldermore': 8221, 'ALD.L': 8222, 'ALD': 8223, '+1.50': 8224, 'TalkTalk': 8225, '-2.36': 8226, 'Initial': 8227, 'Petrofac': 8228, 'PFC': 8229, '54.25p': 8230, 'DRX.L': 8231, '-2.02': 8232, 'TIG': 8233, 'Stockholm': 8234, '+0.27': 8235, 'XPP.L': 8236, 'participation': 8237, 'alleged': 8238, 'Reporting': 8239, 'Potter': 8240, 'Sharecast': 8241, 'Neil': 8242, 'Fool.co.uk': 8243, 'quotes': 8244, 'exporter': 8245, 'axe': 8246, 'geared': 8247, 'promotions': 8248, 'Amatil': 8249, 'MINISTER': 8250, 'Return': 8251, 'doubtful': 8252, 'BA.N': 8253, 'stem': 8254, 'researcher': 8255, 'Jason': 8256, 'Policy': 8257, 'Alice': 8258, 'MarketWatch': 8259, 'forum': 8260, 'censorship': 8261, 'angry': 8262, 'Ellen': 8263, 'ran': 8264, 'Ask': 8265, 'drag': 8266, 'VS': 8267, 'Forte': 8268, 'Trend': 8269, 'attends': 8270, 'Exor': 8271, 'pioneer': 8272, 'explanation': 8273, 'chose': 8274, 'amongst': 8275, 'Brexit': 8276, 'Garners': 8277, 'ForexClear': 8278, 'yielded': 8279, 'TrailBlazer': 8280, 'EXT': 8281, 'LT': 8282, 'Sable': 8283, 'evident': 8284, 'CMA': 8285, 'writes': 8286, 'forcing': 8287, 'reshuffle': 8288, 'Whether': 8289, 'cheer': 8290, 'worried': 8291, 'hammer': 8292, 'Chanticleer': 8293, 'PR': 8294, 'multibillion': 8295, '4554.TWO': 8296, 'JAZTF': 8297, 'influence': 8298, 'SBS': 8299, 'TPP': 8300, 'Cooperation': 8301, 'granular': 8302, 'trademarks': 8303, 'Lucasfilm': 8304, 'Ashmore': 8305, 'Responsibilities': 8306, '9.2': 8307, 'interact': 8308, 'Create': 8309, 'manuals': 8310, 'Statements': 8311, 'Provide': 8312, 'GL': 8313, 'KK': 8314, '9.1': 8315, 'AR': 8316, 'Knowledge': 8317, 'andor': 8318, '1,500': 8319, 'richer': 8320, '20.1': 8321, 'broadening': 8322, 'upheaval': 8323, 'rallying': 8324, 'stance': 8325, 'unexpected': 8326, 'Durkin': 8327, 'McKinsey': 8328, 'Geelong': 8329, '8.72': 8330, 'horrifying': 8331, 'mega': 8332, 'prudential': 8333, 'mortgages': 8334, 'issuing': 8335, 'redeem': 8336, 'broken': 8337, 'gloom': 8338, 'debuted': 8339, 'hampered': 8340, 'Advent': 8341, 'FTSE-100': 8342, 'flotation': 8343, '+2.07': 8344, '0.6': 8345, 'thin': 8346, 'multinational': 8347, 'linking': 8348, 'involves': 8349, 'painful': 8350, 'Update': 8351, 'Summer': 8352, 'Athabasca': 8353, 'drill': 8354, 'Mann': 8355, 'Ram': 8356, '7.85': 8357, 'risers': 8358, 'Newport': 8359, 'Cardiff': 8360, 'Anselmo': 8361, 'Luxury': 8362, 'ING': 8363, 'UFJ': 8364, 'Mellon': 8365, 'SpA': 8366, 'Keefe': 8367, 'Bruyette': 8368, 'Woods': 8369, 'Nicolaus': 8370, 'Clubs': 8371, 'Hertz': 8372, 'travelers': 8373, 'Ferrero': 8374, 'Self': 8375, 'Managed': 8376, 'Reid': 8377, 'VCs': 8378, 'entertaining': 8379, 'Gazal': 8380, 'shake': 8381, 'PVH': 8382, 'flood': 8383, 'blog': 8384, 'Photos': 8385, 'arrange': 8386, 'denominated': 8387, 'Automation': 8388, 'ESPN': 8389, 'soccer': 8390, 'men': 8391, 'flags': 8392, '4.2': 8393, 'Keller': 8394, 'handled': 8395, 'contaminated': 8396, 'smelting': 8397, 'congressional': 8398, 'youth': 8399, 'Antonis': 8400, 'Samaras': 8401, 'leftwing': 8402, 'Haryana': 8403, '300th': 8404, '):': 8405, 'aerospace': 8406, 'inability': 8407, 'ventures': 8408, 'proceedings': 8409, 'investigations': 8410, 'prove': 8411, 'TM': 8412, 't': 8413, 'Rome': 8414, 'Fitzgerald': 8415, 'considers': 8416, 'Marketplace': 8417, 'acts': 8418, 'Utah': 8419, 'Trans': 8420, 'dependent': 8421, 'Scott': 8422, 'explorer': 8423, 'AFR.L': 8424, 'Electrolux': 8425, 'divisional': 8426, 'curated': 8427, '-0.62': 8428, '45': 8429, 'Hugh': 8430, 'eye': 8431, '04.00AM': 8432, 'Boris': 8433, 'BK': 8434, '8.2': 8435, 'warranty': 8436, 'televisions': 8437, 'Fesseha': 8438, 'Globo': 8439, 'MADP': 8440, 'AIM': 8441, 'OTCQX': 8442, 'geothermal': 8443, 'subsidies': 8444, 'Rajendran': 8445, 'Nanyang': 8446, '1949': 8447, 'Representatives': 8448, 'Enter': 8449, 'refinancing': 8450, 'SoFi': 8451, 'finding': 8452, 'Tag': 8453, 'Heuer': 8454, 'shaking': 8455, 'placement': 8456, 'CMCSA': 8457, 'VIA': 8458, '-0.77': 8459, '-1.76': 8460, 'compatible': 8461, 'MAKATI': 8462, '0700': 8463, 'Prix': 8464, 'Kid': 8465, 'Contest': 8466, 'peace': 8467, 'programmes': 8468, 'extends': 8469, 'counting': 8470, 'Neighborly': 8471, 'carmakers': 8472, 'Kazakhstan': 8473, 'dirhams': 8474, '425': 8475, 'impress': 8476, 'Vanke': 8477, 'awaiting': 8478, 'cholesterol': 8479, 'Amgen': 8480, 'counterbalances': 8481, 'assessment': 8482, 'participate': 8483, 'Assistive': 8484, 'Richmond': 8485, 'Holmes': 8486, 'spun': 8487, 'MRO': 8488, 'Around': 8489, 'judgment': 8490, 'invaluable': 8491, 'flaws': 8492, 'skidded': 8493, 'colossus': 8494, '180.50': 8495, 'Robin': 8496, 'invests': 8497, 'BYD': 8498, 'GWh': 8499, 'Chem': 8500, 'Users': 8501, 'reveal': 8502, 'attending': 8503, 'Grid': 8504, 'Agnico': 8505, 'Eagle': 8506, 'Struggling': 8507, '867.50p': 8508, '128.50p': 8509, 'Fankhauser': 8510, 'Connect': 8511, 'Nicholas': 8512, 'Gresham': 8513, 'Pamela': 8514, 'Cri': 8515, 'CDF': 8516, 'Husqvarna': 8517, 'Inventory': 8518, '06:56': 8519, 'dovish': 8520, 'shakeup': 8521, 'Costs': 8522, 'FAA': 8523, 'Fortus': 8524, 'prototyping': 8525, 'Azim': 8526, 'Premji': 8527, 'surges': 8528, 'Lupin': 8529, 'guide': 8530, 'Avago': 8531, 'slips': 8532, 'CNBC.com': 8533, '11:42': 8534, 'Helios': 8535, 'AC': 8536, 'Himani': 8537, 'Chandna': 8538, 'cola': 8539, 'acknowledged': 8540, 'sufficient': 8541, 'controversy': 8542, 'studies': 8543, 'surveillance': 8544, 'Norway': 8545, '5.0': 8546, 'Saab': 8547, 'precinct': 8548, 'felony': 8549, 'conviction': 8550, 'crotch': 8551, 'trained': 8552, 'Doha': 8553, 'Goyder': 8554, 'sustain': 8555, 'Bobby': 8556, 'Yakub': 8557, 'Memon': 8558, 'judges': 8559, 'dismissed': 8560, 'plea': 8561, 'slid': 8562, 'IPG': 8563, 'Inclusive': 8564, 'Talent': 8565, 'Octagon': 8566, 'Diverse': 8567, 'Ed': 8568, 'Tallis': 8569, 'Editor': 8570, 'TrueCar': 8571, 'stole': 8572, 'Elementis': 8573, 'credits': 8574, 'Brad': 8575, 'Bit': 8576, 'Ryan': 8577, 'Ethereum': 8578, 'exists': 8579, 'printers': 8580, 'Navy': 8581, 'plunge': 8582, 'horizon': 8583, 'jubilant': 8584, 'Front': 8585, 'Groups': 8586, 'Vienna': 8587, 'NBD': 8588, 'dampen': 8589, 'shedding': 8590, 'mounting': 8591, 'bubble': 8592, 'black': 8593, 'Mobily': 8594, 'weathered': 8595, 'Past': 8596, 'Organisation': 8597, 'Springs': 8598, 'tighter': 8599, 'playlists': 8600, 'Net': 8601, 'lets': 8602, 'notebook': 8603, 'ISRO': 8604, 'Vehicle': 8605, '1980': 8606, 'nearby': 8607, 'ceremony': 8608, 'downbeat': 8609, 'buyback': 8610, 'marine': 8611, 'Domestic': 8612, 'Between': 8613, 'FOMC': 8614, 'Dove': 8615, 'shower': 8616, 'odds': 8617, 'WA': 8618, 'chaebol': 8619, 'sprawling': 8620, 'grouped': 8621, 'realms': 8622, 'semiconductors': 8623, 'OBG': 8624, '59': 8625, 'corporations': 8626, 'Caliber': 8627, 'COMPANIES': 8628, 'REPORTING': 8629, 'LAND.L': 8630, 'ST': 8631, '0800': 8632, 'Anywhere': 8633, 'surprised': 8634, 'brief': 8635, 'stoppage': 8636, 'Stability': 8637, 'Mechanism': 8638, 'ESM': 8639, 'performers': 8640, 'SDI': 8641, 'Tianjin': 8642, 'Howell': 8643, 'grower': 8644, 'publications': 8645, 'MakerBot': 8646, 'drugmaker': 8647, '675213': 8648, 'Energia': 8649, 'presenting': 8650, 'Ilmor': 8651, 'bedroom': 8652, 'Parents': 8653, 'university': 8654, 'students': 8655, 'microblogging': 8656, 'adorable': 8657, 'cat': 8658, 'Bridavsky': 8659, '200,000': 8660, 'Tingyi': 8661, 'ten': 8662, 'COST': 8663, 'Rightmove': 8664, 'interbank': 8665, 'levy': 8666, 'bowing': 8667, 'heights': 8668, 'Starting': 8669, '4.6': 8670, 'AMSTERDAM': 8671, 'stiff': 8672, 'maligned': 8673, 'Jeremy': 8674, 'syndicated': 8675, 'shed': 8676, 'popularised': 8677, '1960s': 8678, 'Lorem': 8679, 'Ipsum': 8680, 'overwhelming': 8681, 'lodged': 8682, 'shortages': 8683, 'skill': 8684, 'MoZaic': 8685, 'foot': 8686, 'Uptown': 8687, 'brewed': 8688, 'emailed': 8689, 'Hunting': 8690, 'Batman': 8691, 'BMI': 8692, 'fish': 8693, 'canned': 8694, 'centred': 8695, 'end-2019': 8696, 'Louise': 8697, 'accelerator': 8698, 'promising': 8699, 'ESUR': 8700, 'steadily': 8701, 'Saracen': 8702, 'Consolidation': 8703, 'Templeton': 8704, 'MUDRA': 8705, 'prevents': 8706, 'Kinross': 8707, 'Permira': 8708, 'Ghosn': 8709, 'Too': 8710, 'RBA': 8711, 'Pearce': 8712, 'tests': 8713, 'warnings': 8714, 'imposed': 8715, 'froze': 8716, 'initiated': 8717, 'remedies': 8718, 'Mayor': 8719, 'OL.HK': 8720, 'totalled': 8721, 'incorporating': 8722, 'pollution': 8723, 'Friedman': 8724, '1970s': 8725, 'capitalize': 8726, 'poverty': 8727, 'Ways': 8728, 'chances': 8729, 'EDF': 8730, 'COPENHAGEN': 8731, '3i': 8732, 'Esvagt': 8733, 'Brattleboro': 8734, 'Gardner': 8735, 'Savings': 8736, 'Keen': 8737, 'worse': 8738, 'designers': 8739, 'Diane': 8740, 'image': 8741, 'frame': 8742, 'pair': 8743, 'institutional': 8744, 'FY16': 8745, 'ACA.L': 8746, 'complained': 8747, 'continuity': 8748, 'GOP': 8749, 'justice': 8750, 'marriages': 8751, 'Indonesian': 8752, 'HM': 8753, 'Tbk': 8754, 'comply': 8755, 'Contact': 8756, 'shelved': 8757, 'Island': 8758, 'reshaped': 8759, 'practice': 8760, 'bidders': 8761, 'explore': 8762, 'brace': 8763, 'relinquish': 8764, 'cautiously': 8765, 'Domestos': 8766, 'Tips': 8767, 'affecting': 8768, 'Accounts': 8769, '00:54': 8770, 'Officeworks': 8771, 'Sebi': 8772, 'relax': 8773, '1990s': 8774, 'Coors': 8775, 'Ale': 8776, 'Kiev': 8777, 'attempts': 8778, 'Brugal': 8779, 'rum': 8780, 'informal': 8781, 'waters': 8782, 'Conlumino': 8783, 'Muilenburg': 8784, 'influential': 8785, 'generational': 8786, 'SAGR.UL': 8787, 'patriarch': 8788, 'Kun': 8789, 'consolidate': 8790, 'PN': 8791, '10.7': 8792, 'diversion': 8793, 'employ': 8794, 'consideration': 8795, 'conservatives': 8796, 'vigilant': 8797, 'supporters': 8798, 'Bundy': 8799, 'master': 8800, 'Borghetti': 8801, 'nemesis': 8802, 'blocked': 8803, 'Cooperative': 8804, 'sewing': 8805, 'harassment': 8806, 'centimetres': 8807, 'kit': 8808, 'INTERIM': 8809, 'PCC': 8810, 'Claims': 8811, '07:00': 8812, 'Unemployment': 8813, 'Average': 8814, 'FINALS': 8815, 'Amati': 8816, 'Anpario': 8817, 'Cadogan': 8818, 'Hon': 8819, 'Hai': 8820, 'GDR': 8821, 'Chemical': 8822, 'Ohanian': 8823, 'WINNIPEG': 8824, 'Manitoba': 8825, 'finish': 8826, 'potash': 8827, 'confidential': 8828, 'intra': 8829, 'Brookfield': 8830, 'Operating': 8831, '2bn': 8832, 'rating': 8833, 'believing': 8834, 'conquer': 8835, 'traders': 8836, 'sat': 8837, 'Lowell': 8838, 'McAdam': 8839, 'Animation': 8840, 'Every': 8841, 'Sox': 8842, 'Alley': 8843, 'heavyweights': 8844, 'IMI': 8845, '+1.55': 8846, 'ETO': 8847, 'CKN': 8848, '+1.35': 8849, '333.60p': 8850, 'presentations': 8851, 'allowance': 8852, 'Accepted': 8853, 'Adjei': 8854, 'Ecobank': 8855, 'PwC': 8856, 'Framework': 8857, 'injunction': 8858, 'FP.SW': 8859, 'MBDA': 8860, 'Finmeccanica': 8861, 'Evan': 8862, 'Garland': 8863, 'treasury': 8864, 'ally': 8865, 'KCC': 8866, 'impose': 8867, 'Official': 8868, 'village': 8869, 'resort': 8870, 'SEWA': 8871, 'McNamee': 8872, 'popularity': 8873, 'Archie': 8874, 'Hesse': 8875, 'Settlement': 8876, 'GhIPSS': 8877, 'Cerro': 8878, 'length': 8879, 'stick': 8880, 'oversight': 8881, 'Skechers': 8882, 'MGI': 8883, 'criticised': 8884, 'faith': 8885, 'steer': 8886, 'Ronald': 8887, 'backers': 8888, 'forms': 8889, 'salaries': 8890, 'leaner': 8891, 'facto': 8892, 'Wallis': 8893, 'Dale': 8894, 'Meredith': 8895, 'surging': 8896, 'Capacity': 8897, 'Alfred': 8898, 'Anna': 8899, 'cinema': 8900, 'turnover': 8901, 'Sees': 8902, 'Pace': 8903, 'JANEIRO': 8904, 'PETR4.SA': 8905, 'VRTX': 8906, 'cystic': 8907, 'fibrosis': 8908, 'Barnes': 8909, '+0.97': 8910, '-1.32': 8911, 'Stronger': 8912, 'EBay': 8913, 'Bonds': 8914, 'Krzanich': 8915, 'brainchild': 8916, 'Vishal': 8917, 'Sikka': 8918, 'Stanford': 8919, 'BREAKING': 8920, 'NEWS': 8921, 'FactSet': 8922, '1.43': 8923, '1.67': 8924, 'BAC': 8925, 'BLK': 8926, '+1.57': 8927, 'proposing': 8928, 'postage': 8929, 'grams': 8930, '4.0': 8931, 'Belfort': 8932, 'His': 8933, 'AXA': 8934, 'Metlife': 8935, 'LIC': 8936, 'upstream': 8937, 'fares': 8938, 'choose': 8939, 'boutiques': 8940, 'skittish': 8941, 'SAO': 8942, 'PAULO': 8943, 'widebody': 8944, 'penetrate': 8945, 'WCFCB': 8946, 'suitable': 8947, 'legislation': 8948, 'disclosures': 8949, 'spots': 8950, 'Telecommunications': 8951, 'BHEL': 8952, 'veterans': 8953, '64': 8954, 'Classic': 8955, 'RDSB.L': 8956, 'Stensholt': 8957, 'Daewoo': 8958, 'Shipbuilding': 8959, 'shipbuilding': 8960, 'immensely': 8961, 'FDIC': 8962, 'AML': 8963, 'Claycomo': 8964, 'Mrs': 8965, 'Atul': 8966, 'UBM.L': 8967, 'ETI.L': 8968, 'HGG.L': 8969, 'HGG': 8970, 'Synergy': 8971, 'SYHEF': 8972, 'SYR': 8973, '-4.13': 8974, '159.00p': 8975, '-2.11': 8976, 'UCW': 8977, 'Sepura': 8978, 'SEPU': 8979, '+0.11': 8980, 'RM.L': 8981, 'mental': 8982, 'century': 8983, '3.2': 8984, 'jumping': 8985, 'watchdog': 8986, 'Businesses': 8987, 'Planned': 8988, '14.3': 8989, '13.8': 8990, 'Reprint': 8991, 'Motorways': 8992, 'toll': 8993, 'Motorway': 8994, 'leg': 8995, 'Viking': 8996, 'Gate': 8997, 'Reitzle': 8998, '504482.EX': 8999, 'Gottesman': 9000, 'continental': 9001, 'Findus': 9002, 'TechCrunch': 9003, 'Acquisitions': 9004, 'YES': 9005, 'Polo': 9006, 'Kehoe': 9007, 'demonstrates': 9008, 'Barry': 9009, 'releasing': 9010, 'explain': 9011, 'incurred': 9012, 'handles': 9013, 'Weill': 9014, 'Cornell': 9015, 'Australians': 9016, 'distance': 9017, 'codeshare': 9018, 'Berlusconi': 9019, 'Fininvest': 9020, 'biotechnology': 9021, 'Novozymes': 9022, 'possession': 9023, 'Tarun': 9024, 'Samant': 9025, 'Ramanan': 9026, 'CAMS': 9027, 'Repository': 9028, 'Behara': 9029, 'Irdai': 9030, 'convenor': 9031, 'indicator': 9032, 'rubber': 9033, 'antivirus': 9034, 'illness': 9035, 'Rashtriya': 9036, 'Swayamsevak': 9037, 'Sangh': 9038, 'Ebola': 9039, 'journey': 9040, 'Vima': 9041, 'workplace': 9042, 'bargaining': 9043, 'IIT': 9044, 'Railway': 9045, 'neglect': 9046, 'granting': 9047, 'Jenner': 9048, 'considerably': 9049, 'Synthomer': 9050, 'Colored': 9051, 'Coins': 9052, 'Startup': 9053, 'Beta': 9054, '7.25': 9055, 'Xenophon': 9056, 'reserve': 9057, 'Calderon': 9058, '2888.HK': 9059, 'Publisher': 9060, 'Hours': 9061, 'Date': 9062, 'Minutes': 9063, '408': 9064, 'lagging': 9065, '426': 9066, 'guaranteeing': 9067, 'Identify': 9068, 'Qualifications': 9069, 'starter': 9070, 'Saga': 9071, 'Redefine': 9072, 'RDI': 9073, '+1.97': 9074, 'Moser': 9075, 'Baer': 9076, 'recover': 9077, 'litres': 9078, 'stops': 9079, 'Many': 9080, 'separated': 9081, 'quietly': 9082, 'milking': 9083, '2.60': 9084, '2,976': 9085, 'Expect': 9086, 'earmarked': 9087, 'inject': 9088, 'Raleigh': 9089, 'CSI': 9090, 'Later': 9091, 'Vitol': 9092, 'GNPC': 9093, 'movement': 9094, 'Macdonald': 9095, 'Jake': 9096, 'receipts': 9097, 'Checking': 9098, 'Fee': 9099, 'hoc': 9100, 'Dominican': 9101, 'owning': 9102, 'Peñasquito': 9103, 'recoveries': 9104, 'Whampoa': 9105, 'merging': 9106, 'Micromax': 9107, 'incubating': 9108, 'ancillaries': 9109, 'assemblers': 9110, 'Apart': 9111, 'spur': 9112, 'locals': 9113, 'Cindy': 9114, 'automakers': 9115, 'Paxton': 9116, 'hop': 9117, 'J.C.': 9118, 'Penney': 9119, 'pledge': 9120, 'renegotiate': 9121, 'Fridman': 9122, 'LetterOne': 9123, '47.9': 9124, 'scrapping': 9125, 'eleventh': 9126, 'relying': 9127, 'Newman': 9128, 'forget': 9129, 'timetable': 9130, 'JAKARTA': 9131, 'Konica': 9132, 'Minolta': 9133, 'Ricoh': 9134, 'Foster': 9135, 'Cabinet': 9136, 'Edinburgh': 9137, 'Salt': 9138, 'IBISWorld': 9139, 'arch': 9140, 'Handlin': 9141, 'instinct': 9142, 'Advance': 9143, 'tone': 9144, 'Shin': 9145, 'Questions': 9146, 'Ethiopian': 9147, 'flag': 9148, 'signaled': 9149, 'complementary': 9150, 'Synchrony': 9151, 'drought': 9152, 'rain': 9153, 'Ackah': 9154, 'CSIRO': 9155, 'validly': 9156, 'Offer': 9157, 'Interfax': 9158, 'league': 9159, 'HI': 9160, 'FI': 9161, 'Georgia': 9162, 'Southpaw': 9163, 'Weinstein': 9164, 'stemming': 9165, 'Bing': 9166, 'MOU': 9167, 'Vikings': 9168, 'Vanessa': 9169, 'Solid': 9170, 'emails': 9171, 'indictment': 9172, 'Similarly': 9173, 'ALSO': 9174, 'READ': 9175, 'Drinkers': 9176, 'Desktop': 9177, 'Gadgets': 9178, 'achievements': 9179, 'Cancelling': 9180, 'Maintenance': 9181, 'Vendors': 9182, 'friendly': 9183, 'sink': 9184, 'yourself': 9185, 'dealmaking': 9186, 'dissent': 9187, 'hardliners': 9188, '2.1': 9189, '-1.01': 9190, '+0.50': 9191, 'Maverick': 9192, '+0.44': 9193, 'eastern': 9194, 'western': 9195, 'super': 9196, 'speeds': 9197, '99': 9198, 'Please': 9199, 'trafficking': 9200, 'Transfer': 9201, 'Commodities': 9202, 'CVX.SW': 9203, 'Fred': 9204, '350,000': 9205, 'MindStream': 9206, 'convincing': 9207, 'Franco': 9208, 'BIRLACORPN.BO': 9209, '692': 9210, 'politicians': 9211, 'bargain': 9212, '73,424': 9213, 'roof': 9214, 'describe': 9215, 'reputed': 9216, 'Bankers': 9217, 'dues': 9218, 'suddenly': 9219, 'liabilities': 9220, 'deliberate': 9221, 'Survey': 9222, 'Impact': 9223, 'Srinivasan': 9224, 'Malaysian': 9225, '1MDB': 9226, 'cardholders': 9227, 'Kenyans': 9228, 'Schwab': 9229, 'Cap': 9230, '/p': 9231, 'compiled': 9232, 'choosing': 9233, 'Rafale': 9234, '9.9': 9235, 'Unibanco': 9236, 'PB': 9237, 'SME': 9238, 'Oriental': 9239, '20¢': 9240, 'Ackman': 9241, 'Mondelez': 9242, 'oilfield': 9243, 'Schlumberger': 9244, 'halting': 9245, 'Abbey': 9246, 'crowd': 9247, 'gloves': 9248, 'Sonadih': 9249, 'Jharkhand': 9250, 'Plans': 9251, 'advertised': 9252, '640': 9253, 'Sue': 9254, 'Books': 9255, 'Liberum': 9256, 'appeared': 9257, 'Pixar': 9258, 'consternation': 9259, 'washed': 9260, '222.20p': 9261, '116.80p': 9262, '+5.98': 9263, 'Bovis': 9264, 'BVS': 9265, '+2.23': 9266, '108.30p': 9267, 'height': 9268, 'Convention': 9269, 'redevelopment': 9270, 'accord': 9271, 'pile': 9272, 'Statistics': 9273, 'Promethean': 9274, 'ire': 9275, 'GIB': 9276, 'Talks': 9277, 'upwards': 9278, 'superior': 9279, 'dipped': 9280, '+0.88': 9281, '965.00p': 9282, '-1.24': 9283, '580001.BO': 9284, '-0.85': 9285, '+4.29': 9286, 'FOXT.L': 9287, 'OneSavings': 9288, '2OS.SG': 9289, 'OSB': 9290, 'OCDDY': 9291, '453.00p': 9292, 'ERM': 9293, '+3.45': 9294, '+2.68': 9295, '+1.90': 9296, '+0.30': 9297, '-3.54': 9298, 'Large': 9299, 'FN6.AS': 9300, 'topping': 9301, 'Louie': 9302, 'Douvis': 9303, 'buoyant': 9304, 'Constellation': 9305, 'Mastek': 9306, 'TPMS': 9307, 'mainland': 9308, 'Investing': 9309, 'boats': 9310, 'coast': 9311, '+1.82': 9312, '-4.58': 9313, '6.25p': 9314, 'deficit': 9315, 'HPR': 9316, 'Passes': 9317, 'Container': 9318, 'Educational': 9319, 'Bandung': 9320, 'complaining': 9321, 'Mozilla': 9322, 'laptop': 9323, 'anytime': 9324, '70,000': 9325, 'petrol': 9326, '16th': 9327, 'fortnight': 9328, 'NRL': 9329, 'DIB': 9330, 'VR': 9331, 'GRE': 9332, 'Fitch': 9333, 'Hosts': 9334, 'Joy': 9335, 'Photography': 9336, 'Newspapers': 9337, 'delaying': 9338, 'Lindsey': 9339, 'Piegza': 9340, 'Pontiac': 9341, 'defence': 9342, 'Defence': 9343, 'Panjab': 9344, 'Chandigarh': 9345, 'Wars': 9346, '+1.20': 9347, 'Zoopla': 9348, 'WI': 9349, 'ZPLA': 9350, 'HOME': 9351, '157.80p': 9352, '-2.64': 9353, '113.30p': 9354, '-2.58': 9355, '-2.53': 9356, 'Dialight': 9357, 'DIA': 9358, 'DI6.AS': 9359, 'alleges': 9360, 'Phosagro': 9361, 'Latham': 9362, 'hated': 9363, 'Inspectors': 9364, 'coordination': 9365, 'REFS': 9366, 'internally': 9367, 'Accounting': 9368, 'Confirms': 9369, 'Final': 9370, 'FXall': 9371, 'Operational': 9372, 'Procedures': 9373, 'UBGXF': 9374, 'ranks': 9375, 'pressed': 9376, 'blamed': 9377, 'dump': 9378, 'PAC': 9379, 'candidate': 9380, 'conclude': 9381, 'cup': 9382, 'unexpectedly': 9383, 'Charlottesville': 9384, 'GAAP': 9385, '234': 9386, 'museum': 9387, 'Films': 9388, 'Appoints': 9389, 'Paulo': 9390, 'Inside': 9391, 'Madonna': 9392, 'Pope': 9393, 'cycles': 9394, 'mulls': 9395, '65.24': 9396, 'seismic': 9397, 'declare': 9398, 'Hiranandani': 9399, 'brother': 9400, 'breather': 9401, 'ripples': 9402, 'drone': 9403, 'witnessing': 9404, '524.50p': 9405, '-3.79': 9406, 'kill': 9407, 'wait': 9408, 'Synlab': 9409, 'FDX.SW': 9410, '/deadline': 9411, 'A657': 9412, 'condition': 9413, 'backlash': 9414, 'necessity': 9415, 'pulling': 9416, 'Sandisk': 9417, 'SNDK': 9418, '-1.39': 9419, '+1.37': 9420, 'BIIB': 9421, 'Seagate': 9422, 'CAMI': 9423, 'himself': 9424, 'greatest': 9425, 'quell': 9426, 'seal': 9427, 'Eldorado': 9428, 'industrials': 9429, 'sitting': 9430, 'hands': 9431, 'Qaeda': 9432, 'Osama': 9433, 'Laden': 9434, 'slows': 9435, 'Adds': 9436, 'Narayana': 9437, 'Murthy': 9438, 'Anand': 9439, 'BUENOS': 9440, 'AIRES': 9441, 'Amaya': 9442, 'Hosted': 9443, 'forged': 9444, 'Electricity': 9445, 'vigilance': 9446, 'inch': 9447, 'USB': 9448, 'DisplayPort': 9449, '0.00': 9450, 'CPIF': 9451, 'correct': 9452, 'Polymetal': 9453, 'AUCOY': 9454, 'Regus': 9455, '-4.33': 9456, '-3.35': 9457, 'workshop': 9458, 'Interior': 9459, 'Pant': 9460, 'unprecedented': 9461, '2.508': 9462, 'matches': 9463, 'mooted': 9464, 'Seiko': 9465, 'SII': 9466, 'LDO': 9467, 'S-19311': 9468, 'TO-252': 9469, '5S': 9470, 'dissipation': 9471, 'spare': 9472, 'Cycles': 9473, 'notch': 9474, 'immigration': 9475, 'Wisconsin': 9476, 'equipped': 9477, 'Principal': 9478, 'knocked': 9479, 'TORONTO': 9480, '409': 9481, '219': 9482, '0.35': 9483, 'Hedged': 9484, 'Galactic': 9485, 'informed': 9486, '2WD': 9487, '550': 9488, 'absolute': 9489, 'storm': 9490, 'GST': 9491, 'meaningful': 9492, 'wars': 9493, 'MGM': 9494, 'Mixpanel': 9495, 'wellness': 9496, 'guarantee': 9497, 'evade': 9498, 'kyat': 9499, 'Myanmar': 9500, 'rebounds': 9501, 'jumps': 9502, 'forex': 9503, '4.9': 9504, 'Plastics': 9505, 'PLAS': 9506, 'IRESS': 9507, 'obtain': 9508, 'butter': 9509, 'Ritter': 9510, 'Candies': 9511, 'Confectionery': 9512, 'Everest': 9513, 'Enclave': 9514, 'Sell': 9515, 'JMC': 9516, 'SHANGHAI': 9517, '114': 9518, 'US100': 9519, 'RC': 9520, 'Bhargava': 9521, 'Mohammed': 9522, 'Omar': 9523, 'NWSAL': 9524, 'Encore': 9525, 'weakness': 9526, 'Spun': 9527, 'auctioneer': 9528, 'copies': 9529, 'businessmen': 9530, 'Cencosud': 9531, 'Abboud': 9532, 'Rights': 9533, 'complying': 9534, 'hacking': 9535, 'restricted': 9536, 'unresolved': 9537, 'roaster': 9538, 'Trucks': 9539, 'tasked': 9540, 'emissions': 9541, 'Dhan': 9542, 'Yojana': 9543, 'Rosneft': 9544, 'Israeli': 9545, 'Slow': 9546, 'assault': 9547, 'Chengdu': 9548, 'province': 9549, 'Ashe': 9550, 'Darwin': 9551, 'dry': 9552, 'escort': 9553, 'Rentboy.com': 9554, 'MolsonCoors': 9555, 'authorized': 9556, 'Eurex': 9557, 'integral': 9558, 'Clear': 9559, 'accurate': 9560, 'Railcar': 9561, 'Unit': 9562, 'Envoy': 9563, 'Denali': 9564, 'Kuwait': 9565, 'TRI': 9566, 'Stagecoach': 9567, 'WPP': 9568, 'reversal': 9569, 'cheated': 9570, 'Savills': 9571, 'slots': 9572, 'Seafolly': 9573, 'Birds': 9574, 'takeovers': 9575, 'Mack': 9576, 'GROUP': 9577, 'corporation': 9578, 'Basic': 9579, 'Each': 9580, 'Senator': 9581, 'RTL': 9582, 'Amit': 9583, 'Sood': 9584, 'Bonus': 9585, '32kgs': 9586, '23kgs': 9587, 'IDC': 9588, 'GOOGL': 9589, 'beverage': 9590, 'WAM': 9591, 'Spillers': 9592, 'Bloomfield': 9593, 'iPod': 9594, 'Euromonitor': 9595, 'focuses': 9596, 'Delivering': 9597, 'IoT': 9598, 'Taiwanese': 9599, 'ZenFone': 9600, 'Initiates': 9601, 'Common': 9602, 'Repurchase': 9603, 'Indigo': 9604, 'Epicor': 9605, 'G4S': 9606, 'Prf': 9607, 'Jardine': 9608, 'GMS': 9609, 'Registrations': 9610, 'Move': 9611, 'Naked': 9612, 'Booth': 9613, 'guided': 9614, 'imagery': 9615, 'blood': 9616, 'Ektefaa': 9617, 'containers': 9618, 'log': 9619, 'PDL': 9620, '-2.97': 9621, 'accessible': 9622, 'AirBnb': 9623, 'fundraising': 9624, 'earmarks': 9625, 'Rs.60,000': 9626, 'girding': 9627, 'Essar': 9628, 'Budweiser': 9629, 'LogMeIn': 9630, 'complexities': 9631, 'Sling': 9632, 'secondary': 9633, 'railway': 9634, 'NMDC': 9635, 'addresses': 9636, 'OnePlus': 9637, 'flyers': 9638, 'ABI': 9639, 'Pacifica': 9640, 'FXCM': 9641, 'dismisses': 9642, 'AUC': 9643, 'Dlamini': 9644, 'UNECA': 9645, 'Lopes': 9646, 'Addis': 9647, 'Ababa': 9648, 'Session': 9649, '6.0': 9650, '5X': 9651, 'sea': 9652, 'Skanska': 9653, 'Piech': 9654, 'keyboard': 9655, 'simulcast': 9656, 'leveraged': 9657, 'Strategy': 9658, '5.34': 9659, 'Ark': 9660, 'Gail': 9661, 'N.C.': 9662, 'Colo.': 9663, 'LONDON--(BUSINESSWIRE)--': 9664, 'IGL': 9665, 'FIIs': 9666, 'reacted': 9667, 'Rock': 9668, 'DUBLIN': 9669, 'Highlights': 9670, '-3.58': 9671, 'OXGGY': 9672, 'unsubscribe': 9673, 'competes': 9674, 'Ajai': 9675, 'Puri': 9676, 'VALE': 9677, 'BTU': 9678, 'Teck': 9679, 'TCK': 9680, 'Arch': 9681, 'Daraprim': 9682, 'strip': 9683, 'Homegrown': 9684, 'Hyderabad': 9685, 'Kalanick': 9686, 'Salesforce.com': 9687, 'Benioff': 9688, '09/29/2015': 9689, 'PDT': 9690, 'severe': 9691, 'U.S.-listed': 9692, 'replays': 9693, 'A0B7FY': 9694, 'canceled': 9695, 'memorandum': 9696, 'Xs': 9697, 'projections': 9698, 'obligation': 9699, 'contacting': 9700, 'officers': 9701, 'deemed': 9702, 'solicitation': 9703, 'Certain': 9704, 'Instant': 9705, '-0.08': 9706, 'drew': 9707, 'ING.SW': 9708, 'Expeditions': 9709, 'bold': 9710, 'motorbike': 9711, 'eyes': 9712, 'CREDAI': 9713, 'bedrooms': 9714, '827': 9715, 'damaging': 9716, 'IFA': 9717, 'Novatek': 9718, 'Shuns': 9719, 'Opting': 9720, 'Brewer': 9721, 'MillerCoors': 9722, 'satisfaction': 9723, 'subcommittee': 9724, 'Abdullah': 9725, 'ACCC': 9726, 'Maureen': 9727, 'Hinton': 9728, 'clothes': 9729, 'Converters': 9730, 'A6': 9731, 'TFSI': 9732, '45.90': 9733, '4.59': 9734, '128,650': 9735, 'loads': 9736, 'emotional': 9737, 'Trucks/': 9738, 'Construction/': 9739, 'Rentals': 9740, '-4.6': 9741, '-3.4': 9742, 'FMC': 9743, '-3.2': 9744, '-3.1': 9745, 'Emory': 9746, 'Campbell': 9747, 'onto': 9748, 'Eurowings': 9749, 'NZ': 9750, 'Editorial': 9751, 'obesity': 9752, 'polls': 9753, 'Bottling': 9754, 'sponsors': 9755, 'Latina': 9756, 'Carmen': 9757, 'Mercado': 9758, 'Franchise': 9759, 'multicultural': 9760, 'Martinez': 9761, 'Alicia': 9762, 'Trevino': 9763, 'Crisil': 9764, 'Trai': 9765, 'hurry': 9766, 'Naik': 9767, 'Fremantle': 9768, 'Dockers': 9769, '24.9': 9770, 'Marshall': 9771, 'Wace': 9772, 'Kennett': 9773, 'Inn': 9774, 'Zhaoxu': 9775, 'fingers': 9776, 'airwaves': 9777, 'autonomous': 9778, 'Newsom': 9779, 'Warriors': 9780, 'Festus': 9781, 'Ezeli': 9782, 'Greifeld': 9783, 'Aust': 9784, 'remotely': 9785, 'AGCO': 9786, 'ABS': 9787, 'Liberal': 9788, 'Lorna': 9789, 'shoulder': 9790, 'Confirmit': 9791, 'Edison': 9792}
  if relation_to_index=='default':
    relation_to_index={'product_or_material_produced': 0, 'manufacturer': 1, 'distributed_by': 2, 'industry': 3, 'position_held': 4, 'original_broadcaster': 5, 'owned_by': 6, 'founded_by': 7, 'distribution_format': 8, 'headquarters_location': 9, 'stock_exchange': 10, 'currency': 11, 'parent_organization': 12, 'chief_executive_officer': 13, 'director_or_manager': 14, 'owner_of': 15, 'operator': 16, 'member_of': 17, 'employer': 18, 'chairperson': 19, 'platform': 20, 'subsidiary': 21, 'legal_form': 22, 'publisher': 23, 'developer': 24, 'brand': 25, 'business_division': 26, 'location_of_formation': 27, 'creator': 28}

  flipped_relation = {value: key for key, value in relation_to_index.items()} #flip the relation_to_index for searching by index
  X_seq=[]
  for i in range(len(X)):
    sentence = X[i]
    sequence = []  # Convert sentence to sequence of word indices
    for word in sentence.split():
      if word not in ['.',',','(', ')', '-',"%", "'", ':']:
        if word in word_to_index:
            sequence.append(word_to_index[word])
        else:
            sequence.append(0)  # Use 0 for out-of-vocabulary words
    X_seq.append(sequence)
  X_seq = pad_sequences(X_seq, maxlen=MAX_SEQ_LENGTH)
  X_seq = tf.cast(X_seq, tf.float32)
  y=model(X_seq)

  threshold = 0.5
  y = tf.where(y > threshold, 1, 0)

  result=[]
  for each_prediction in y:
    for i,digit in enumerate(each_prediction):
      if digit==1:
        result.append(flipped_relation[i])

  return result



['chief_executive_officer']


## Predictor usage

In [ ]:
model= tf.saved_model.load('./batch_size=8;epoch=14;num_NN=128;num_Dense=64')

X=['Adam Smith recently appointed Tim Cook as the Chief Baking Officer for his apple pie business.']
result = predict_with_model(model,X)
print(result)

In [ ]:
X_train_pos1,X_train_pos2,X_train_pos3=mark_positions(X_train,train_POE_list)
X_test_pos1,X_test_pos2,X_test_pos3=mark_positions(X_test,test_POE_list)

In [ ]:
  # Padding sequences to ensure uniform length
X_train_pos1 = pad_sequences(X_train_pos1, maxlen=MAX_SEQ_LENGTH)
X_train_pos2 = pad_sequences(X_train_pos2, maxlen=MAX_SEQ_LENGTH)
X_train_pos3 = pad_sequences(X_train_pos3, maxlen=MAX_SEQ_LENGTH)

X_test_pos1 = pad_sequences(X_test_pos1, maxlen=MAX_SEQ_LENGTH)
X_test_pos2 = pad_sequences(X_test_pos2, maxlen=MAX_SEQ_LENGTH)
X_test_pos3 = pad_sequences(X_test_pos3, maxlen=MAX_SEQ_LENGTH)

In [ ]:
#Define Bi-LSTM model with position embedding
optimizer = tf.keras.optimizers.Adam()

#https://medium.com/southpigalle/simple-relation-extraction-with-a-bi-lstm-model-part-2-52086246c66
embed_layer = Embedding(VOCABULARY_SIZE, EMBEDDING_DIM)
input_seq = Input(shape=(MAX_SEQ_LENGTH,))
embed_seq = embed_layer(input_seq)

input_seq_pos1 = Input(shape=(X_train_pos1.shape[1],))
embed_seq_pos1 = Embedding(MAX_SEQ_LENGTH,EMBEDDING_DIM,input_length=MAX_SEQ_LENGTH,trainable=True)(input_seq_pos1)
# input_seq_pos2 = Input(shape=(X_train_pos2.shape[1],))
# embed_seq_pos2 = Embedding(MAX_SEQ_LENGTH,EMBEDDING_DIM,input_length=MAX_SEQ_LENGTH,trainable=True)(input_seq_pos2)
# input_seq_pos3 = Input(shape=(X_train_pos3.shape[1],))
# embed_seq_pos3 = Embedding(MAX_SEQ_LENGTH,EMBEDDING_DIM,input_length=MAX_SEQ_LENGTH,trainable=True)(input_seq_pos2)



# x = tf.concat([embed_seq,embed_seq_pos1, embed_seq_pos2,embed_seq_pos3],1)
x = tf.concat([embed_seq,embed_seq_pos1],1)
x = Bidirectional(LSTM(128,dropout=0.7))(x)
preds = Dense(NUM_CLASSES,activation="softmax")(x)
# model = Model(inputs=[input_seq,input_seq_pos1,input_seq_pos2,input_seq_pos3], outputs=preds)
model = Model(inputs=[input_seq,input_seq_pos1], outputs=preds)

  # Compile model
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 907)]                0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 907)]                0         []                            
                                                                                                  
 embedding_3 (Embedding)     (None, 907, 300)             3568500   ['input_4[0][0]']             
                                                                                                  
 embedding_4 (Embedding)     (None, 907, 300)             272100    ['input_5[0][0]']             
                                                                                            

In [ ]:

# Train model

model.fit([X_train, X_train_pos1], y_train, batch_size=4, epochs=5, validation_data=([X_test,X_test_pos1], y_test))



Epoch 1/5
1425/1425 [==============================] - 175s 121ms/step - loss: 0.1649 - accuracy: 0.2098 - val_loss: 0.1685 - val_accuracy: 0.1695
Epoch 2/5
1425/1425 [==============================] - 150s 105ms/step - loss: 0.1601 - accuracy: 0.2146 - val_loss: 0.1675 - val_accuracy: 0.1695
Epoch 3/5
1425/1425 [==============================] - 148s 104ms/step - loss: 0.1600 - accuracy: 0.2070 - val_loss: 0.1686 - val_accuracy: 0.1695
Epoch 4/5
1425/1425 [==============================] - 145s 102ms/step - loss: 0.1592 - accuracy: 0.2075 - val_loss: 0.1660 - val_accuracy: 0.1498
Epoch 5/5
1425/1425 [==============================] - 148s 104ms/step - loss: 0.1584 - accuracy: 0.2137 - val_loss: 0.1682 - val_accuracy: 0.1760


In [ ]:
# #Define LSTM model
# def grid_search(batch_size,epochs,dropout):
#   optimizer = tf.keras.optimizers.Adam()
#   model = Sequential()
#   model.add(Embedding(len(word_to_index) + 1, output_dim=EMBEDDING_DIM, input_shape=(MAX_SEQ_LENGTH,)))
#   model.add(Bidirectional(LSTM(128, dropout=dropout)))
#   model.add(Dense(64, activation='relu'))

#   model.add(Dense(NUM_CLASSES, activation='softmax'))

#     # Compile model
#   model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#   # Train model

#   model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

# batch_size_list=[2,8,16,64]
# epochs=6
# dropout_list=[0,0.2,0.4,0.6,0.8]

# for each_batch_size in batch_size_list:
#   for each_dropout in dropout_list:
#     print("batch size: {}\ndropout: {}".format(each_batch_size,each_dropout))
#     grid_search(each_batch_size,epochs,each_dropout)


In [ ]:
y_test_predicted=model.predict(X_test)
y_test_predicted = tf.convert_to_tensor(y_test_predicted)
loss = tf.keras.losses.categorical_crossentropy(y_test, y_test_predicted)

In [ ]:
X_train